In [232]:
import pandas as pd
pd.set_option('display.max_colwidth', 700)
import numpy as np
import re

import requests
import json
# from pandas.io.json import json_normalize
from pandas import json_normalize

# 경고 메시지를 무시
import warnings
warnings.filterwarnings(action='ignore')

import pickle

In [233]:
# 영양제 데이터셋 기존 작업 버전 불러오기
with open('..//data//recom_product_dataset_fin_240515_v4.pkl', 'rb') as f:
    cleaning_p_df = pickle.load(f)

In [234]:
print(len(cleaning_p_df)) # 32946 행

32946


## 영양제 데이터셋 불러오기(CSV)

In [235]:
p_df = pd.read_csv('..//data//product_v2.csv')
print(len(p_df)) # RAW 데이터 총 36792 행


# 데이터셋 전처리 
# # 주된기능성 컬럼 값이 결측치일 경우 해당 행 제거
# drop_mask_a = p_df['주된기능성'].isnull()
# drop_index_list = p_df[drop_mask_a].index.tolist()
# print(len(drop_index_list)) # 81행

# # 품목명 컬럼에 '수출용' 문자열이 있을 경우 해당 행 제거
# drop_mask_b = p_df['품목명'].str.contains('수출용')
# print(len(p_df[drop_mask_b])) # 1902 행

# drop_index_list.extend(p_df[drop_mask_b].index.tolist())

# # 품목명 컬럼에 '원료성' 문자열이 있을 경우 해당 행 제거
# drop_mask_c = p_df['품목명'].str.contains('원료성')
# print(len(p_df[drop_mask_c])) # 9행

# drop_index_list.extend(p_df[drop_mask_c].index.tolist())

# print(len(drop_index_list)) # 제거할 행 총 1992 행 확인

# # 제거할 행 인덱스가 담긴 리스트를 이용해 제거
# p_df.drop(index=drop_index_list, inplace=True)
# p_df.reset_index(drop=True)

# print(len(p_df)) # 36792 행 => 34802 행으로 제거됨

36792


### 필요한 컬럼만 가져오기

In [236]:
# 품목제조번호가 cleaning_p_df 와 일치하는 행만 가져오기
p_df = p_df[p_df['품목제조번호'].isin(cleaning_p_df['product_id'].tolist())]
p_df.reset_index(drop=True)

# 필요한 컬럼만 가져오기
p_df = p_df[['품목제조번호', '주된기능성', '섭취시주의사항', '품목유형(기능지표성분)', '기능성 원재료']]

# 컬럼명 변경
p_df.columns = ['product_id', 'product_function', 'product_caution', 'ingredient_type1', 'ingredient_type2']

p_df.head()

,product_id,product_function,product_caution,ingredient_type1,ingredient_type2
0,200400200021640,혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.,"임산부, 수유부, 청소년 및 어린이는 섭취를 피하십시오.\n심혈관약 등 복용, 혈액 응고장애가 있거나 수술예정인 사람은 전문가와 상담하십시오. \n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n본 제품은 섭취자의 신체상태에 따라 반응에 차이가 있을 수 있습니다.\n임산부, 수유부, 질병으로 인해 치료중인 분은 섭취 전 고객센터나 구입처로 문의하십시오.\n특정 성분에 알레르기 체질이신 분은 섭취 전에 반드시 원료(성분)를 확인하신 후 섭취하시기 바랍니다.\n섭취량 및 섭취방법을 확인 한 후 섭취하시고, 기타 제품관련사항은 고객센터로 문의해 주십시오.",폴리코사놀-사탕수수 왁스알코올(2006-4),폴리코사놀-사탕수수왁스알코올(기능성원료인정제2006-4호)
2,201100200015,①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n2) 특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.\r\n 3) 15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.\r\n 4) 제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의 하십시오.",홍삼제품,홍삼추출 농축액(농축추출물)
3,20040015100230,⑴ 면역력 증진에 도움을 줄 수 있음\n ⑵ 피로개선에 도움을 줄 수 있음\n ⑶ 혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n ⑷ 기억력 개선에 도움을 줄 수 있음\n ⑸ 항산화에 도움을 줄 수 있음,"⑴ 특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n⑵ 제품 개봉 또는 섭취 시 포장재에 의한 상처에 주의하시기 바랍니다.\n⑶ 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의 하십시오.\n⑷ 유통기한이 경과된 제품은 섭취하지 마십시오.",홍삼,"홍삼, 6년근(고시형)"
8,200400150831149,[EPA 및 DHA 함유유지] 혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음.\n[비타민E] 유해산소로부터 세포를 보호하는데 필요,알레르기 체질이신 경우 성분을 확인 한 후 섭취하시기 바랍니다.,"비타민E, EPA 및 DHA 함유 유지","D-알파-토코페롤(고시형), EPA및DHA함유유지(고시형)"
9,200400200072802,[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌,고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.,비타민 D,건조효모(비타민D)


## 텍스트에서 제거할 특수문자 분석

In [237]:
from data_processing import find_special_chars, find_chars_df

# 전처리가 필요한 특수문자 확인 - 함수 활용
function_special_chars = find_special_chars(p_df, 'product_function')
caution_special_chars = find_special_chars(p_df, 'product_caution')

# 두 컬럼에 있는 특수문자 리스트 합치기
special_cahrs = function_special_chars + caution_special_chars

# 리스트 내 중복된 문자열 제거
special_cahrs = sorted(list(set(special_cahrs)))

print(special_cahrs)

['"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '[', ']', '_', '`', '{', '}', '~', '\xad', '®', '±', '·', 'ß', '˙', '·', 'Β', 'β', '–', '‘', '’', '“', '”', '•', '․', '‧', '※', '₂', '℃', '™', 'Ⅱ', '∙', '∼', '⋅', '①', '②', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '⑪', '⑫', '⑬', '⑭', '⑴', '⑵', '⑶', '⑷', '⑸', '⑹', '⑺', '⑻', 'Ⓡ', 'ⓛ', '⓵', '⓶', '⓷', '⓻', '⓼', '■', '○', '●', '➀', '➁', '➂', '➃', '➅', '➆', '⦁', '〔', '〕', '〮', '・', 'ㅇ', 'ㅍ', 'ㅡ', 'ㆍ', '㈜', '㉮', '㉯', '㉰', '㉱', '\ufeff', '）', '，', '．', '／', '１', '５', '７', '：', '･', '𐩐']


In [238]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['\"'])[['product_function', 'product_caution']] # '"' 기호는 제거

,product_function,product_caution
0,"[폴리덱스트로스] ①배변활동 원활에 도움을 줄 수 있음\n\n[아연] ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n[자일로올리고당] ""장내 유익균 증식, 유해균 억제, 배변활동에 도움을 줄 수 있음""(기타기능 Ⅱ)",반드시 충분한 물과 함께 섭취할 것 (액상 제외)
1,"[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\t\t\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\t\t\n[나이아신]①체내 에너지 생성에 필요\t\t\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요\t\t\n""[망간]①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 \n세포를 보호하는데 필요""\t\t\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\t\t\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\t\t\n[비타민B1]①탄수화물과 에너지 대사에 필요\t\t\n[비타민B2]①체내 에너지 생성에 필요","(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
2,"""건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n (생리활성기능 2등급)""\n\n①유해산소로부터 세포를 보호하는데 필요","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오\n의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담하십시오.\n개인에 따라 피부 관련 이상반응이 발생할 수 있습니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
3,"[칼슘]\n① 뼈와 치아 형성에 필요\n② 신경과 근육 기능 유지에 필요\n③ 정상적인 혈액응고에 필요\n골다공증발생 위험 감소에 도움을 줌\t\n[마그네슘]\n① 에너지 이용에 필요\n② 신경과 근육 기능 유지에 필요""\t\t\n[비타민D]\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌""",① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\t\t\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
4,"[피브로인추출물BF-7] ""기억력 개선에 도움을 줄 수 있음""","섭취 시 목에 걸리거나 불편할 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오. \n임산부 및 수유부는 섭취를 피하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
5,"[비타민D]\n칼슘과 인이 흡수되고 이용되는데 필요\n뼈의 형성과 유지에 필요\n골다공증발생 위험 감소에 도움을 줌\n\n[아연]\n정상적인 면역기능에 필요\n정상적인 세포분열에 필요\n\n[프로폴리스추출물]\n""항산화 ․ 구강에서의 항균작용에 도움을 줄 수 있음\n※구강에서의 항균작용은 구강에 직접 접촉할 수 있는 형태에 한하며, 섭취량을 적용하지 않음 """,[비타민D]\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n[프로폴리스추출물]\n프로폴리스에 알레르기를 나타내는 사람은 섭취에 주의\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
6,"""체지방 감소에 도움을 줄 수 있습니다"" (기타기능II)","1. 어린이, 임산부 및 수유부는 섭취를 피할 것.\n2. 질환이 있거나 의약품 복용시 전문가와 상담할 것.\n3. 카페인을 함유한 식품의 섭취에 주의할 것.\n4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것.\n5. 특이체질, 알레르기체질의 경우 성분을 확인하신 후 섭취하십시오.\n6. 용기 개봉 후에는 반드시 뚜껑을 닫아서 보관하십시오.\n7. 개봉 또는 섭취 시 절단면에 의해 상처를 입을 수 있으니 주의하십시오."
7,"①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n①탄수화물과 에너지 대사에 필요\n\n①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n①체내 에너지 생성에 필요\n\n①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\n\n①체내 에너지 생성에 필요\n\n①유해산소로부터 세포를 보호하는데 필요\n\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n①갑상선 호르몬의 합성에 필요②에너지 생성에 필요③신경발달에 필요\n\n①유해산소로부터 세포를 보호하는데 필요\n\n①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n\n①철의 운반과 이용에 필요②유해산소로부터 세포를 보호하는데 필요\n\n①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌","1) 알레르기 반응이 나타나는 경우에는 섭취를 중단하시기 바랍니다.\n2) 유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n3) 어린이, 임산부 및 수유부는 섭취를 피하시기 바랍니다.\n4) 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취에 주의하시기 바랍니다.\n5) 간, 신장, 심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다.\n6) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다."""
8,(테아닌)스트레스로 인한 긴장완화에 도움을 줄 수 있음\n(비타민B6)단백질 및 아미노산 이용에 필요 /혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n(나이아신)체내 에너지 생성에 필요,"1. 카페인 함유 음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하십시오.\n2. 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하시기 바랍니다\n3. 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다\n4. 섭취량 및 섭취 방법을 준수하시기 바랍니다\n5. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다\n6. 소비기한을 확인하시기 바라며, 소비기한이 지난 제품은 섭취하지 마시기를 바랍니다\n7. 제품을 개봉할 때 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하십시오\n8. ""원료 특성상 침전물이 생길 수 있으나, 제품에 이상이 있는 것은 아니므로 안심하고 잘 흔들어 드십시오"""
9,(국문) 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)\n(영문) May help reduce body fat mass(Other function claim 2 grade)\n\n-기억력 개선·혈행 개선에 도움을 줄 수 있음,"""`- 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의\n- 의약품(항응고제) 복용 시 섭취에 주의\n- 영,유아, 어린이, 임산부 및 수유부는 섭취어 주의할 것\n- 특정질환 (알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 일일섭취량 이상으로 섭취 시 

In [239]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['#'])[['product_function', 'product_caution']] # '&#8228;'는 ' '로 대체

product_function 컬럼값에 "#" 문자열이 없습니다.


,product_function,product_caution
0,[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[비타민A]①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\r\n[비타민D]①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요\r\n[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요,① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.
1,[차전자피 제품]1. 콜레스테롤 개선 2. 배변활동 원활,[차전자피 제품]충분한 물로 내용물을 완전히 부풀린 후 섭취할 것\r\n① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.


In [240]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['%'])[['product_function', 'product_caution']] # '%' 기호는 그대로 사용

,product_function,product_caution
0,"[루테인지아잔틴복합추출물20%]\n(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging\n[비타민C] 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n[비타민B12] 정상적인 엽산 대사에 필요\n[비타민A] 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n[비타민B6] 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n[엽산] 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[루테인지아잔틴복합추출물20%]\n영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n흡연자는 섭취 시 전문가와 상담할 것\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n[비타민D] 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
1,"[루테인지아잔틴복합추출물20%]\n(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging\n[은행잎 추출물] 기억력 개선·혈행 개선에 도움을 줄 수 있음\n[비타민C] 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[셀레늄(또는 셀렌)] 유해산소로부터 세포를 보호하는데 필요\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n[망간] 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요\n[판토텐산] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B12] 정상적인 엽산 대사에 필요\n[비타민 A] 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n[비타민B6] 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B1] 탄수화물과 에너지 대사에 필요\n[비타민B2] 체내 에너지 생성에 필요","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n소비기한을 확인 후 섭취하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n특이체질, 알레르기 체질이신 경우 성분을 확인 후 섭취하시기 바랍니다.\n이상사례 발생 시 즉시 섭취를 중단하시고, 전문의와 상담하시거나 소비자 상담실로 문의하시기 바랍니다.\n[루테인지아잔틴복합추출물20%]\n영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n흡연자는 섭취 시 전문가와 상담할 것\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n[은행잎 추출물]\n임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의\n의약품(항응고제) 복용 시 섭취에 주의"
2,"[루테인지아잔틴복합추출물20%(기능성원료인정 제2013-23호)]\n(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging\n[헤마토코쿠스추출물] 눈의 피로도 개선에 도움을 줄 수 있음\n[비타민A] 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[루테인지아잔틴복합추출물20%(기능성원료인정 제2013-23호)]\n영유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n흡연자는 섭취 시 전문가와 상담할 것\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n[헤마토코쿠스추출물]\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\nβ-카로틴의 흡수를 저해할 수 있음"
3,"[루테인지아잔틴복합추출물20%]\n(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging\n[EPA 및 DHA 함유 유지] 혈중 중성지질 개선･혈행 개선에 도움을 줄 수 있음, 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n[비타민C] 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[셀레늄(또는 셀렌)] 유해산소로부터 세포를 보호하는데 필요\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n[비타민A] 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n소비기한을 확인 후 섭취하시기 바라며, 섭취량 및 섭취 방법을 준수하시기 바랍니다.\n특이체질, 알레르기 체질이신 경우 성분을 확인 후 섭취하시기 바랍니다.\n이상사례 발생 시 즉시 섭취를 중단하시고, 전문의와 상담하시거나 소비자 상담실로 문의하시기 바랍니다.\n[루테인지아잔틴복합추출물20%]\n영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n흡연자는 섭취 시 전문가와 상담할 것\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n[EPA 및 DHA 함유 유지]\n의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n개인에 따라 피부 관련 이상반응이 발생할 수 있음\n[비타민D]\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
4,"[루테인지아잔틴복합추출물20%]\n(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging\n[헤마토코쿠스 추출물]눈의 피로도 개선에 도움을 줄 수 있음\n[비타민C]결합조직 형성과 기능유지에 필요, 철의 

In [241]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['&'])[['product_function', 'product_caution']] # '&#8228;' 제거 후'&' 기호 '·'로 대체

product_function 컬럼값에 "&" 문자열이 없습니다.


,product_function,product_caution
0,[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[비타민A]①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\r\n[비타민D]①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요\r\n[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요,① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.
1,①유산균 증식 및 유해균 억제에 도움을 줄 수 있음 \n②배변활동 원활에 도움을 줄 수 있음,"섭취량 및 섭취방법을 확인하시고 섭취하여 주십시오.\n- 유통기한이 경과된 제품은 섭취하지 않도록 주의바랍니다.\n- 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기바랍니다.\n- 제품개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다.\n- 어린이 섭취 시 보호자의 지도&감독하에 섭취시키십시오."
2,①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌,"섭취량 및 섭취방법을 확인하시고 섭취하여 주십시오.\n- 음용수에 4배수 희석하여 섭취하십시오 (제품 50ml를 물 150ml에 혼합하여 음용하십시오) \n- 유통기한이 경과된 제품은 섭취하지 않도록 주의 바랍니다.\n- 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n- 제품개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다.\n- 어린이 섭취 시 보호자의 지도&감독하에 섭취시키십시오. \n- 원료성분에 의해 침전물이 생길 수 있으나 제품에는 이상이 없으므로 잘 흔들어 드십시오."
3,[차전자피 제품]1. 콜레스테롤 개선 2. 배변활동 원활,[차전자피 제품]충분한 물로 내용물을 완전히 부풀린 후 섭취할 것\r\n① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.


In [242]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["'"])[['product_function', 'product_caution']] # "'"는 영문 사이에 사용된 경우가 아니면 제거

,product_function,product_caution
0,"'비타민C : ①결합조직 형성과 기능유지에 필요, \n ②철의 흡수에 필요, \n ③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요","특이체질, 알레르기 체질의 경우에는 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오"
1,[Lactobacillus acidophilus YT1 (제2019-22호)] (국문) 갱년기 여성 건강에 도움을 줄 수 있음 (영문) May help to improve menopausal women's health\n[프로바이오틱스] 유산균 증식 및 유해균 억제․배변활동 원활·장건강에 도움을 줄 수 있음,질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n-어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2,(국문) 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n갱년기 여성의 건강에 도움을 줄 수 있음\n(영문) May help to remove reactive oxygen species which are harmful to human body\nMay help to climacteric women's health\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n①체내 에너지 생성에 필요\n①뼈 건강에 도움을 줄 수 있음\n①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것\n수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n식사 후 또는 식사와 함께 섭취할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n영유아, 어린이, 임산부와 수유부는 섭취에 주의\n대두에 알레르기를 나타내는 사람은 섭취에 주의\n에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,"'1) 비타민 A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n2) 비타민 B1 : 탄수화물과 에너지 대사에 필요\n3) 비타민 B2 : 체내 에너지 생성에 필요\n4) 비타민 B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n5) 비타민 B12 : 정상적인 엽산 대사에 필요\n6) 비타민 C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n7) 비타민 D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n8) 비타민 E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n9) 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n10) 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n11) 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n12) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n13) 구리 : 철의 운반과 이용에 필요, 유해산소로부터 세포를 보호하는데 필요\n14) 셀레늄(또는 셀렌) : 유해산소로부터 세포를 보호하는데 필요\n15) 망간...","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n3) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n4) 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것"
4,비타민D : (1) 칼슘과 인이 흡수되고 이용되는데 필요 (2) 뼈의 형성과 유지에 필요 (3) 골다공증발생 위험 감소에 도움을 줌 \n루바브뿌리추출물(제2019-6호) : (국문) 갱년기 여성 건강에 도움을 줄 수 있음 (영문) May help to menopausal women's health\n비타민K : (1) 정상적인 혈액응고에 필요 (2) 뼈의 구성에 필요,"(1) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(3) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n(4) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n(5) 메스꺼움 등 소화계통의 불편함과 설사를 유발할 수 있음\n(6) 알레르기 체질이거나 특정질환이 있는 사람은 섭취 전 전문가와 상담할 것\n(7) 항응고제 등 복용 시 전문가와 상담할 것"
5,"[락티움정제]\n- 유단백가수분해물(락티움)(기능성원료인정제2020-2호)\n(국문)수면의 질 개선에 도움을 줄 수 있음\n(영문)May help to improve sleep\n- 셀레늄\n유해산소로부터 세포를 보호하는데 필요\n- 판토텐산\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n- 비타민B1\n탄수화물과 에너지 대사에 필요\n- 엽산\n(가) 세포와 혈액생성에 필요(나) 태아 신경관의 정상 발달에 필요(다) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[은행잎정제]\n- 은행잎추출물\n기억력 개선･혈행 개선에 도움을 줄 수 있음\n- 비오틴\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n- 아연\n(가) 정상적인 면역기능에 필요(나) 정상적인 세포분열에 필요\n- 비타민D\n(가) 칼슘과 인이 흡수되고 이용되는데 필요(나) 뼈의 형성과 유지에 필요(다) 골다공증발생 위험 감소에 도움을 줌\n\n[엉겅퀴정제]\n- MS-10 엉겅퀴등복합추출물(기능성원료인정제2020-6호)\n(국문)갱년기 여성건강에 도움을 줄 수 있음\n(영문)May help to menopausal women's health\n- 나이아신\n체내 에너지 생성에 필요\n- 비타민B6\n(가) 단백질 및 아미노산 이용에 필요(나) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 비타민B12\n정상적인 엽산 대사에 필요","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n[판토텐산, 비타민B1, 비오틴, 비타민D, 비타민B12, MS-10 엉겅퀴등복합추출물(기능성원료인정제2020-6호)]\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[비타민D]\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n[은행잎추출물]\n임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의\n의약품(항응고제) 복용 시 섭취에 주의\n[유단백가수분해물(락티움)(기능성원료인정제2020-2호)] \n임산부, 수유여성 및 어린이는 섭취에 주의\n우유 및 유제품에 대하여 알레르기를 나타내는 사람은 섭취에 주의\n[MS-10 엉겅퀴등복합추출물(기능성원료인정제2020-6호)]\n임산부 및 수유부는 섭취를 피할 것\n알레르기를 나타내는 사람은 섭취 시 주의\n에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n항우울제 또는 호르몬제를 복용하는 사람은 섭취에 주의"
6,"비타민 A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n- 비타민 B1 : 탄수화물과 에너지 대사에 필요\n- 비타민 B2 : 체내 에너지 생성에 필요\n- 비타민 B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 비타민 B12 : 정상적인 엽산 대사에 필요\n- 비타민 C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 

In [243]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["*"])[['product_function', 'product_caution']] # '*'는 구분자로 사용됨 > '\n'로 대체

,product_function,product_caution
0,* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* 아연\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n\n* 프로바이오틱스\n① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음\n② 배변활동 원활에 도움을 줄 수 있음\n③ 장건강에 도움을 줄 수 있음,* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
1,* 프로바이오틱스\n① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음\n② 배변활동 원활에 도움을 줄 수 있음\n③ 장건강에 도움을 줄 수 있음,* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2,* 셀레늄(or 셀렌)\n① 유해산소로부터 세포를 보호하는데 필요\n\n* 아연\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n\n* 프로바이오틱스\n① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음\n② 배변활동 원활에 도움을 줄 수 있음\n③ 장 건강에 도움을 줄 수 있음,* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,* 프로바이오틱스\n(1) 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음\n\n* 아연\n(1) 정상적인 면역기능에 필요\n(2) 정상적인 세포분열에 필요\n\n* 비타민D\n(1) 칼슘과 인이 흡수되고 이용되는데 필요\n(2) 뼈의 형성과 유지에 필요\n(3) 골다공증발생 위험 감소에 도움을 줌\n\n* 셀렌\n(1) 유해산소로부터 세포를 보호하는데 필요,* 프로바이오틱스\n(가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(다) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민D\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
4,* 가르시니아캄보지아 추출물\n① 탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n\n* 프락토올리고당\n① 장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n\n* 프로바이오틱스\n① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음\n② 배변활동 원활에 도움을 줄 수 있음\n③ 장건강에 도움을 줄 수 있음,"* 가르시니아캄보지아 추출물\n① 어린이, 임산부 및 수유부는 섭취를 피할 것\n② 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n③ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 프락토올리고당\n① 섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있음\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
...,...,...
391,유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음\n\n① 간 건강에 도움을 줄 수 있음,"* 밀크씨슬(카르두스 마리아누스) 추출물 \n① 어린이, 임산부 및 수유부는 섭취를 피할 것\n② 알레르기 반응이 나타나는 경우에는 섭취 중단\n③ 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취에 주의\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
392,①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음② 배변활동 원활에 도움을 줄 수 있음③ 장건강에 도움을 줄 수 있음,* 프로바이오틱스\n① 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
393,①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요,"* 포장이 변형 ,팽창, 손상되었거나 내용물이 변질되었을 경우 음용하지 마십시오.\n* 제품의 개봉 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다. \n* 특이체질, 알레르기 체질인 경우 소비자 상담실에 문의하신 후 섭취하시기 바랍니다."
394,①혈중 중성지질 개선·혈행 개선·기억력 개선·건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n\n①기억력 개선에 도움을 줄 수 있음 ②혈행 개선에 도움을 줄 수 있음\n\n①유해산소로부터 세포를 보호하는데 필요\n\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"[은행잎 추출물]\n① 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의 ② 의약품(항응고제) 복용 시 섭취에 주의\n\n[EPA 및 DHA함유 유지]\n* 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n* 개인에 따라 피부 관련 이상반응이 발생할 수 있음\n* 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [244]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["+"])[['product_function', 'product_caution']] # '+'는 제거하지 않음

,product_function,product_caution
0,"[비타민B1]탄수화물과 에너지 대사에 필요 \n[비타민B2]체내 에너지 생성에 필요 \n[비타민B6]단백질 및 아미노산 이용에 필요, 혈액의 효모시스테인 수준을 정상으로 유지하는데 필요 \n[비타민B12] 정상적인 엽산 대사에 필요 \n[나이아신]체내 에너지 생성에 필요 \n[판토텐산]지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[엽산]세포와 혈액생성에 필요, 태아신경관의 정상 발달에 필요+B11, 혈액의 효모 시스테인 수준을 정상으로 유지하는데 필요 \n[아연]정상적인 면역기능에 필요, 정상적인 세포분열에 필요 \n[셀레늄(셀렌)]유해산소로부터 세포를 보호하는데 필요 \n[비오틴]지방, 탄수화물, 단백질 대사와 에너지 생성에 필요 \n[비타민C]결합조직 셩성과 기능유지에 필요, 철의 흡수에 필요, 유해산소로부터 세포를 보호하는데 필요 \n[비타민D]칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌 \n[망간]뼈 형성에 필요, 에너지 이용에 필요, 유해산로부터 세포를 보호하는데 필요 \n[몰리브덴]산화·환원 효소의 활성에 필요 \n[크롬]",(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
1,"[Lactobacillus 복합물 HY7601+KY1032]\n(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat\n\n[프로바이오틱스]\n유산균 증식 및 유해균 억제·배변활동 원활·장 건강에 도움을 줄 수 있음\n\n[바나바잎 추출물]\n①식후 혈당상승 억제에 도움을 줄 수 있음\n\n[비오틴]\n(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[판토텐산]\n(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n-어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n\n\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n(1)
2,"Lactobacillus 복합물 HY7601 + KY1032(제2019-4호) : (국문) 체지방 감소에 도움을 줄 수 있음, (영문) May help to reduce body fat, 유산균 증식 및 유해균 억제에 도움을 줄 수 있음, 배변활동 원활에 도움을 줄 수 있음, 장 건강에 도움을 줄 수 있음\n나이아신 : (1) 체내 에너지 생성에 필요\n비오틴 : (1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n판토텐산 : (1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n비타민B6 : (1) 단백질 및 아미노산 이용에 필요 (2) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n비타민B1 : (1) 탄수화물과 에너지 대사에 필요",(1) 질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n(2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,"Lactobacillus 복합물 HY7601+KY1032(제2019-4호) :(국문) 체지방 감소에 도움을 줄 수 있음, 유산균 증식 및 유해균 억제에 도움을 줄 수 있음, 배변활동 원활에 도움을 줄 수 있음, 장 건강에 도움을 줄 수 있음 (영문) May help to reduce body fat",(1) 질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n(2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
4,"Lactobacillus복합물 HY7601+KY1032 : (국문)체지방 감소에 도움을 줄 수 있음, 유산균 증식 및 유해균 억제, 배변활동 원활, 장건강에 도움을 줄 수 있음 (영문) May help to reduce body fat",(1) 질환이 있거나 의약품(면역억제제 등) 복용시 전문가와 상담하십시오.\n(2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼수 있습니다.\n(3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하십시오.\n(4) 이상사례 발생시 섭취를 중단하고 전문가와 상담하십시오.
5,"[Lactobacillus 복합물 HY7601 + KY1032, (제2019-4호)]\n(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat.\n\n[프로바이오틱스] 유산균 증식 및 유해균 억제•배변활동 원활•장 건강에 도움을 줄 수 있음",[Lactobacillus 복합물 HY7601 + KY1032] \n·질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것 \n·알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n·어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것 \n·이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
6,"[Lactobacillus복합물 HY7601+KY1032]\n(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat\n\n[프로바이오틱스]\n유산균 증식 및 유해균 억제·배변활동 원활·장 건강에 도움을 줄 수 있음\n\n[비타민B1]\n(1) 탄수화물과 에너지 대사에 필요\n\n[나이아신]\n①체내 에너지 생성에 필요\n\n[판토텐산]\n(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비타민B6]\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[비타민D]\n①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n[아연]\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요",[Lactobacillus 복합물 HY7601+KY1032]\n-질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n-어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n[비타민B1]\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n[판토텐산]\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n[비타민D]\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
7,[Lactobacillus 복합물 HY7601+KY1032] 체지방 감소에 도움을 줄 수 있음\n\n[프로바이오틱스] 유산균 증식 및 유해균 억제･ 배변활동 원활･ 장 건강에 도움을 줄 수 있음\n\n[아연] ①정상적인 면역기능에 필요②정상적인 세포분열에 필요,질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n-어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
8,"Lactobacillus복합물 HY7601 + KY1032 : (국문) 체지방 감소에 도움을 줄 수 있음 (영문) May help to reduce body fat\n\n판토텐산 : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",·질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것 \n·알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음 \n·어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것 \n·이상사례 발생 시 섭취를 중단하고 전문가와 상

In [245]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["- "])[['product_function', 'product_caution']] 
# '-' 기호의 경우'\n-' -> '\n'으로 대체, '-'으로 텍스트가 시작하는 경우 제거, 그 외에는 그대로 둔다.

,product_function,product_caution
0,"단백질 : (가) 근육, 결합조직 등 신체조직의 구성성분\n (나) 효소, 호르몬, 항체의 구성에 필요\n (다) 체내 필수 영양성분이나 활성물질의 운반과 저장에 필요\n (라) 체액, 산-염기의 균형 유지에 필요\n (마) 에너지, 포도당, 지질의 합성에 필요- 비타민 B1 : 탄수화물과 에너지 대사에 필요\n- 비타민 B6 : (가) 단백질 및 아미노산 이용에 필요\n (나) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 비타민 D : (가) 칼슘과 인이 흡수되고 이용되는데 필요\n (나) 뼈의 형성과 유지에 필요\n (다) 골다공증발생 위험 감소에 도움을 줌\n- 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n- 칼슘 : (가) 뼈와 치아 형성에 필요\n (나) 신경과 근육 기능 유지에 필요\n (다) 정상적인 혈액응고에 필요\n (라) 골다공증발생 위험 감소에 도움을 줌\n- 마그네슘 : (가) 에너지 이용에 필요 \n (나) 신경과 근육 기능 유지에 필요\n- 아연 : (가) 정상적인 면역기능에 필요 \n (나) 정상적인 세포분열에 필요","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n3) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n4) 특정 단백질에 알레르기를 나타내는 경우에는 섭취 주의"
1,"프로바이오틱스 : 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음 \n- 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n- 셀렌 : 유해산소로부터 세포를 보호하는데 필요",1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2,"EPA 및 DHA 함유 유지 : 혈중 중성지질 개선･혈행 개선･건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음.\n- 비타민 D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n- 철 : 체내 산소운반과 혈액생성에 필요, 에너지 생성에 필요","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오\n2) 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n3) 개인에 따라 피부 관련 이상반응이 발생할 수 있음\n4) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n5) 특히 6세 이하는 과량섭취하지 않도록 주의\n6) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,"EPA및DHA함유유지 : 혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음, 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n- 비타민A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n- 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n- 비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요","1) 특이체질, 알레르기 체질의 경우에는 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n3) 개인에 따라 피부 관련 이상반응이 발생할 수 있음\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n5) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
4,홍삼 - ①면역력 증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선·항산화에 도움을 줄 수 있음\n\n아연 - ①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의하십시오."
...,...,...
2676,면역력 증진･피로개선･혈소판 응집억제를 통한 혈액흐름･기억력 개선･항산화에 도움을 줄 수 있음,당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의\n- 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2677,"[비타민B1]①탄수화물과 에너지 대사에 필요\n\n[비타민B2]①체내 에너지 생성에 필요\n\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\n\n[나이아신]①체내 에너지 생성에 필요\n\n[엽산]①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비오틴]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n[셀렌]①유해산소로부터 세포를 보호하는데 필요",개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n- 알레르기 체질이신 경우 성분을 확인 하신 후 섭취하여 주시기 바랍니다.
2678,[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n\n[비타민A] (1) 어두운 곳에서 시각 적응을 위해 필요 (2) 피부와 점막을 형성하고 기능을 유지하는데 필요 (3) 상피세포의 성장과 발달에 필요\n\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n[비타민D] ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"영‧유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n- 흡연자는 섭취 시 전문가와 상담할 것\n- 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
2679,[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,"어린이, 임산부 및 수유부는 섭취를 피할 것\n- 간⦁신장⦁심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [246]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["/"])[['product_function', 'product_caution']] 

,product_function,product_caution
0,[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌,고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
1,"* 마그네슘\n(1) 에너지 이용에 필요\n(2)신경과 근육 기능 유지에 필요\n\n* 비타민B1\n(1) 탄수화물과 에너지 대사에 필요\n\n* 비타민B2\n(1) 체내 에너지 생성에 필요\n\n* 비타민B6\n(1) 단백질 및 아미노산 이용에 필요\n(2) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n* 비타민C\n(1) 결합조직 형성과 기능유지에 필요\n(2) 철의 흡수에 필요\n(3) 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n* 비타민D\n(1) 칼슘과 인이 흡수되고 이용되는데 필요\n(2) 뼈의 형성과 유지에 필요\n(3) 골다공증발생 위험 감소에 도움을 줌\n\n* 아연\n(1) 정상적인 면역기능에 필요\n(2) 정상적인 세포분열에 필요\n\n* 키토산/키토올리고당\n(1) 혈중 콜레스테롤 개선·체지방 감소에 도움을 줄 수 있음\n\n* 판토텐산\n(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",* 비타민B1\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민B2\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민D\n(1) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 키토산/키토올리고당\n(1) 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의(게 또는 새우를 원재료로 사용한 경우에 한함)
2,"[비타민 B1] 탄수화물과 에너지 대사에 필요\n[비타민 A] 어두운 곳에서 시각 적응을 위해 필요/피부와 점막을 형성하고 기능을 유지하는데 필요/상피세포의 성장과 발달에 필요\n[비타민 C] 결합조직 형성과 기능유지에 필요/철의 흡수에 필요/항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민 D] 칼슘과 인이 흡수되고 이용되는데 필요/뼈의 형성과 유지에 필요/골다공증발생 위험 감소에 도움을 줌\n[비타민 E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[나이아신] 체내 에너지 생성에 필요\n[망간] 뼈 형성에 필요/에너지 이용에 필요/유해산소로부터 세포를 보호하는데 필요\n[비오틴] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[아연] 정상적인 면역기능에 필요②정상적인 세포분열에 필요\n[판토탄산] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것
3,면역력 증진･피로개선에 도움을 줄 수 있음/혈소판 응집억제를 통한 혈액흐름･기억력 개선･항산화에 도움을 줄 수 있음,당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의하시기 바랍니다.\n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.\n-이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다.
4,"[바나바잎 추출물] 식후 혈당상승 억제에 도움을 줄 수 있음\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[크롬] 체내 탄수화물, 지방, 단백질 대사에 관여\n[나이아신] 체내 에너지 생성에 필요\n[아연] 정상적인 면역기능에 필요 / 정상적인 세포분열에 필요\n[비오틴] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B1] 탄수화물과 에너지 대사에 필요\n[비타민B2] 체내 에너지 생성에 필요\n[엽산] 세포와 혈액생성에 필요 / 태아 신경관의 정상 발달에 필요 / 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n소비기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n당뇨병이 있는 경우 섭취 전 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다."
...,...,...
913,①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"1. 알러지 등 특이체질의 경우 과민반응이 우려될 수 있으니 성분을 확인하신후 섭취하십시오.\n2. 15세 이하의 어린이는 1/2분량을 섭취하십시오.\n3. 제품의 개봉 시 유리파손으로 인한 상처를 입을 수 있으니 주의하십시오.\n4. 의약품(당뇨치료제, 혈액항응고제)복용 시 섭취에 주의하십시오.\n5. 제품의 섭취 시 섭취량 및 섭취방법을 확인하고 섭취하십시오."
914,[홍삼제품]면역력 증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선·항산화에 도움을 줄 수 있음,어린이는 성인기준 1/2만 드십시오.
915,"[비타민B1] ①탄수화물과 에너지 대사에 필요\n[판토텐산] ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B2] ①체내 에너지 생성에 필요\n[비타민B6] ①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[나이아신] ①체내 에너지 생성에 필요\n[비타민B12] ①정상적인 엽산 대사에 필요\n[비오틴] ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[엽산] ①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요",[비타민B1/판토텐산/비타민B2/비타민B12/비오틴]\n1. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
916,"[비오틴]\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B12]\n정상적인 엽산 대사에 필요\n[비타민B1]\n탄수화물과 에너지 대사에 필요\n[비타민B6]\n①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[나이아신]\n체내 에너지 생성에 필요\n[판토텐산]\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B2]\n체내 에너지 생성에 필요\n[비타민C]\n①결합조직 형성과 기능유지에 필요 ②철의 흡수에 필요 ③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[엽산]\n①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요",[비타민B1/비타민B2/판토텐산/비타민B12/비오틴]\n1. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것


- '[', ']' 안의 '/'가 있는 경우, 숫자 사이에 분수 기호로 '/'가 있는 경우, word_patterns 문자열과 같이 '/'가 있는 경우 제외하고 '\n'으로 대체

```python
word_patterns = [
    r'구아검/구아검가수분해물',
    r'식물스테롤/식물스테롤에스테르',
    r'이눌린/치커리추출물',
    r'키토산/키토올리고당',
    r'루테인/지아잔틴',
    r'판토텐산/비오틴',
    r'엠에스엠/MSM',
    r'NAG/N-아세틸글루코사민'   
]
```

In [247]:
find_chars_df(p_df, ['product_function', 'product_caution'], [":"])[['product_function', 'product_caution']] # ':' 기호 제거

,product_function,product_caution
0,"1) 비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요.\n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌. \n3) 비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n4) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요.","섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취 하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있고, 개봉 시 내용물이 흘러나올 수 있으므로취급 시 주의하십시오. \n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오\n이상사례 발생시 섭취를 중단하고 전문가와 상담하십시오."
1,"나이아신: ①체내 에너지 생성에 필요\n\n판토텐산: ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민B1: ①탄수화물과 에너지 대사에 필요\n\n비타민B2: ①체내 에너지 생성에 필요\n\n비타민B12: ①정상적인 엽산 대사에 필요\n\n비오틴: ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n(2) 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.
2,"풋사과추출물 애플페논(Applephenon) : 체지방 감소에 도움을 줄 수 있음\n\n비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민B1 : 탄수화물과 에너지 대사에 필요\n\n비타민B2 : 체내 에너지 생성에 필요","① 사과에 알레르기가 있으신 분은 섭취시 주의.\n ② 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\n ③ 임산부, 수유부, 질병치료 중인 분은 섭취 전 의사와 상담 후 섭취하십시오.\n ④ 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다."
3,가르시니아캄보지아추출물 : 탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌.\n비타민C : ① 결합조직 형성과 기능유지에 필요\n ② 철의 흡수에 필요\n ③ 유해산소로부터 세포를 보호하는데 필요\n비타민B2 : 체내 에너지 생성에 필요\n비타민B1 : 탄수화물과 에너지 대사에 필요,"① 임산부, 수유여성 및 어린이는 섭취에 주의하십시오.\n② 특정 성분에 알레르기가 있으신 분은 원료명 확인 후 섭취하십시오.\n③ 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다."
4,"비타민C : ① 결합조직 형성과 기능유지에 필요\n ② 철의 흡수에 필요\n ③ 유해산소로부터 세포를 보호하는데 필요\n아연 : ① 정상적인 면역기능에 필요, ② 정상적인 세포분열에 필요","① 임산부, 수유여성 및 어린이는 섭취에 주의하십시오.\n② 특정 성분에 알레르기가 있으신 분은 원료명 확인 후 섭취하십시오.\n③ 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다."
...,...,...
3345,[EPA 및 DHA 함유 유지] ①혈중 중성지질 개선 · 혈행 개선에 도움을 줄 수 있음 ②건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음 ③기억력 개선에 도움을 줄 수 있음\n[비타민D] ①칼슘과 인이 흡수되고 이용되는데 필요 ②뼈의 형성과 유지에 필요 ③골다공증발생 위험 감소에 도움을 줌,"11. 섭취 시 주의사항 : \n1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[EPA 및 DHA 함유유지]①의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n②개인에 따라 피부 관련 이상반응이 발생할 수 있음\n③이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[비타민D]\n①고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n②이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3346,[프로폴리스추출물 제품]①항산화에 도움을 줄 수 있음 ②구강에서의 항균작용에 도움을 줄 수 있음\r\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"특이체질이거나 알러지 체질의 경우, 개인에 따라 과민반응이 나타날 수 있으므로 원료명을 확인하시고 섭취하십시오.\r\n- 유통기한을 확인하시기 바라며, 임산부, 수유 여성, 어린이는 섭취에 주의하십시오.\r\n- 〔프로폴리스추출물〕 : 프로폴리스에 알레르기를 나타내는 사람은 섭취에 주의"
3347,①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n①정상적인 엽산 대사에 필요,"1)원료의 선택 : 모든 원료는 건강기능식품공전, 식품공전, 식품첨가물공전에서 정하는\n\t\t 기준 및 규격에 적합한 원료만을 사용한다.\n 2)제조설비의 소독 : 사용되는 모든 기기 및 기구는 세척 후 주정으로 소독, 건조하여 사용한다.\n 3)칭 량 : 1회 배합에 사용되는 원료량을 기준으로 칭량하며 전자저울로 칭량한다.\n 4)선 별 : 분말원료는 진동체로 선별하여 이물을 제거하며 원료 입자의 크기를 균질화 한다.\n 5)혼 합 및 살균 : 원료를 배합비율에 맞추어 혼합탱크에 투입하여 100℃(±2℃)에서 15~30분간\n\t\t 균질하게 혼합 및 살균 후 필터를 거쳐 배출한다.\n 6)충진 및 포장 : 배출된 원료를 스틱 충진기로 15g 씩 충진 한 후 단상자에 넣어 포장한다.\n 7)완제품 검사 : 기준 및 규격에 맞게 자가 품질검사를 실시하여 적합여부를 판정한다.\n 8)출 하 : 품질관리인으로부터 최종 적합 판정된 완제품을 출하한다."
3348,"[단백질]①근육, 결합조직 등 신체조직의 구성성분②효소, 호르몬, 항체의 구성에 필요③체내 필수 영양성분이나 활성물질의 운반과 저장에 필요④체액, 산-염기의 균형 유지에 필요⑤에너지, 포도당, 지질의 합성에 필요",1) 원료 : 식품공전 등에서 정하는 적합한 원료을 구입 적합품만 사용한다.\r\n2) 칭량 : 미량원료는 정밀전자저울(5g~10kg) 그외 원료는 전자저울(200g~150kg)로 선택 칭량한다.\r\n3) 체과 및 혼합 : 칭량한 원료 20mesh 체로 체과한 후 혼합기에 넣고 균질하게 혼합한다.\r\n4) 필름 및 용기포장 : 균질 혼합된 원료를 포장단위별로 필름 및 용기에 충진 포장하여 박스포장후 외포장에 유통기한을 표시한다.\r\n5) 검사 : 건강기능식품 규격에 의한 검사를 실시한다.\r\n6) 보관 및 출하 : 검사 결과 기준규격에 적합한 제품에 한하여 보관 및 출하한다.


In [248]:
find_chars_df(p_df, ['product_function', 'product_caution'], [";"])[['product_function', 'product_caution']] # ';' 기호 제거

,product_function,product_caution
0,"1) 시서스추출물 : 체지방 감소에 도움을 줄 수 있음. \n2) 비타민B1 : 탄수화물과 에너지 대사에 필요.\n3) 비타민B2 : 체내 에너지 생성에 필요.\n4) 나이아신 ; 체내 에너지 생성에 필요.\n5) 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요.","섭취 시 목에 걸리거나 불편할 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오. \n영·유아, 어린이, 임산부 및 수유부는 섭취에 주의하십시오.\n의약품(혈당강하제) 복용 시 섭취에 주의하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
1,[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[비타민A]①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\r\n[비타민D]①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요\r\n[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요,① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.
2,[차전자피 제품]1. 콜레스테롤 개선 2. 배변활동 원활,[차전자피 제품]충분한 물로 내용물을 완전히 부풀린 후 섭취할 것\r\n① 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.\r\n② 임신&#8228;수유부나 의약품을 섭취하시는 분은 본 제품을 드시기 전에 의사와 상담 후 섭취하십시오.


In [249]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["<"])[['product_function', 'product_caution']] # 꺽쇠 기호 포함 안에 있는 문자열 제거 X

,product_function,product_caution
0,"1) 피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL> : 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음, 갱년기 여성의 건강에 도움을 줄 수 있음.\n2) 비타민C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n임산부 및 수유부와 12세 이하의 어린이는 섭취를 피하십시오.\n수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n식사 후 또는 식사와 함께 섭취하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
1,"1) 피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL> : 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음, 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하는데 도움을 줄 수 있음, 갱년기 여성의 건강에 도움을 줄 수 있음.\n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n임산부 및 수유부와 12세 이하의 어린이는 섭취를 피하십시오.\n수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n식사 후 또는 식사와 함께 섭취하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오."
2,<프로바이오틱스> 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음\n\n<테아닌> 스트레스로 인한 긴장완화에 도움을 줄 수 있음,"1) 질환이 있거나 의약품 복용 시 의사, 약사 등 전문가와 상담 후 섭취하시기 바랍니다.\n2) 특정성분에 알레르기가 있는 분은 과민반응이 나타날 수 있으니 섭취 전에 반드시 원료(성분)를 확인하시기 바랍니다.\n3) 어린이가 함부로 섭취하지 않도록 일일섭취량을 확인하시기 바랍니다.\n4) 이상사례 발생 시 섭취를 중단하고 의사, 약사 등 전문가와 상담하시기 바랍니다.\n5) 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하시기 바랍니다.\n6) 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하시기 바랍니다."
3,<마카젤라틴화분말>\n①정자 운동성 개선에 도움을 줄 수 있음\n<아연>\n①정상적인 면역기능에 필요 ②정상적인 세포분열에 필요\n<옥타코사놀>\n①지구력 증진에 도움을 줄 수 있음,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 \n바랍니다."
4,[피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호)]\n① 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n② 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하는데 도움을 줄 수 있음\n[망간]\n① 뼈 형성에 필요 \n② 에너지 이용에 필요 \n③ 유해산소로부터 세포를 보호하는데 필요\n[셀레늄(또는 셀렌)]\n유해산소로부터 세포를 보호하는데 필요\n[비타민B12]\n정상적인 엽산 대사에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요 \n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B2]\n체내 에너지 생성에 필요,"① 알레르기 체질이신 경우 성분을 확인한 후 섭취여부를 결정하여야 함\n[피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호)]\n② 임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것 \n③ 수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것 \n④ 식사 후 또는 식사와 함께 섭취할 것 \n⑤ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,"<정제>\n1) 비타민 A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n2) 비타민 E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n3) 비타민 K : 정상적인 혈액응고에 필요, 뼈의 구성에 필요\n4) 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n5) 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6) 비타민B12 : 정상적인 엽산 대사에 필요\n7) 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n8) 비타민 C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n9) 셀렌 : 유해산소로부터 세포를 보호하는데 필요\n10) 망간 : 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요 \n11) 몰리브덴 : 산화·환원 효소의 활성에 필요\n12) 크롬 : 체내 탄수화물, 지방, 단백질 대사에 관여\n\n<액상>\n1) 비타민 D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n2) 비타민 B1 : 탄수화물과 에너지 대사에 필요\n3) 비타민 B2 : 체내 에너지 생성에 필요...","1) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n2) 항응고제 등 복용 시 전문가와 상담하십시오.\n3) 당뇨병이 있는 경우 섭취 전 전문가와 상담하십시오.\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n5) 알레르기 특이체질이신 경우 성분을 확인하신 후 섭취하시기 바랍니다.\n6) 제품의 개봉 또는 섭취 시, 포장재에 의해 상처를 입을 수 있으니 주의하십시오."
6,<아연>\n①정상적인 면역기능에 필요\n②정상적인 세포분열에 필요\n\n<MSM>\n① 관절 및 연골건강에 도움을 줄 수 있음,(가) 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
7,"<정제1>\n[비타민A] ① 어두운 곳에서 시각 적응을 위해 필요 ② 피부와 점막을 형성하고 기능을 유지하는데 필요 ③ 상피세포의 성장과 발달에 필요\n[비타민D] ① 칼슘과 인이 흡수되고 이용되는데 필요 ② 뼈의 형성과 유지에 필요 ③ 골다공증발생 위험 감소에 도움을 줌\n[비타민E] ① 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민K] ① 정상적인 혈액응고에 필요 ② 뼈의 구성에 필요\n[비타민B1] ① 탄수화물과 에너지 대사에 필요\n[비타민B2] ① 체내 에너지 생성에 필요\n[나이아신] ① 체내 에너지 생성에 필요\n[판토텐산] ① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비오틴] ① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B6] ① 단백질 및 아미노산 이용에 필요 ② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[엽산] ① 세포와 혈액생성에 필요 ② 태아 신경관의 정상 발달에 필요 ③ 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B12] ① 정상적인 엽산 대사에 필요\n[비타민C] ① 결합조직 형성과 기능유지에 필요 ② 철의 흡수에 필요 ③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[철] ① 체내 산소운반과 혈액생성에 필요 ② 에너지 생성에 필요\n[구리] ① 철의 운반과 이용에 필요 ② 유해산소로부터 세포를 보호하는...","정

In [250]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["["])[['product_function', 'product_caution']] # 대괄호 안에 있는 문자열 제거 X

,product_function,product_caution
0,[EPA 및 DHA 함유유지] 혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음.\n[비타민E] 유해산소로부터 세포를 보호하는데 필요,알레르기 체질이신 경우 성분을 확인 한 후 섭취하시기 바랍니다.
1,[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌,고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
2,"[비타민B1]①탄수화물과 에너지 대사에 필요\n[비오틴]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[셀레늄(또는 셀렌)]①유해산소로부터 세포를 보호하는데 필요","1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[비타민B1, 비오틴]\n 1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,[프로바이오틱스 제품]①유익한 유산균 증식②유해균 억제 또는 배변활동 원활\r\n[홍삼제품]①면역력 증진②피로회복③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n① 알레르기 체질이신 경우 성분을 확인한 후 섭취하시기 바랍니다.\r\n② 제품을 개봉할 때 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의 하십시오."
4,[히알루론산]피부보습에 도움을 줄 수 있음\n\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요,NaN
...,...,...
12249,①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"[홍삼제품]-의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n -알레르기 등 특정, 특이체질이신 분은 제품성분 확인 후 섭취여부를 결정.\n -식물성 성분등으로 침전이 생기는 경우가 있으니 흔든 후 섭취."
12250,①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n[홍삼제품]특이체질 및 알러지 체질의 경우 성분을 확인 후 섭취"
12251,①면역력 증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선·항산화·갱년기 여성의 건강에 도움을 줄 수 있음\n\n1. 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음,"[홍삼] 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n\n[피크노제놀-프랑스해안송껍질추출물]\n - 임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것\n - 수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n - 식사 후 또는 식사와 함께 할 것\n - 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
12252,(1) 탄수화물과 에너지 대사에 필요\n\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n(1) 체내 에너지 생성에 필요\n\n①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n(1) 정상적인 엽산 대사에 필요\n\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[비타민B1, 비타민B2, 비타민B12]\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [251]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["="])[['product_function', 'product_caution']] # '=' 특수문자 제거

product_caution 컬럼값에 "=" 문자열이 없습니다.


,product_function,product_caution
0,"비타민B1 : 탄수화물과 에너지 대사에 필요\n 비타민B2 : 체내 에너지 생성에 필요\n 비타민B6 : 단백질 및 아미노산 이용에 필요 / 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n 비타민B12 : 정상적인 엽산 대사에 필요\n 비타민C : 결합조직 형성과 기능유지에 필요 / 철의 흡수에 필요 / 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌\n 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n 나이아신 : 체내 에너지 생성에 필요\n 셀레늄(=셀렌) : 유해산소로부터 세포를 보호하는데 필요\n 크롬 : 체내 탄수화물, 지방, 단백질 대사에 관여\n 망간 : 뼈 형성에 필요 / 에너지 이용에 필요 / 유해산소로부터 세포를 보호하는데 필요\n 몰리브덴 : 산화·환원 효소의 활성에 필요\n 아연 : 정상적인 면역기능에 필요 / 정상적인 세포분열에 필요\n 엽산 : 세포와 혈액생성에 필요 / 태아 신경관의 정상 발달에 필요 / 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요",(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(다) 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것


In [252]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["?"])[['product_function', 'product_caution']] # 오류 문자 예외사항 처리 후 ' '로 대체

,product_function,product_caution
0,[차전자피 제품]①혈중 콜레스테롤 개선에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 수 있음\r\n혈중 콜레스테롤 개선?배변활동 원활에 도움을 줄 수 있음혈중 콜레스테롤 개선?배변활동 원활에 도움을 줄 수 있음,"임신, 수유중인 여성과 성장기 어린이는 섭취에 주의하여야 합니다. 특정 성분에 민감하거나 알레르기 체질이신 경우 성분을 확인한 후 섭취하십시오.\r\n반드시 충분한 물과 함께 섭취할 것."
1,[차전자피 제품]①혈중 콜레스테롤 개선에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 수 있음\r\n혈중 콜레스테롤 개선·? 배변활동 원활에 도움을 줄 수 있음\r\n혈중 콜레스테롤 개선·? 배변활동 원활에 도움을 줄 수 있음,"[차전자피제품] 반드시 충분한 물과 함께 섭취할 것.(액상제외)\r\n1) 본 제품은 반드시 물과 함께 섭취(섭취방법 참조)하거나, 물에 타서 섭취 하십시오.\r\n2) 6세 미만의 어린이는 섭취를 삼가시고, 취침직전 섭취는 삼가 해 주십시오.\r\n3) 섭취 후 복통, 알레르기, 기타 증상이 나타나면 섭취를 중단하시고, 약사 또는 의사와 상의하십시오."
2,[밀크씨슬추출물]간 건강에 도움을 줄 수 있음\r\n[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요\r\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"[밀크씨슬추출물]① 알레르기 반응이 나타나는 경우에는 섭취를 중단하십시오 ② 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취를 주의 하십시오\r\n- 유통기한을 확인하시기 바라며, 권장섭취량 및 섭취방법을 준수하시기 바랍니다.\r\n- 영?유아, 임산부, 수유부는 섭취에 주의하시기 바랍니다."
3,[홍삼제품]①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n-? 알러지 등 특이체질의 경우 과민반응이 우려될 수 있으니 성분을 확인하신 후 섭취 하십시오.\r\n- 개봉된 제품은 이물질이 혼입되지 않도록 뚜껑으로 밀봉하여 보관 하십시오."
4,[홍삼제품]①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n?- 알러지 등 특이체질의 경우 과민반응이 우려될 수 있으니 성분을 확인하신 후 섭취 하십시오.\r\n- 제품 개봉 또는 섭취시 포장재에 의한 상처를 입을 수 있으니 주의 하십시오.\r\n- 개봉된 제품은 이물질이 혼입되지 않도록 뚜껑으로 밀봉하여 보관 하십시오."
5,[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[비타민E]①유해산소로부터 세포를 보호하는데 필요\r\n[구리]①철의 운반과 이용에 필요②유해산소로부터 세포를 보호하는데 필요\r\n[망간]①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[셀레늄(또는 셀렌)]①유해산소로부터 세포를 보호하는데 필요,"과량섭취시 부작용이 있을 수 있으며, 특정 원료에 알레르기가 있거나 질병 치료나 약물투여 중인 분들은 섭취 전 전문가와 상의하십시오. \r\n?제품 개봉 또는 섭취시 포장재에 의한 상처를 입을수 있으니 주의하시기 바랍니다. \r\n-유통기한이 경과된 제품은 섭취하지 마십시오."
6,"[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요\r\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\r\n[비타민B12]①정상적인 엽산 대사에 필요\r\n[나이아신]①체내 에너지 생성에 필요\r\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\r\n[엽산]①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요","과량섭취시 부작용이 있을 수 있으며, 특정 원료에 알레르기가 있거나 질병 치료나 약물투여 중인 분들은 섭취 전 전문가와 상의하십시오. \r\n?제품 개봉 또는 섭취시 포장재에 의한 상처를 입을수 있으니 주의하시기 바랍니다. \r\n-유통기한이 경과된 제품은 섭취하지 마십시오."
7,"[나이아신]①체내 에너지 생성에 필요\r\n[비오틴]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\r\n[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B12]①정상적인 엽산 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요\r\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[비타민E]①유해산소로부터 세포를 보호하는데 필요\r\n[엽산]①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\r\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","1. 특이체질, 알레르기 체질인 경우에는 간혹 개인에 따라 과민반능을 나타낼 수 있으므로 원료를 확인한 후 섭?십시오.\r\n2. 어린이가 정제를 삼킬 경우 목에 걸릴 수 있으니 그냥 삼키지 않도록 보호자의 주의가 필요합니다."
8,[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌\n[알로에겔 제품]①피부건강에 도움을 줄 수 있음 ②장 건강에 도움을 줄 수 있음 ③면역력 증진에 도움을 줄 수 있음,"1) 임산부, 수유부, 알레르기 체질, 노약자 및 치료를 받고 계신 중증 질환자 등은 구입처로 문의 후 섭취하시기 바랍니다.\r\n2) 본제품의 섭취량은 성인 기준으로 표시되어 있습니다. 영?유아 및 어린이에게 섭취하게 하실 경우에는 구입처 및 고객 상담실로 문의하시고 섭취 시 보호자의 관찰 하에 섭취하도록 하십시오.\r\n3) 권장섭취량을 확인하신 후 섭취하십시오.\r\n4) 섭취자의 신체상태에 따라 특이반응이 나타날 수 있습니다. 이 경우 섭취를 중단하신 후 구입처로 문의하시기 바랍니다.\r\n5) 제품의 보관상태 및 천연 원료에 의해 약간의 점도 및 색상의 변화가 있을 수 있으나 품질에는 이상이 없으니 안심하고 드셔도 됩니다.\r\n6) 천연 원료에 의해 침전물이 생길 수 있으나 이물질이 아니오니 안심하시고 잘 흔들어 드십시오. \r\n7) 본제품은 우유, 대두, 게, 밀, 복숭아, 토마토를 사용한 제품과 같은 시설에서 제조하였습니다. \r\n8) 섭취 시 계량컵이나 컵에 따라 드시기 바랍니다."
9,"[비타민B1]①탄수화물과 에너지 대사에 필요\r\n[비타민B2]①체내 에너지 생성에 필요\r\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\r\n[비타민B12]①정상적인 엽산 대사에 필요\r\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③유해산소로부터 세포를 보호하는데 필요\r\n[나이아신]①체내 에너지 생성에 필요\r\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\r\n[비오틴]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\r\n[엽산]①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요","과량섭취시 부작용이 있을 수 있으며, 특정 원료에 알레르기가 있거나 질병 치료나 약물투여 중인 분들은 섭취 전 전문가와 상의하십시오. \r\n?제품 개봉 또는 섭취시 포장재에 의한 상처를 입을 있으니 주의하시기 바랍니다.\r\n-유통기한이 경과된 제품은 섭취

In [253]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["_"])[['product_function', 'product_caution']] # '_' 기호는 ' '로 대체

,product_function,product_caution
0,바나바잎추출물 _ ①식후 혈당상승 억제에 도움을 줄 수 있음\n\n아연 _ ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n비타민B1 _ (1) 탄수화물과 에너지 대사에 필요,비타민B1 _ (1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
1,밀크씨슬추출물_ ① 간 건강에 도움을 줄 수 있음\n\n비타민B1 _ ①탄수화물과 에너지 대사에 필요\n\n비타민B2_①체내 에너지 생성에 필요\n\n비타민B6_①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n나이아신 _ ①체내 에너지 생성에 필요,"(1) 어린이, 임산부 및 수유부는 섭취를 피할 것 , (2) 알레르기 반응이 나타나는 경우에는 섭취 중단 , (3) 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취에 주의 (4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [254]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["`"])[['product_function', 'product_caution']] # 영문 사이에 사용된 것이 아니면 제거

,product_function,product_caution
0,(국문) 갱년기 여성건강에 도움을 줄 수 있음\n(영문) May help to menopausal women`s health,· 임산부 및 수유부는 섭취를 피할 것\n· 알레르기를 나타내는 사람은 섭취 시 주의\n· 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n· 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n· 항우울제 또는 호르몬제를 복용하는 사람은 섭취에 주의
1,[호로파종자추출물(제2019-15호)]갱년기 남성 건강에 도움을 줄 수 있음\n(영문) May help to climacteric men`s health,"1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 유통기한이 경과된 제품은 섭취하지 마십시오.\n[호로파종자추출물(제2019-15호)]\n1) 땅콩 알레르기 있는 사람은 섭취에 주의\n2) 영·유아, 어린이. 임산부 및 수유부는 섭취에 주의\n3) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담 할 것"
2,(국문) 갱년기 여성건강에 도움을 줄 수 있음\n(영문) May help to menopausal women`s health,"·영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n·에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n·메스꺼움 등 소화계통의 불편함과 설사를 유발 할 수 있음\n·알레르기 체질이거나 특정질환이 있는 사람은 섭취 전 전문가와 상담할 것\n·이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,"(국문) 갱년기 여성 건강에 도움을 줄 수 있음 (영문) May help to menopausal women`s health\n- 유산균 증식 및 유해균 억제. 배변활동 원활, 장건강에 도움을 줄 수 있음",① 질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담하십시오.\n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.\n③ 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하십시오.\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
4,(국문) 갱년기 여성건강에 도움을 줄 수 있음\n(영문) May help to menopausal women`s health\n①스트레스로 인한 긴장 완화에 도움을 줄 수 있음\n항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n①탄수화물과 에너지 대사에 필요\n①체내 에너지 생성에 필요,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n· 임산부 및 수유부는 섭취를 피할 것\n· 알레르기를 나타내는 사람은 섭취 시 주의\n· 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n· 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n· 항우울제 또는 호르몬제를 복용하는 사람은 섭취에 주의\n(가) 임산부, 수유여성 및 어린이는 섭취에 주의\n(나) 우유 및 유제품에 대하여 알레르기를 나타내는 사람은 섭취에 주의\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,(국문) 갱년기 여성 건강에 도움을 줄 수 있음 (영문) May help to menopausal women`s health\n- 유산균 증식 및 유해균 억제. 배변활동 원활. 장건강에 도움을 줄 수 있음,․질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것 \n․알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음 \n․어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것 \n․이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
6,"칼슘 : ①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌\n\n백수오 등 복합추출물 : (국문) 갱년기 여성의 건강에 도움을 줄 수 있음(기타Ⅱ) (영문) May help to men-opausal women`s health\n\n비타민C :①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n아연 :①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n비타민D :①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n비타민B6 :①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n비타민B12 :①정상적인 엽산 대사에 필요\n\n엽산 : ①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n비오틴 :①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","① 임산부와 수유부는 섭취를 피할 것\n② 항응고제, 항혈전제 등 복용 시 전문가와 상담할 것\n③ 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n⑤ 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
7,[MS-10 엉겅퀴등복합추출물 (제2020-6호)]\n(국문) 갱년기 여성건강에 도움을 줄 수 있음\n(영문) May help to menopausal women`s health\n[비타민C]\n결합조직 형성과 기능유지에 필요 / 철의 흡수에 필요 / 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요 \n[테아닌]\n스트레스로 인한 긴장완화에 도움을 줄 수 있음\n[비타민D]\n칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌,"· 임산부 및 수유부는 섭취를 피할 것\n· 알레르기를 나타내는 사람은 섭취 시 주의\n· 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n· 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n· 항우울제 또는 호르몬제를 복용하는 사람은 섭취에 주의\n· 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n· 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의\n· 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의"
8,호로파종자추출물 : (국문) 갱년기 남성 건강에 도움을 줄 수 있음 (영문) May help to climacteric men`s health\n\n은행잎추출물 : ①기억력 개선에 도움을 줄 수 있음 ②혈행 개선에 도움을 줄 수 있음,"① 땅콩 알레르기가 있는 사람은 섭취에 주의\n② 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의 \n③ 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n⑤ 의약품(항응고제) 복용 시 섭취에 주의\n⑥ 질병치료 및 수술전후 환자는 섭취에 주의."
9,(국문)\n- 갱년기 여성의 건강에 도움을 줄 수 있음\n- 피부보습에 도움을 줄 수 있음\n- 자외선에 의한 피부 손상으로부터 피부건강을 유지하는데 도움을 줄 수 있음\n(영문)\n- May help to menopausal women`s health\n- May help to maintain skin moisturizing\n- May help to maintain skin health from skindamage by UV radiation\n\n항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"(1) 어린이, 임산부와 수유부는 섭취를 피하는 것이 좋습니다. (2) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다.\n\n\n\n\n\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전

In [255]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["}"])[['product_function', 'product_caution']] # 오타인 경우 ']'로 예외사항 처리. 중괄호 제거

,product_function,product_caution
0,{리스펙타(Respecta®) [프로바이오틱스 등 복합물](제2019-26호)}\n(국문) 질내 유익균 증식 및 유해균 억제에 도움을 줄 수 있음 \n(영문) May help to proliferate vaginal beneficial flora and to suppress pathogenic bacteria\n\n[프로바이오틱스] 유산균 증식 및 유해균 억제 • 배변활동 원활 • 장건강에 도움을 줄 수 있음\n\n[히알루론산] 피부보습에 도움을 줄 수 있음,"영•유아, 어린이, 임산부 및 수유부는 섭취에 주의\n우유, 유제품 및 기타 제품에 대해 알레르기가 있는 사람은 섭취에 주의\n질환이 있거나 의약품 복용 시 전문가와 상담할 것\n알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,"[녹차추출물]\n-항산화･체지방 감소･혈중 콜레스테롤 개선에 도움을 줄 수 있음\n\n[회화나무열매추출물}\n-갱년기 여성의 건강에 도움을 줄 수 있음\n\n[비타민B2]\n-체내 에너지 생성에 필요\n\n[나이아신]\n-체내 에너지 생성에 필요\n\n[비타민C]\n- 결합조직 형성과 기능유지에 필요\n- 철의 흡수에 필요\n- 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n[비오틴]\n- 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비타민B6]\n-단백질 및 아미노산 이용에 필요\n-혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[판토텐산]\n- 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비타민B1]\n- 탄수화물과 에너지 대사에 필요\n\n[비타민E]\n-항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n[비타민D]\n-칼슘과 인이 흡수되고 이용되는데 필요\n-뼈의 형성과 유지에 필요\n-골다공증발생 위험 감소에 도움을 줌\n\n[아연]\n-정상적인 면역기능에 필요\n-정상적인 세포분열에 필요\n\n[철]\n-체내 산소운반과 혈액생성에 필요\n-에너지 생성에 필요","(가) 어린이, 임산부 및 수유부는 섭취에 주의하십시오.\n(나) 간질환이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다.\n(다) 카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있으니 주의하시기 바랍니다.\n(라) 식사 후 섭취하십시오.\n(마) 카페인을 함유한 식품의 섭취에 주의하십시오.\n(바) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다.\n(사) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의하시기 바랍니다.\n(아) 특히 6세 이하는 과량섭취하지 않도록 주의하시기 바랍니다.\n(자) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다."
2,[녹차추출물]\n-항산화･체지방 감소･혈중 콜레스테롤 개선에 도움을 줄 수 있음\n\n[은행잎추출물}\n-기억력 개선･혈행 개선에 도움을 줄 수 있음\n\n[비타민B2]\n-체내 에너지 생성에 필요\n\n[나이아신]\n-체내 에너지 생성에 필요\n\n[셀렌]\n-유해산소로부터 세포를 보호하는데 필요,"(가) 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오.\n(나) 간질환이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다.\n(다) 카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있으니 주의하시기 바랍니다.\n(라) 식사 후 섭취하십시오.\n(마) 카페인을 함유한 식품의 섭취에 주의하십시오.\n(바) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다.\n(사) 의약품(항응고제) 복용 시 섭취에 주의하시기 바랍니다."
3,[비타민 C} ①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,NaN
4,"[비타민 C] : ①결합조직 형성과 기능유지에 필요 ②철의 흡수에 필요 ③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민 E] : ①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[철] : ①체내 산소운반과 혈액생성에 필요 ②에너지 생성에 필요\n[판토텐산] : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민A] : ①어두운 곳에서 시각 적응을 위해 필요 ②피부와 점막을 형성하고 기능을 유지하는데 필요 ③상피세포의 성장과 발달에 필요\n[아연] : ①정상적인 면역기능에 필요 ②정상적인 세포분열에 필요\n[비타민 B12] : ①정상적인 엽산 대사에 필요 \n[비타민B6] : ①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민D] : ①칼슘과 인이 흡수되고 이용되는데 필요 ②뼈의 형성과 유지에 필요 ③골다공증발생 위험 감소에 도움을 줌\n[비타민B2] : ①체내 에너지 생성에 필요 \n[나이아신] : ①체내 에너지 생성에 필요\n[비타민B1] : ①탄수화물과 에너지 대사에 필요\n[비타민K] : ①정상적인 혈액응고에 필요 ②뼈의 구성에 필요\n[엽산] : ①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴] : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",[철] ①특히 6세 이하는 과량섭취하지 않도록 주의하십시오.\n[판토텐산] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B12} ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 D] ①고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오. ②이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B2] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B1] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 K] ①항응고제 등 복용 시 전문가와 상담하십시오 ②이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비오틴] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
5,"[비타민 C] : ①결합조직 형성과 기능유지에 필요 ②철의 흡수에 필요 ③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민 E] : ①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[철] : ①체내 산소운반과 혈액생성에 필요 ②에너지 생성에 필요\n[판토텐산] : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민A] : ①어두운 곳에서 시각 적응을 위해 필요 ②피부와 점막을 형성하고 기능을 유지하는데 필요 ③상피세포의 성장과 발달에 필요\n[아연] : ①정상적인 면역기능에 필요 ②정상적인 세포분열에 필요\n[비타민 B12] : ①정상적인 엽산 대사에 필요 \n[비타민B6] : ①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민D] : ①칼슘과 인이 흡수되고 이용되는데 필요 ②뼈의 형성과 유지에 필요 ③골다공증발생 위험 감소에 도움을 줌\n[비타민B2] : ①체내 에너지 생성에 필요 \n[나이아신] : ①체내 에너지 생성에 필요\n[비타민B1] : ①탄수화물과 에너지 대사에 필요\n[비타민K] : ①정상적인 혈액응고에 필요 ②뼈의 구성에 필요\n[엽산] : ①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴] : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",[철] ①특히 6세 이하는 과량섭취하지 않도록 주의하십시오.\n[판토텐산] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B12} ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오\n[비타민 D] ①고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오. ②이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B2] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 B1] ①이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민 K]

In [256]:
find_chars_df(p_df, ['product_function', 'product_caution'], ["~"])[['product_function', 'product_caution']] # '~' 기호는 그대로 둔다

,product_function,product_caution
0,"1.면역력 증진 ･ 피로개선에 도움을 줄 수 있음 : 진세노사이드 Rg1,Rb1 및 Rg3의 합계로서 3 ~ 80 mg\n2.혈소판 응집억제를 통한 혈액흐름 ･ 기억력 개선 ･ 항산화에 도움을줄 수 있음 : 진세노사이드 Rg1, Rb1 및 Rg3의 합계로서 2.4 ~ 80 mg",1.당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의\n2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n3. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
1,관절 및 연골건강에 도움을 줄 수 있음 : N-아세틸글루코사민으로서 0.5 ~ 1 g,N-아세틸글루코사민: 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (새우 원재료 사용)
2,"* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* N-아세틸글루코사민\n① 관절 및 연골건강에 도움을 줄 수 있음 (: N-아세틸글루코사민으로서 0.5~1 g)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 관절 및 연골 건강에 도움을 줄 수 있음",* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* N-아세틸글루코사민\n① 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (게 또는 새우를 원재료로 사용한 경우에 한함)
3,"* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* N-아세틸글루코사민\n① 관절 및 연골건강에 도움을 줄 수 있음 (: N-아세틸글루코사민으로서 0.5~1 g)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 관절 및 연골 건강에 도움을 줄 수 있음\n\n* 아연\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요","* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* N-아세틸글루코사민\n① 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (게 또는 새우를 원재료로 사용한 경우에 한함) \n\n*엠에스엠(MSM, Methyl sulfonylmethane, 디메틸설폰)\n① 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
4,"* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* N-아세틸글루코사민\n① 관절 및 연골건강에 도움을 줄 수 있음 (: N-아세틸글루코사민으로서 0.5~1 g)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 관절 및 연골 건강에 도움을 줄 수 있음","* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* N-아세틸글루코사민\n① 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (게 또는 새우를 원재료로 사용한 경우에 한함)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,"*비타민 D\n①칼슘과 인이 흡수되고 이용되는데 필요\n②뼈의 형성과 유지에 필요\n③골다공증발생 위험 감소에 도움을 줌\n\n*비타민 K\n①정상적인 혈액응고에 필요\n②뼈의 구성에 필요\n\n*셀렌\n유해산소로부터 세포를 보호하는데 필요\n\n* N-아세틸글루코사민\n① 관절 및 연골건강에 도움을 줄 수 있음 (: N-아세틸글루코사민으로서 0.5~1 g)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 관절 및 연골 건강에 도움을 줄 수 있음","* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n비타민 K\n1. 항응고제 등 복용 시 전문가와 상담할 것\n2. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* N-아세틸글루코사민\n① 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (게 또는 새우를 원재료로 사용한 경우에 한함)\n\n* 엠에스엠(MSM, 디메틸설폰)\n① 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
6,"* 마그네슘\n① 에너지 이용에 필요\n② 신경과 근육 기능 유지에 필요\n\n* 비오틴\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n* 비타민B1\n① 탄수화물과 에너지 대사에 필요\n\n* 비타민B2\n① 체내 에너지 생성에 필요\n\n* 비타민B6\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n* 비타민C\n① 결합조직 형성과 기능유지에 필요\n② 철의 흡수에 필요\n③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* 아연\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n\n* 키토산 / 키토올리고당\n① 혈중 콜레스테롤 개선·체지방 감소에 도움을 줄 수 있음\n\t1) 혈중 콜레스테롤 개선에 도움을 줄 수 있음 ( : 키토산 또는 키토올리고당으로서 1.2~4.5 g)\n\t2) 체지방 감소에 도움을 줄 수 있음 ( : 키토산으로서 3.0~4.5 g, 키토올리고당으로서 3 g)\n\n* 판토텐산 ① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요",* 비오틴\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민B1\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민D\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 키토산 / 키토올리고당\n① 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의 (게 또는 새우를 원재료로 사용한 경우에 한함)\n\n* 판토텐산\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
7,"* 대두이소플라본\n① 뼈 건강에 도움을 줄 수 있음\n\n* 비타민A\n① 어두운 곳에서 시각 적응을 위해 필요\n② 피부와 점막을 형성하고 기능을 유지하는데 필요\n③ 상피세포의 성장과 발달에 필요\n\n* 비타민B1\n① 탄수화물과 에너지 대사에 필요\n\n* 비타민B2\n① 체내 에너지 생성에 필요\n\n* 비타민B6\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n* 비타민C\n① 결합조직 형성과 기능유지에 필요\n② 철의 흡수에 필요\n③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n* 비타민D\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n\n* 비타민E\n① 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n* 아연\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n\n* 홍삼\n① 면역력 증진･피로개선에 도움을 줄 수 있음 ( : 진세노사이드 Rg1, Rb1 및 Rg3의 합계로서 3~80 mg)\n② 혈소판 응집억제를 통한 혈액흐름･기억력 개선･항산화에 도움을 줄 수 있음 ( : 진세노사이드 Rg1, Rb1 및 Rg3의 합계로서 2.4~80 mg)\n\n* 회화나무열매추출물\n① 갱년기 여성의 건강에 도움을 줄 수 있음\n\n*프로바이오틱스\n① 유...","* 대두이소플라본\n① 영유아, 어린이, 임산부와 수유부는 섭취에 주의\n② 대두에 알레르기를 나타내는 사람은 섭취에 주의\n③ 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n\n* 비타민B1\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n* 비타민B2\n① 이상사례 발생 시 섭취를 중단하고

In [257]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['\xad', '\ufeff'])[['product_function', 'product_caution']] # pass

product_function 컬럼값에 "­" 문자열이 없습니다.


,product_function,product_caution
0,﻿[핑거루트추출분말(판두라틴)(기능성원료 인정 제2013-5호)]\n① 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n② 체지방감소에 도움을 줄 수 있음\n③ 피부보습에 도움을 줄 수 있음,NaN
1,﻿[철]\n① 체내 산소운반과 혈액생성에 필요\n② 에너지 생성에 필요,﻿[철]\n① 특히 6세 이하는 과량섭취하지 않도록 주의하십시오.
2,﻿[피크노제놀-프랑스해안송껍질추출물(제2006-1호)]\n① 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n② 갱년기 여성의 건강에 도움을 줄 수 있음\n[테아닌]\n스트레스로 인한 긴장완화에 도움을 줄 수 있음,"﻿[피크노제놀-프랑스해안송껍질추출물(제2006-1호)]\n① 임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것\n② 수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n③ 식사 후 또는 식사와 함께 할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n[테아닌]\n① 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하십시오.\n② 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오."
3,"[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n[비타민C]①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[나이아신]①체내 에너지 생성에 필요\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B1]①탄수화물과 에너지 대사에 필요\n[비타민B2]①체내 에너지 생성에 필요","(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(라) 뚜껑에 있는 습기제거제는 먹어서는 안됩니다.\n(마) 반드시물과 함께 드십시오.­"
4,"①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n\n①체내 에너지 생성에 필요\n\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","반드시물과 함께 드십시오.­\n임산부와 수유기 여성, 어린이는 섭취를 피하는 것이 좋습니다.\n­간질환, 신장질환, 심장질환 등의 기능에 이상이 있는 경우와 복용하는 약물이 있는 경우 섭취 전 전문가와 상담하시기 바랍니다.\n­알레르기 또는 천식을 보유한 경우 섭취 전 전문가와 상담하시기 바랍니다.\n­이상증상발생시 섭취를 중단하고 전문가와 상담하시기 바랍니다.\n뚜껑에 있는 습기제거제는 먹어서는 안됩니다."
5,"[은행잎 추출물]\n① 기억력 개선･혈행 개선에 도움을 줄 수 있음\n[HK나토배양물]\n① 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하여, 혈액 흐름을 원활히 하는데 도움을 줄 수 있음\n[아연]\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n[비타민 D]\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n[철]\n① 체내 산소운반과 혈액생성에 필요\n② 에너지 생성에 필요","﻿[은행잎 추출물]\n① 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오.\n② 의약품(항응고제) 복용 시 섭취에 주의하십시오.\n[HK나토배양물]\n① 알레르기 체질이신 분은 원재료를 확인하신 후 섭취하십시오.\n② 임산부와 수유부 여성은 섭취를 피하는 것이 좋습니다.\n③ 과다 섭취 시 혈액응고가 저해되어 출혈 위험이 증가될 수 있습니다.\n④ 항혈전 관련 약품(항응고제, 항혈소판제제)을 섭취하는 사람은 의사와 상담 후 섭취하여야 합니다.\n⑤ 수술 전과 후에는 섭취를 피해야 합니다.\n[비타민 D]\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[철]\n① 특히 6세 이하는 과량섭취하지 않도록 주의하십시오."
6,"[코엔자임Q10]\n① 항산화•높은 혈압 감소에 도움을 줄 수 있음\n[비타민C]\n① 결합조직 형성과 기능유지에 필요 \n② 철의 흡수에 필요\n③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[나이아신]\n① 체내 에너지 생성에 필요\n[아연]\n① 정상적인 면역기능에 필요 ② 정상적인 세포분열에 필요\n[판토텐산]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B1]\n① 탄수화물과 에너지 대사에 필요\n[비타민B2]\n① 체내 에너지 생성에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[엽산]\n① 세포와 혈액생성에 필요\n② 태아 신경관의 정상 발달에 필요\n③ 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","﻿[비타민 B1, 비타민 B2, 판토텐산, 비오틴]\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
7,[홍삼]\n① 면역력 증진･피로개선･혈소판 응집억제를 통한 혈액흐름･기억력 개선･항산화에 도움을 줄 수 있음\n[아연]\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n[비타민B2]\n① 체내 에너지 생성에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[나이아신]\n① 체내 에너지 생성에 필요,﻿[홍삼]\n① 당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의 하십시오. \n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. \n③ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민B2]\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것


In [258]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['®'])[['product_function', 'product_caution']] # '®' -> 'Ⓡ'로 대체, 'Ⓡ'는 제거X

,product_function,product_caution
0,"1) 가자추출물(AyuFlex®) : 관절 및 연골 건강에 도움을 줄 수 있음. \n2) 비타민C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n3) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌. \n4) 비타민K : 정상적인 혈액응고에 필요, 뼈의 구성에 필요\n5) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요. \n6) 셀렌 : 유해산소로부터 세포를 보호하는데 필요\n7) 망간 : 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n영·유아, 어린이, 임산부 및 수유부는 섭취에 주의하십시오.\n특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오. \n항응고제 등 복용 시 전문가와 상담하십시오."
1,"[돌외잎주정추출분말(액티포닌®)] (인정번호 : 제2013-8호)\n(국문) 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)\n(영문) May help reduce body fat mass(Other function claim 2 grade)\n\n[나이아신] 체내 에너지 생성에 필요\n\n[비타민B1] 탄수화물과 에너지 대사에 필요\n\n[비타민B2] 체내 에너지 생성에 필요\n\n[비타민C] ①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n[비타민D] ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n[판토텐산] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[철] ①체내 산소운반과 혈액생성에 필요②에너지 생성에 필요","[돌외잎주정추출분말(액티포닌®)]\n- 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것\n- 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n- 일일섭취량 이상으로 섭취 시 위장장애 (구토와 위장운동 증가)가 발생할 수 있음.\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n특히 6세 이하는 과량섭취하지 않도록 주의"
2,[천심련추출물(ParActin®)(제2022-3호)]\n- (국문) 관절 건강에 도움을 줄 수 있음\n- (영문) May help to maintain healthy joint\n[비타민D]\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌,"[천심련추출물(ParActin®)(제2022-3호)]\n1. 영·유아, 어린이, 청소년, 임산부 및 수유부, 임신을 계획하고 있는 자는 섭취하지 말 것 \n2. 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의 \n3. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n4. 의약품을 복용하시는 분은 섭취 전 전문의와 상담할 것 \n[비타민D]\n5. 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n6. 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,[타히보 추출물 (Tabetri®)(제2022-2호)]\nㆍ관절 및 연골 건강 개선에 도움을 줄 수 있음\n[비타민 C] \nㆍ결합조직 형성과 기능유지에 필요 ㆍ철의 흡수에 필요 ㆍ항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민 A]\nㆍ어두운 곳에서 시각 적응을 위해 필요 ㆍ피부와 점막을 형성하고 기능을 유지하는데 필요 ㆍ상피세포의 성장과 발달에 필요\n[비타민 D]\nㆍ칼슘과 인이 흡수되고 이용되는데 필요 ㆍ뼈의 형성과 유지에 필요 ㆍ골다공증발생 위험 감소에 도움을 줌\n[셀렌]\nㆍ유해산소로부터 세포를 보호하는데 필요\n[아연]\nㆍ정상적인 면역기능에 필요 ㆍ정상적인 세포분열에 필요,"영 ･유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것\n- 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 항응고제 또는 면역억제제를 복용하는 분은 섭취에 주의할 것\n- 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
4,가자추출물(Ayuflex®)(제2022-12호) : (국문) 관절 및 연골 건강에 도움을 줄 수 있음 (영문) May help to maintain joint and cartilage health\n셀렌 : 유해산소로부터 세포를 보호하는데 필요\n아연 : (1) 정상적인 면역기능에 필요 (2) 정상적인 세포분열에 필요\n망간 : (1) 뼈 형성에 필요 (2) 에너지 이용에 필요 (3) 유해산소로부터 세포를 보호하는데 필요\n비타민D : (1) 칼슘과 인이 흡수되고 이용되는데 필요 (2) 뼈의 형성과 유지에 필요 (3) 골다공증발생 위험 감소에 도움을 줌\n비타민K : (1) 정상적인 혈액응고에 필요 (2) 뼈의 구성에 필요,"(1) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(4) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(5) 항응고제 등 복용 시 전문가와 상담할 것"
...,...,...
117,[가자추출물(기능성원료인정제2022-12호)]\n(국문) 관절 및 연골 건강에 도움을 줄 수 있음\n(영문) May help to maintain joint and cartilage health\n[아연] ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n[셀레늄] ①유해산소로부터 세포를 보호하는데 필요\n[비타민K](1) 정상적인 혈액응고에 필요(2) 뼈의 구성에 필요\n[망간] ①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\n[비타민D] ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 (가자추출물(Ayuflex®), 비타민K, 비타민D)\n(1) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n항응고제 등 복용 시 전문가와 상담할 것\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
118,(국문) 관절 및 연골 건강에 도움을 줄 수 있음\n(영문) May help to maintain joint and cartilage health\n(1) 정상적인 혈액응고에 필요(2) 뼈의 구성에 필요\n①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 (가자추출물(Ayuflex®), 비타민K, 비타민D)\n(1) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n(2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n(1) 항응고제 등 복용 시 전문가와 상담할 것\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
119,[돌외잎주정추출물(액티포닌Ⓡ)(제2013-8호)

In [259]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['±'])[['product_function', 'product_caution']] # '±'는 제거X

product_function 컬럼값에 "±" 문자열이 없습니다.


,product_function,product_caution
0,①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n①정상적인 엽산 대사에 필요,"1)원료의 선택 : 모든 원료는 건강기능식품공전, 식품공전, 식품첨가물공전에서 정하는\n\t\t 기준 및 규격에 적합한 원료만을 사용한다.\n 2)제조설비의 소독 : 사용되는 모든 기기 및 기구는 세척 후 주정으로 소독, 건조하여 사용한다.\n 3)칭 량 : 1회 배합에 사용되는 원료량을 기준으로 칭량하며 전자저울로 칭량한다.\n 4)선 별 : 분말원료는 진동체로 선별하여 이물을 제거하며 원료 입자의 크기를 균질화 한다.\n 5)혼 합 및 살균 : 원료를 배합비율에 맞추어 혼합탱크에 투입하여 100℃(±2℃)에서 15~30분간\n\t\t 균질하게 혼합 및 살균 후 필터를 거쳐 배출한다.\n 6)충진 및 포장 : 배출된 원료를 스틱 충진기로 15g 씩 충진 한 후 단상자에 넣어 포장한다.\n 7)완제품 검사 : 기준 및 규격에 맞게 자가 품질검사를 실시하여 적합여부를 판정한다.\n 8)출 하 : 품질관리인으로부터 최종 적합 판정된 완제품을 출하한다."


In [260]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['ß', 'β'])[['product_function', 'product_caution']] # 제거X

product_function 컬럼값에 "ß" 문자열이 없습니다.
product_function 컬럼값에 "β" 문자열이 없습니다.


,product_function,product_caution
0,[EPA 및 DHA 함유 유지] 혈중 중성지질 개선 · 혈행 개선 · 기억력 개선 · 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스추출물] 눈의 피로도 개선에 도움을 줄 수 있음 \n[비타민B2] 체내 에너지 생성에 필요\n[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌 \n[마그네슘] 에너지 이용에 필요 / 신경과 근육 기능 유지에 필요,"소비기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다. \n- 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다. \n- 제품의 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다.\n- 어린이의 경우 섭취 시 목에 걸릴 우려가 있으니 보호자의 지도하에 섭취하십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n- 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n- 개인에 따라 피부 관련 이상반응이 발생할 수 있음\n- 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n- ß-카로틴의 흡수를 저해할 수 있음"
1,[EPA 및 DHA 함유 유지] 혈중 중성지질 개선 · 혈행 개선 · 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스추출물] 눈의 피로도 개선에 도움을 줄 수 있음 \n[비타민A] 어두운 곳에서 시각 적응을 위해 필요 / 피부와 점막을 형성하고 기능을 유지하는데 필요 / 상피세포의 성장과 발달에 필요\n[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌 \n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n③ 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것\n④ 개인에 따라 피부관련 이상반응이 발생할 수 있음\n⑤ 영·유아, 어린이, 임산부 및 수유부는 섭취를 피하십시오.\n⑥ 흡연자는 섭취 시 전문가와 상담할 것\n⑦ 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n⑧ ß-카로틴의 흡수를 저해할 수 있음"
2,[마리골드꽃추출물]노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스추출물]①눈의 피로도 개선에 도움을 줄 수 있음\n[비타민A]①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n[비타민E]①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[베타카로틴]①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n[셀렌]①유해산소로부터 세포를 보호하는데 필요,"특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다. \n-유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n-과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있습니다.\n-ß-카로틴의 흡수를 저해할 수 있습니다.\n-흡연자는 섭취 시 전문가와 상담하십시오.\n-이상사례 발생 시 섭취를 중단하고 전문가와 상답하십시오."
3,"1) 마리골드꽃추출물 : 노화로 인해 감소될 수 있느 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음.\n2) 헤마토코쿠스추출물 : 눈의 피로도 개선에 도움을 줄 수 있음.\n3) 비타민A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요.\n4) 베타카로틴 : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요.\n5) 비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,\n6) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요.\n7) 셀렌 : 유해산소로부터 세포를 보호하는데 필요.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오.\n섭취 시 위장장애. 소화불량의 중상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있습니다.\nß-카로틴의 흡수를 저해할 수 있습니다.\n흡연자는 섭취 시 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n영.유아, 어린이, 임산부 및 수유부는 섭취를 피하십시오."
4,[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스추출물] 눈의 피로도 개선에 도움을 줄 수 있음,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n③ 영·유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n④ 흡연자는 섭취 시 전문가와 상담할 것\n⑤ 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n⑥ 이상사례 발생시 섭취를 중단하고 전문가와 상담할 것\n⑦ ß-카로틴의 흡수를 저해할 수 있음"
...,...,...
405,"[EPA 및 DHA 함유 유지] 혈중 중성지질 개선, 혈행 개선, 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스 추출물] 눈의 피로도 개선에 도움을 줄 수 있음\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요 / 정상적인 세포분열에 필요\n[코엔자임Q10] 항산화, 높은 혈압 감소에 도움을 줄 수 있음\n[히알루론산] 피부보습에 도움을 줄 수 있음","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n소비기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담하십시오.\n개인에 따라 피부 관련 이상반응이 발생할 수 있습니다.\n영, 유아, 어린이, 임산부 및 수유부는 섭취를 피하십시오.\n흡연자는 섭취 시 전문가와 상담하십시오.\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있습니다.\nβ-카로틴의 흡수를 저해할 수 있습니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
406,"[EPA 및 DHA 함유 유지] 혈중 중성지질 개선, 혈행 개선, 건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[헤마토코쿠스 추출물] 눈의 피로도 개선에 도움을 줄 수 있음\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요 / 정상적인 세포분열에 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n소비기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담하십시오.\n개인에 따라 피부 관련 이상반응이 발생할 수 있습니다.\n영,유아, 어린이, 임산부 및 수유부는 섭취를 피하십시오.\n흡연자는 섭취 시 전문가와 상담하십시오.\n과다 섭취 시 일시적으로 피부가 황색으로 변

In [261]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['–'])[['product_function', 'product_caution']] # 구분자로 사용되어서 '\n'으로 대체

,product_function,product_caution
0,"아연 - 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n참당귀추출분말(Nutragen)(기능성원료인정 제2014-44호) – 노화로 저하된 인지기능 개선에 도움을 줄 수 있음, 관절건강에 도움을 줄 수 있음(생리활성기능 2등급)","·소화불량, 속쓰림 등이 나타날 수 있습니다. ·혈액응고방지제 또는 혈당강하제를 복용하시는 분은 의사의 상담 하에 사용하여 주십시오."
1,"1)비타민A - 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n2)비타민B1 - 탄수화물과 에너지 대사에 필요\n3)비타민B6 - 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n4)나이아신 - 체내 에너지 생성에 필요\n5)엽산 - 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6)비타민E – 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n7)비타민C - 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n8)비타민B12 - 정상적인 엽산 대사에 필요\n9)구리 - 철의 운반과 이용에 필요, 유해산소로부터 세포를 보호하는데 필요\n10)아연: 정상적인 면역기능에 필요, 정상적인 세포분열에 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 제품의 특성상(흡습성) 색이 진해지거나 연해질 수 있습니다.\n- 용기 안의 방습제는 섭취하지 마십시오."
2,"1)비타민B1 -탄수화물과 에너지 대사에 필요\n2)비타민B2 - 체내 에너지 생성에 필요\n3)비타민B6 -(가) 단백질 및 아미노산 이용에 필요\n (나) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n4)판토텐산 - 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n5)나이아신 - 체내 에너지 생성에 필요\n6)엽산 - (가) 세포와 혈액생성에 필요\n (나) 태아 신경관의 정상 발달에 필요\n (다) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n7)셀레늄 - 유해산소로부터 세포를 보호하는데 필요\n8)비타민B12 - 정상적인 엽산 대사에 필요\n9)비오틴 - 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n10)철 - (가) 체내 산소운반과 혈액생성에 필요\n (나) 에너지 생성에 필요\n11)아연 - (가) 정상적인 면역기능에 필요\n (나) 정상적인 세포분열에 필요\n12)비타민A - (가) 어두운 곳에서 시각 적응을 위해 필요\n (나) 피부와 점막을 형성하고 기능을 유지하는데 필요\n (다) 상피세포의 성장과 발달에 필요\n13)비타민D -(가) 칼슘과 인이 흡수되고 이용되는데 필요\n (...","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n3) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n4) 항응고제 등 복용 시 전문가와 상담할 것\n5) 특히 6세 이하는 과량 섭취하지 않도록 주의하십시오.\n6) 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것\n7) 식물성 원료가 함유되어 직사광선 등에 의해 색이 변화될 수 있으나 제품 품질에는 이상이 없으므로 안심하고 섭취하셔도 좋습니다."
3,– 배변활동 원활에 도움을 줄 수 있음.\n- 탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 \n줄 수 있음.,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(다) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
4,[홍삼제품]①면역력 증진②피로회복③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음,"– 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의하십시오. \n- 어린이, 임산부, 수유부, 혈압이 높은 사람은 섭취에 주의하십시오. \n– 특이체질, 알레르기 체질인 경우 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오."
5,①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n\n①유해산소로부터 세포를 보호하는데 필요\n\n①혈중 중성지질 개선·혈행 개선·건조한 눈을 개선하여 눈 건강에 도움을 줄 수 있음\n\n노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줌,"특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다. \n- 의약품(항응고제, 항혈소판제, 혈압강하제 등) 복용 시 전문가와 상담할 것 \n– 개인에 따라 피부 관련 이상반응이 발생할 수 있음 \n– 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n– 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음"
6,"유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음\n\n①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있습니다. \n-질환이 있거나 의약품 복용 시 전문가와 상담하십시오. \n-알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. \n-어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하십시오. \n-이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. \n-제품 개봉 또는 섭취 시 포장재질에 의해 상처를 입을 수 있으니 주의하십시오. \n–영∙유아,임신∙수유부나 질환보유자 또는 약 복용 중이신 분은 섭취 전에 의사,약사 등 전문가와 상의하십시오."


In [262]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['※'])[['product_function', 'product_caution']] # 구분자로 사용되어서 제거

,product_function,product_caution
0,[홍삼제품]①면역력 증진에 도움을 줄수 있음\n②피로개선에 도움을 줄 수 있음\n③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n④기억력 개선에 도움을 줄 수 있음\n⑤항산화에 도움을 줄 수 있음 \n※ 면역력증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선.항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n- 당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의\n- 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,[홍삼제품]①면역력 증진에 도움을 줄수 있음②피로개선에 도움을 줄수 있음③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄수 있음④기억력 개선에 도움을 줄 수 있음⑤항산화에 도움을 줄 수 있음 \n※ 면역력증진·피로개선·혈소판 응집억제를 통한 혈액흐름·기억력 개선.항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n- 유통기한을 확인하시기 바라며, 권장섭취량 및 섭취방법을 준수하여 주시기 바랍니다.\n- 특정성분에 알레르기 체질이신 분은 섭취 전 제품의 원료성분을 확인 후 섭취하십시오\n- 질병치료나 약물투여 중인 분들은 섭취 전 전문가와 상의 하십시오.\n- 제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하십시오\n- 식물성 성분 등으로 침전물이 생기는 경우가 있으나 제품 품질 및 기능에는 이상이 없습니다."
2,"[비타민D]\n칼슘과 인이 흡수되고 이용되는데 필요\n뼈의 형성과 유지에 필요\n골다공증발생 위험 감소에 도움을 줌\n\n[아연]\n정상적인 면역기능에 필요\n정상적인 세포분열에 필요\n\n[프로폴리스추출물]\n""항산화 ․ 구강에서의 항균작용에 도움을 줄 수 있음\n※구강에서의 항균작용은 구강에 직접 접촉할 수 있는 형태에 한하며, 섭취량을 적용하지 않음 """,[비타민D]\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n[프로폴리스추출물]\n프로폴리스에 알레르기를 나타내는 사람은 섭취에 주의\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,"①항산화·구강에서의 항균작용에 도움을 줄 수 있음\n\n※ 구강에서의 항균작용은 구강의 직접 접촉할 수 있는 형태(스프레이 또는 팅크제, 십어먹는 연질캡슐)에 한하며, 섭취량을 적용하지 않음",프로폴리스에 알레르기를 나타내는 사람은 섭취에 주의
4,[가르시니아캄보지아 추출물]탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,"※ 어린이, 임산부 및 수유부는 섭취를 피할 것\n※ 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n※ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,[가르시니아캄보지아 추출물]탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,"※ 어린이, 임산부 및 수유부는 섭취를 피할 것.\n※ 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것.\n※ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것."
6,(국문) 면역 기능에 도움을 줄 수 있음\n(영문) May help immune function,"(1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(2) 특정질환(알레르기 체질, 단장증후군, 유당불내증 등)이 있는 분은 섭취에 주의할 것\n(3) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의할 것\n(4) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(5) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n※ 유산균의 정성적 품질관리를 위해 원 균주(전장유전체 정보 등)를 주기적으로 관리하여야 하며, 식약처 요청 시 관련자료를 제출하여야 함"
7,"[엠에스엠(MSM, 디메틸설폰) ① 관절 및 연골건강에 도움을 줄 수 있음\n[NAG(엔에이지, N-Acetylglucosamine) ①관절 및 연골 건강에 도움을 줄 수 있음\n[칼슘] ①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌","※ 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의(게 또는 새우를 원재료로 사용한 경우에 한함)\n※ 제품의 섭취 시 섭취량 및 섭취방법을 확인하고 섭취하여 주십시오.\n※ 알레르기 등 특이체질이신 경우 성분을 확인 후 섭취하여 주십시오.\n※ 원료성분에 의한 침전물이 생기는 경우가 있으나 제품의 품질이나 기능에는 이상이 없으니 잘 흔들어 드십시오.\n※ 파우치 개봉시 내용물이 튈 수있으므로 주의를 기울여 개봉하십시오.\n※ 날카로운 파우치 모서리에 섭취시나 주머니 보관시에 피부가 베이지 않도록 주의하여 주십시오.\n※ 포장이 변형, 손상되었거나, 내용물이 변질 되었을 경우 섭취하지 마십시오."
8,① 관절 및 연골건강에 도움을 줄 수 있음,"※ 제품의 섭취 시 섭취량 및 섭취방법을 확인하고 섭취하여 주십시오.\n※ 알레르기 등 특이체질이신 경우 성분을 확인 후 섭취하여 주십시오.\n※ 원료성분에 의한 침전물이 생기는 경우가 있으나 제품의 품질이나 기능에는 이상이 없으니 잘 흔들어 드십시오.\n※ 파우치 개봉시 내용물이 튈 수있으므로 주의를 기울여 개봉하십시오.\n※ 날카로운 파우치 모서리에 섭취시나 주머니 보관시에 피부가 베이지 않도록 주의하여 주십시오.\n※ 포장이 변형, 손상되었거나, 내용물이 변질 되었을 경우 섭취하지 마십시오."
9,① 관절 및 연골건강에 도움을 줄 수 있음,"※ 개봉 또는 섭취 시 절단면에 의해 상처를 입을 수 있으니 주의하십시오.\n※ 포장이 변형, 팽창, 손상되었거나, 내용물이 변질 되었을 경우 섭취하지 마십시오. \n※ 특이체질, 알레르기체질의 경우 성분을 확인하신 후 섭취하십시오.\n※ 원료성분에 의한 침전물이 생기는 경우가 있으나 제품의 품질 및 기능에는 이상이 없으니 잘 흔들어 드십시오.\n※ 파우치 개봉시 내용물이 튈 수 있으니 주의하여 천천히 개봉하십시오\n※ 파우치 모서리가 날카로우니 섭취하시거나 가지고 다니실 때 피부가 베이지 않도록 주의하십시오\n※ 파우치 포장 그대로 전자레인지에 가열하지 마십시오.\n※ 개봉 후에는 바로 드시기 바랍니다."


In [263]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['₂'])[['product_function', 'product_caution']] # '2'로 수정

product_caution 컬럼값에 "₂" 문자열이 없습니다.


,product_function,product_caution
0,"[키토산] 혈중 콜레스테롤 개선·체지방 감소에 도움을 줄 수 있음\n[비타민B₂] 체내 에너지 생성에 필요\n[판토텐산] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n③ 게 또는 새우에 알레르기가 있는 사람은 섭취에 주의\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [264]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['™'])[['product_function', 'product_caution']] # 제외

,product_function,product_caution
0,포도블루베리추출혼합분말(Memophenol™)(제2019-30호) : 기억력 개선에 도움을 줄 수 있음,"1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 영∙유아, 어린이, 임산부 및 수유부는 섭취에 주의\n3) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,"[연어이리추출물(DOT™연어핵산)(기능성원료인정제2019-5호)] (국문) 관절 건강에 도움을 줄 수 있음, (영문) May help to maintain healthy joint\n[망간] 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요\n[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n[비타민B2] 체내 에너지 생성에 필요","특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[연어이리추출물(DOT™연어핵산)(기능성원료인정제2019-5호)]\n영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n질환이 있거나 의약품(면역억제제 등) 복용 시 전문가와 상담할 것\n이상사례(알러지, 설사, 소화불량 등) 발생 시 섭취를 중단하고 전문가와 상담할 것\n[비타민D] 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
2,포도블루베리추출혼합분말(Memophenol™)(기능성원료인정 제2019-30호) : 기억력 개선에 도움을 줄 수 있음\n- 포스파티딜세린 : 노화로 인해 저하된 인지력 개선·자외선에 의한 피부손상으로부터 피부건강 유지·피부보습에 도움을 줄 수 있음,"특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n- 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 임산부와 수유부는 섭취에 주의\n- 과잉섭취 시 위장장애나 불면증을 유발할 수 있음"
3,포도블루베리추출혼합분말(Memophenol™)(제2019-30호): 기억력 개선에 도움을 줄 수 있음,"1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 영•유아, 어린이, 임산부 및 수유부는 섭취에 주의\n3) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
4,"포도블루베리추출혼합분말(Memophenol™)(제2019-30호) : 기억력 개선에 도움을 줄 수 있음\n- 비타민A : ①어두운 곳에서 시각 적응을 위해 필요 ②피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n- 비타민B1 : ①탄수화물과 에너지 대사에 필요\n- 비타민B2 : ①체내 에너지 생성에 필요\n- 비타민B6 : ①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 비타민C : ① 결합조직 형성과 기능유지에 필요 ② 철의 흡수에 필요 ③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n- 비타민E : ① 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n- 나이아신 : ①체내 에너지 생성에 필요\n- 판토텐산 : ①지방,탄수화물,단백질 대사와 에너지 생성에 필요\n- 엽산 : ①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 철 : ①체내 산소운반과 혈액생성에 필요 ②에너지 생성에 필요","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 유통기한이 경과된 제품은 섭취하지 마십시오.\n3) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n4) 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n5) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n6) 특히 6세 이하는 과량섭취하지 않도록 주의"
5,포도블루베리추출혼합분말(Memophenol™)(기능성원료인정 제2019-30호) : 기억력 개선에 도움을 줄 수 있음,"1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n3) 영∙유아, 어린이, 임산부 및 수유부는 섭취에 주의\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
6,포도블루베리추출혼합분말(Memophenol™)(기능성원료인정 제2019-30호) : 기억력 개선에 도움을 줄 수 있음\n- 테아닌 : 스트레스로 인한 긴장완화에 도움을 줄 수 있음\n- 비타민 B1 : 탄수화물과 에너지 대사에 필요\n- 비타민 B2 : 체내 에너지 생성에 필요\n- 비타민 B6 : (가) 단백질 및 아미노산 이용에 필요\n(나) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 유통기한이 경과된 제품은 섭취하지 마십시오.\n- 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n- 특정질환(알레르기 체질 증)이 있는 분은 섭취에 주의\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의\n- 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의"


In [265]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['∙'])[['product_function', 'product_caution']] # '영∙유아' 예외사항을 제외하고 구분자이기 대문에 '\n'으로 대체

,product_function,product_caution
0,[프로바이오틱스] \n유산균 증식 및 유해균 억제 ∙ 배변활동 원활 ∙ 장 건강에 도움을 줄 수 있음,"[섭취 시 주의사항]\n(가) 섭취량 및 섭취방법을 확인하시고 섭취하여 주시길 바랍니다.\n(나) 질환이 있거나 의약품 복용 중이신 분은 전문의와 반드시 상의 후 섭취하여 주시길 바랍니다.\n(다) 알레르기 체질의 경우, 개인에 따라 과민반응을 나타낼수 있으므로 원료 확인 후 섭취하여 주시길 바랍니다.\n(라) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하여 주시길 바랍니다.\n(마) 이상사례 발생 시 섭취를 중단하시고, 전문의와 상담하시길 바랍니다.\n(바) 제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시길 바랍니다.\n(사) 유통기한이 경과된 제품은 섭취하지 않도록 주의 바랍니다."
1,[홍삼] 면역력 증진∙피로개선∙혈소판 응집억제를 통한 혈액흐름∙기억력 개선∙항산화에 도움을 줄 수 있음\n[회화나무열매추출물] 갱년기 여성의 건강에 도움을 줄 수 있음 \n[대두이소플라본] 뼈 건강에 도움을 줄 수 있음\n[비타민C] 결합조직 형성과 기능유지∙철의 흡수∙유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능∙정상적인 세포분열에 필요\n[비타민A] 어두운 곳에서 시각 적응∙피부와 점막을 형성하고 기능을 유지∙상피세포의 성장과 발달에 필요\n[비타민B1] 탄수화물과 에너지 대사에 필요\n[비타민B2] 체내 에너지 생성에 필요\n[비타민B6] 단백질 및 아미노산 이용∙혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민D] 칼슘과 인이 흡수되고 이용∙뼈의 형성과 유지에 필요∙골다공증발생 위험 감소에 도움을 줌,"1) 섭취량 및 섭취방법을 확인 후 섭취하십시오.\n2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있으니 주의하십시오.\n3) 영유아, 어린이, 임산부와 수유부는 섭취에 주의하시기 바랍니다.\n4) 의약품(당노체료제, 혈액항응고제) 복용 시 섭취에 주의하시기 바랍니다.\n5) 에스트로겐호르몬에 민감한 사람, 대두에 알레르기를 나타내는 사람은 섭취에 주의하시기 바랍니다.\n6) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다.\n7) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다."
2,1) [프로바이오틱스] 유산균 증식 및 유해균 억제 · 배변활동 원활 ∙ 장 건강에 도움을 줄 수 있음\n2) [가르시니아캄보지아 추출물] 탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n3) [비타민C] 결합조직 형성과 기능유지에 필요 · 철의 흡수에 필요 · 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"(가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(다) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(라) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(마) 간·신장·심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n(바) 남성의 경우 과다섭취를 피하시기 바랍니다.\n(사) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,최종제품 : 면역력 증진 ∙ 피로개선 ∙ 혈소판 응집억제를 통한 혈액흐름 ∙ 기억력 개선 ∙ 항산화에 도움을 줄 수 있음\n원료성 : 면역력 증진 ∙ 피로개선 ∙ 혈소판 응집억제를 통한 혈액흐름 ∙ 기억력 개선 ∙ 항산화 ∙ 갱년기 여성의 건강에 도움을 줄 수 있음,당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의\n알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
4,면역력 증진 ∙ 피로회복 ∙ 혈소판 응집억제를 통한 혈액 흐름에 도움. 기억력 개선에 도움. 항산화에 도움.,"의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의"
...,...,...
131,①유산균 증식 및 유해균 억제에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 수 있음,"① 임신, 수유부나 의약품을 섭취하시는 분은 섭취 전 전문가와 상담하여 주십시오.\n② 섭취전 유통기한을 확인한 후 섭취하십시오.\n③ 특이체질, 알레르기 체질이신 분은 성분을 확인한 후 섭취하시기 바랍니다.\n④ 이상증상이 생길 경우 섭취를 중단하시고, 전문의와 상담하시거나 소비자 상담실로 문의하시길바랍니다.\n⑤ 본 제품은 분말타입이므로 영∙유아는 부모님의 지도하에 섭취하시길 바랍니다.\n⑥ 섭취량 및 섭취방법을 확인한 후 섭취하십시오.\n⑦ 제품 개봉 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의하십시오."
132,"루바브뿌리추출물 : 갱년기 여성 건강에 도움을 줄 수 있음 \n- 테아닌 : 스트레스로 인한 긴장완화에 도움을 줄 수 있음\n- 비타민 B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n- 비타민 B12 : 정상적인 엽산 대사에 필요\n- 비타민 D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n- 비타민 K : 정상적인 혈액응고에 필요, 뼈의 구성에 필요\n- 마그네슘 : 에너지 이용에 필요, 신경과 근육 기능 유지에 필요","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 영∙유아, 어린이, 임산부, 수유부 및 수술전후 환자는 섭취에 주의\n3) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의\n4) 메스꺼움 등 소화계통의 불편함과 설사를 유발할 수 있음\n5) 알레르기 체질이거나 특정질환이 있는 사람은 섭취 전 전문가와 상담할 것\n6) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n7) 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의\n8) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n9) 항응고제 등 복용 시 전문가와 상담할 것"
133,[마리골드꽃추출물] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음.\n[헤마토코쿠스추출물] 눈의 피로도 개선에 도움을 줄 수 있음.\n[비타민C] 결합조직 형성과 기능유지에 필요. 철의 흡수에 필요. 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n[비타민E] 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n[아연] 정상적인 면역기능•세포분열에 필요.,"과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\nβ-카로틴의 흡수를 저해할 수 있음\n\n과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n영∙유아, 어린이, 임산부 및 수유부는 섭취하지 마십시오.\n흡연자는 섭취 시 전문가와 상담하십시오.\n이상하례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
134,"[프로바이오틱스] 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음\n[Lactobacillus sakei Probio65]면역과민반응에 의한 피부상태 개선에 도움을 줄 수 있음 (생리활성기능 2등급)\n[자일로올리고당 분말]장내 유익균 증식, 유해균 억제, 배변활동에 도움을 줄 수 있음(기타기능II)",(가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(다) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(마) 과량으로 섭취 시 설사를 유발할 수도 있습니다.\n(바) 알레르기 체질이신 경우 성분을 확인하신 후 섭취하여 주시기 바랍니다. ∙\n(사) 제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다. ∙ \n(아) 개봉 후에는 가급적 빨리 드시기 바랍니다.


In [266]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['∼'])[['product_function', 'product_caution']] # 제외

product_function 컬럼값에 "∼" 문자열이 없습니다.


,product_function,product_caution
0,①혈중 콜레스테롤 개선에 도움을 줄 수 있음\n①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"게 또는 새우에 알레르기가 있는 사람은 섭취에 주의\n특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 바랍니다.\n소비기한을 반드시 확인하시고, 섭취량 및 섭취방법을 준수하시기 바랍니다. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있으니 주의하십시오.\n개인의 상황에 따라 섭취량 및 섭취방법에 관한 영양상담이 필요할 수 있으므로, 섭취 전 영양사와 상담하시기 바랍니다. \n섭취기간에는 2∼3L의 충분한 물의 섭취를 권장합니다. \n과량 섭취 시 복부 팽만감을 느낄 수 있으니 주의해야 합니다. \n장기간 섭취할 경우 지용성 비타민(A,D,E,K)의 부족을 초래할 우려가 있으므로 주의하십시오. \n비타민과 미네랄를 흡수하는데 이상이 있는 분은 섭취를 금합니다. \n식품을 소화하는데 이상이 있는 분은 섭취를 금합니다. \n임산부와 수유기 여성은 섭취를 피하시는 것이 좋습니다. \n이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다."


In [267]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['⋅'])[['product_function', 'product_caution']] # 예외사항 처리후 '\n'으로 대체

,product_function,product_caution
0,면역력 증진 ⋅ 피로개선⋅ 혈소판 응집억제를 통한 혈액흐름 ⋅ 기억력개선 ⋅ 항산화에 도움을 줄 수 있음,"1)당뇨치료제, 혈액항응고제 복용시 섭취에 주의 하십시오.\n2)알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.\n3)영아, 유아, 임신부, 수유부나 질환보유자 또는 약물 복용 중이신 분은 섭취 전에 의사, 약사 등\n 전문가와 상의하십시오.\n4)섭취량 및 섭취방법을 준수하시고 기타사항은 소비자상담실로 문의하십시오.\n5)제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하십시오.\n6)섭취시 기도에 걸리지 않도록 충분한 물과 함께 섭취하십시오.\n7)이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
1,[칼 슘] ①뼈와치아형성에필요 ②신경과근육기능유지에필요 ③정상적인혈액응고에필요 ④골다공증발생위험감소에도움을줌\n\n[비타민 D]①칼슘과 인이 흡수되고 이용되는데 필요 ②뼈의 형성과 유지에 필요 ③골다공증발생위험감소에 도움을 줌\n\n[마그네슘] ①에너지 이용에 필요 ②신경과 근육기능 유지에 필요\n\n[뮤코다당⋅단백] ①관절 및 연골건강에 도움을 줄 수 있음,"1)고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하여 주십시오.\n2)이상사례 발생 시 섭취를 중단하고 전문가와 상담하여 주십시오.\n3)동봉된 방습제를 섭취하지 마십시오.\n4)특이체질 또는 알레르기 체질인 경우에는 원료의 성분을 확인하고 섭취하십시오.\n5)영아, 유아, 임신부, 수유부나 질환보유자 또는 약물 복용 중이신 분은 섭취 전에 의사, 약사 등\n 전문가와 상의하십시오.\n6)섭취량 및 섭취방법을 준수하시고 기타사항은 소비자상담실로 문의하십시오.\n7)제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하십시오.\n8)복용시 기도에 걸리지 않도록 물 또는 기호음료와 함께 섭취하십시오."
2,[프로바이오틱스]①유산균 증식 및 유해균 억제⋅배변활동 원활⋅장 건강에 도움을 줄 수 있음,1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[프로바이오틱스]\n1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,[바나바잎 추출물]식후 혈당상승 억제에 도움을 줄 수 있음. \n[코엔자임Q10]항산화 ⋅높은 혈압 감소에 도움을 줄 수 있음.\n[홍국]혈중 콜레스테롤 개선에 도움을 줄 수 있음. \n[아연]정상적인 면역기능에 필요. 정상적인 세포분열에 필요.\n[셀레늄]유해산소로부터 세포를 보호하는데 필요.,"1) 어린이, 임산부 및 수유부는 섭취를 피할 것\n2) 간 질환이 있거나, 고지혈증 치료제 복용 시 섭취를 피할 것\n3)특이체질 또는 알레르기 체질인 경우에는 원료의 성분을 확인하고 섭취하십시오.\n4)영아, 유아, 임신부, 수유부나 질환보유자 또는 약물 복용 중이신 분은 섭취 전에 의사, 약사 등\n 전문가와 상의하십시오.\n5)섭취량 및 섭취방법을 준수하시고 기타사항은 소비자상담실로 문의하십시오.\n6)제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하십시오.\n7)섭취시 기도에 걸리지 않도록 충분한 물과 함께 섭취하십시오.\n8)이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
4,[프로바이오틱스]①유산균 증식 및 유해균 억제⋅배변활동 원활⋅장 건강에 도움을 줄 수 있음\n[바나바잎추출물]①식후 혈당상승 억제에 도움을 줄 수 있음\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요,1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[프로바이오틱스]\n1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n3) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
5,갱년기 남성의 건강에 도움을 줄 수 있음 (생리활성기능 2등급),"⋅ 영⋅유아, 어린이, 임산부 및 수유부는 섭취에 주의\n⋅ 특정질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n⋅ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
6,[감마리놀렌산 함유 유지]혈중 콜레스테롤 개선·혈행개선·월경전 변화에 의한 불편한 상태 개선·면역과민반응에 의한 피부상태 개선에 도움을 줄 수 있음,1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[감마리놀렌산 함유 유지]\n①영⋅유아⋅어린이는 섭취 전 의사와 상담한 후 섭취할 것(면역과민반응에 의한 피부상태 개선에 도움을 줄 수 있다는 기능성을 표시⋅판매하는 경우에 한함)
7,"[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n[비타민B1]①탄수화물과 에너지 대사에 필요\n[나이아신]①체내 에너지 생성에 필요\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B2]①체내 에너지 생성에 필요\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[셀렌]①유해산소로부터 세포를 보호하는데 필요","1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[가르시니아캄보지아추출물]\n1) 어린이, 임산부 및 수유부는 섭취를 피할 것\n2) 간⋅신장⋅심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n3) 이상사례 발생 시 섭취를 중단하고 전문가와 상의할 것\n[비타민B1, 비타민B2, 판토텐산]\n1) 이상사례 발생 시 섭취를 중단하고 전문가와 상의할 것"
8,"[가르시니아캄보지아 추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n[비타민B1]①탄수화물과 에너지 대사에 필요\n[나이아신]①체내 에너지 생성에 필요\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B2]①체내 에너지 생성에 필요\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요","1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[가르시니아캄보지아 추출물]\n1) 어린이, 임산부 및 수유부는 섭취를 피할 것\n2) 간⋅신장⋅심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n3) 이상사례 발생 시 섭취를 중단하고 전문가와 상의할 것\n[비타민B1, 비타민B2]\n1) 이상사례 발생 시 섭취를 중단하고 전문가와 상의할 것"
9,"[가르시니아캄보지아추출물]①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n[비타민B1]①탄수화물과 에너지 대사에 필요\n[나이아신]①체내 에너지 생성에 필요\n[비타민B2]①체내 에너지 생성에 필요\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[셀렌]①유해산소로부터 세포를 보호하는데 필요","1) 과량섭취 시 부작용이 있을 수 있음.\n2) 특정원료에 알레르기가 있거나 질병치료 약물투여 중인 분은 섭취 전 전문가와 상의하십시오.\n3) 소비기한이 경과된 제품은 섭취하지 마십시오.\n[가

In [268]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['■', '○', '●'])[['product_function', 'product_caution']] # 구분자 사용 - '\n'으로 대체

product_function 컬럼값에 "■" 문자열이 없습니다.
product_function 컬럼값에 "●" 문자열이 없습니다.


,product_function,product_caution
0,[서목태(쥐눈이콩)펩타이드]○ 혈당 조절에 도움을 줄 수 있음 (생리활성기능 2등급)○ 혈압이 높은 사람에게 도움을 줄 수 있음 (생리활성기능 2등급) ○ 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급),"① 임산부, 수유부는 섭취에 주의하십시오.\r\n② 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오\r\n③ 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다."
1,유산균 증식 및 유해균 억제․배변활동 원활·장건강에 도움을 줄 수 있음,■질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n■알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n■어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n■이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n■제품 안의 습기방지제(실리카겔)은 드시지 마십시오.
2,①유산균 증식 및 유해균 억제에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 수 있음,■질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n■알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n■어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것\n■이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n■제품 안의 습기방지제(실리카겔)은 드시지 마십시오.
3,"[콜레우스 포스콜리 추출물(제2012-5호)] 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)\n\n[테아닌] 스트레스로 인한 긴장완화에 도움을 줄 수 있음\n\n[판토텐산] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[비타민B2] 체내 에너지 생성에 필요\n\n[비타민B1] 탄수화물과 에너지 대사에 필요\n\n[비오틴] 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","○ 어린이, 임산부, 수유부는 섭취를 피하십시오.\n○ 항응고제 또는 혈압조절제를 복용하거나 혈압이 낮은 경우 전문가와 상담하십시오.\n○ 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하십시오.\n○ 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오.\n○ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
4,"[콜레우스 포스콜리 추출물(제2012-5호)] 체지방 감소에 도움을 줄 수 있음\n\n[비타민B1] ①탄수화물과 에너지 대사에 필요\n\n[비타민B2] ①체내 에너지 생성에 필요\n\n[비타민B6] ①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n[판토텐산] ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n[셀레늄(또는 셀렌)] ①유해산소로부터 세포를 보호하는데 필요","○ 어린이, 임산부, 수유부는 섭취를 피할 것\n○ 항응고제 또는 혈압조절제를 복용하거나 혈압이 낮은 경우 전문가와 상담할 것\n○ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,"체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)\n\n(1) 탄수화물과 에너지 대사에 필요\n\n(1) 체내 에너지 생성에 필요\n\n①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n(1) 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n①유해산소로부터 세포를 보호하는데 필요","○ 어린이, 임산부, 수유부는 섭취를 피할 것\n○ 항응고제 또는 혈압조절제를 복용하거나 혈압이 낮은 경우 전문가와 상담할 것\n○ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
6,[쏘팔메토 열매 추출물] 전립선 건강의 유지에 도움을 줄 수 있음\n\n[옥타코사놀 함유 유지] 지구력 증진에 도움을 줄 수 있음\n\n[아연] ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n[밀크씨슬(카르두스 마리아누스) 추출물] 간 건강에 도움을 줄 수 있음,"[쏘팔메토 열매 추출물] \n(가) 성인남성만 섭취할 것\n(나) 수술 전후, 출혈성 질환이 있거나 항응고제 등 복용 시 전문가와 상담할 것\n(다) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(라) 메스꺼움 등 소화계통의 불편함과 설사를 유발할 수 있으니 식사 후 섭취할 것\n(마) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n[밀크씨슬(카르두스 마리아누스) 추출물]\n○ 알레르기 반응이 나타나는 경우에는 섭취 중단\n○ 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취에 주의"
7,"비타민C : ①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n철 : ①체내 산소운반과 혈액생성에 필요②에너지 생성에 필요\n\n아연 : ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n나이아신 : ①체내 에너지 생성에 필요\n\n구리 : ①철의 운반과 이용에 필요②유해산소로부터 세포를 보호하는데 필요\n\n비타민D : ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n\n망간 : ①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\n\n셀렌 : ①유해산소로부터 세포를 보호하는데 필요\n\n엽산 : ①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n판토텐산 : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민E : ①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n\n몰리브덴 : ①산화,환원 효소의 활성에 필요\n\n비타민B2 : ①체내 에너지 생성에 필요\n\n요오드 : ①갑상선 호르몬의 합성에 필요②에너지 생성에 필요③신경발달에 필요\n\n비타민K : ①정상적인 혈액응고에 필요②뼈의 구성에 필요\n\n비오틴 : ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민B6 : ①단...","○ 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n○ 이상 사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n○ 특히 6세 이하는 과량 섭취하지 않도록 주의하십시오.\n○ 특이체질, 알레르기 체질의 경우 개인에 따라 과민 반응을 나타낼 수 있으므로 원료 확인 후 섭취하십시오.\n○ 만성질환이 있거나 현재 질병치료 및 약물 복용중이신 분은 전문의와 반드시 상의 후 섭취하십시오. \n○ 섭취방법 및 섭취량을 준수하시기 바랍니다. \n○ 어린이의 경우 섭취 시 목에 걸릴 우려가 있으니 주의하시길 바랍니다.\n○ 유통기한이 경과된 제품은 섭취하지 마시기 바랍니다. \n○ 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것\n○ 항응고제 등 복용 시 전문가와 상담할 것"
8,"비타민B1 : 탄수화물과 에너지 대사에 필요\n\n비타민B2 : 체내 에너지 생성에 필요\n\n비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민B6 : ①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n엽산 : ①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n비타민B12 :①정상적인 엽산 대사에 필요","○ 특이체질, 알레르기 체질의 경우 개인에 따라 과민 반응을 나타낼 수 있으므로 원료 확인 후 섭취하십시오.\n○ 만성질환이 있거나 현재 질병치료 및 약물 복용중이신 분은 전문의와 반드시 상의 후 섭취하십시오. \n○ 이상 사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n○ 섭취방법 및 섭취량을 준수하시기 바랍니다. \n○ 어린이의 경우 섭취 시 목에 걸릴 우려가 있으니 주의하시길 바랍니다.\n○ 소비기한이 경과된 제품은 섭취하지 마시기 바랍니다."
9,①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"○ 특이체질, 알레르기 체질의 경우 개인에 따라 과민 반응을 나타낼 수 있으므로 원료 확인 후 섭취하십시오.\n○ 만성질환이 있거나 현재 질병치료 및 약물 복용중

In [269]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['⦁'])[['product_function', 'product_caution']] # '⦁' -> '⋅'로 대체 후 일괄처리

,product_function,product_caution
0,면역력 증진⦁피로개선⦁혈소판 응집 억제를 통한 혈액흐름⦁기억력 개선⦁항산화⦁갱년기여성의 건강에 도움을 줄 수 있음,"알러지 등 특이체질의 경우 성분을 확인 후 섭취한다.\n어린이의 섭취 시에는 보호자의 관리 하에 섭취하시기 바랍니다.\n의약품(당뇨치료제, 혈액항응고제)복용 시 섭취에 주의하십시오."
1,"[뮤코다당⦁단백] : 관절 및 연골건강에 도움을 줄 수 있음\n[비타민 D] : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n[망간] : 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요","고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n섭취량 및 섭취방법을 확인하시고 기타사항은 고객상담실로 문의하십시오.\n소비기한을 확인 후 섭취하십시오.\n특이체질, 알레르기 체질, 임신⦁수유부 및 질병 치료 중인 분은 원료의 성분을 확인 후 섭취하십시오.\n용기안의 습기방지제는 먹지 마십시오."
2,[뮤코다당⦁단백] : 관절 및 연골건강에 도움을 줄 수 있음,이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n섭취량 및 섭취방법을 확인하시고 기타사항은 고객상담실로 문의하십시오.\n특이체질 또는 알레르기 체질인 경우에는 개인에 따라 과민반응을 나타낼 수 있으므로 원재료를 확인한 후 섭취하십시오.\n용기안의 습기방지제는 먹지 마십시오.
3,"1) 프로바이오틱스 : 유산균 증식 및 유해균 억제⦁배변활동 원활⦁장 건강에 도움을 줄 수 있음.\n2) 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요.","섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.\n어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
4,[인삼제품] 면역력 증진⦁피로개선에 도움을 줄 수 있음,1) 당뇨치료제 및 혈액 항응고제 복용 시 섭취에 주의 \n2) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음 \n3) 이상 사례 발생 시 섭취를 중단하고 전문가와 상담할 것
5,"①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌\n\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n①탄수화물과 에너지 대사에 필요\n\n①체내 에너지 생성에 필요","어린이, 임산부 및 수유부는 섭취를 피할 것\n- 간⦁신장⦁심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
6,①필수지방산의 보충\n\n①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요,"⦁특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼수 있으므로 원료를 확인한 후 섭취하십시오\n 유통기한이 경과된 제품은 섭취하지 마십시오"
7,[가르시니아캄보지아추출물] ①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음\n\n[나이아신] ①체내 에너지 생성에 필요\n\n[비타민A] ①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n\n[비타민B1] ①탄수화물과 에너지 대사에 필요\n\n[비타민B6] ①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"어린이, 임산부 및 수유부는 섭취를 피할 것\n- 간⦁신장⦁심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
8,[프로바이오틱스] 유산균 증식 및 유해균 억제 · 배변활동 원활 · 장건강에 도움을 줄 수 있음 \n[가르시니아캄보지아 추출물] 탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌 \n[차전자피식이섬유] 혈중 콜레스테롤 개선 · 배변활동 원활에 도움을 줌,"1) 반드시 충분한 물과 함께 섭취하여 주시길 바랍니다.\n2) 어린이, 임산부 및 수유부는 섭취를 피해주시길 바랍니다.\n3) 간⦁신장⦁심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문의와 반드시 상의 후 섭취하여 주시길 바랍니다.\n4) 이상사례 발생 시 섭취를 중단하시고, 전문의와 상담하시길 바랍니다.\n5) 섭취량 및 섭취방법을 확인하시고 섭취하여 주시길 바랍니다.\n6) 제품 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시길 바랍니다.\n7) 유통기한이 경과된 제품은 섭취하지 않도록 주의 바랍니다."
9,①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,"어린이, 임산부 및 수유부는 섭취를 피할 것\n- 간⦁신장⦁심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [270]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['〔'])[['product_function', 'product_caution']] # '〔', '〕' 일 경우 '[', ']'으로 대체하여 제거

,product_function,product_caution
0,〔엠에스엠〕① 관절 및 연골건강에 도움을 줄 수 있음\n\n〔아연〕①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n〔망간〕①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\n\n〔비타민 D〕①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,(가) 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
1,"〔프락토올리고당〕장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n\n〔마그네슘〕①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n〔아연〕①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n〔비타민 B1〕①탄수화물과 에너지 대사에 필요\n\n〔비타민 B6〕①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n〔판토텐산〕①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n〔단백질〕①근육, 결합조직 등 신체조직의 구성성분②효소, 호르몬, 항체의 구성에 필요③체내 필수 영양성분이나 활성물질의 운반과 저장에 필요④체액, 산-염기의 균형 유지에 필요⑤에너지, 포도당, 지질의 합성에 필요","(가) 섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있음\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n\n\n\n\n\n\n\n\n\n\n특정 단백질에 알레르기를 나타내는 경우에는 섭취 주의"
2,〔칼슘〕①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌\n\n〔마그네슘〕①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n〔망간〕①뼈 형성에 필요②에너지 이용에 필요③유해산소로부터 세포를 보호하는데 필요\n\n〔비타민 D〕①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
3,〔마리골드꽃추출물〕노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n〔베타카로틴〕①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n〔비타민 D〕①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n〔비타민 E〕①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요 \n〔아연〕①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"(가)과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n(나)고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(다)이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n(라)흡연자는 섭취 시 전문가와 상담할 것\n\n(가) 영‧유아, 어린이, 임산부 및 수유부는 섭취를 피할 것"
4,〔밀크씨슬(카라두스 마리아누스)추출물〕① 간 건강에 도움을 줄 수 있음\n\n〔아연〕①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n\n〔비타민 B1〕(1) 탄수화물과 에너지 대사에 필요\n\n〔비타민 B2〕(1) 체내 에너지 생성에 필요\n\n〔비타민 B6〕①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n\n〔비타민 D〕①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌,"(가) 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 알레르기 반응이 나타나는 경우에는 섭취 중단\n(다) 설사, 위통, 복부팽만 등의 위장관계 장애가 나타나는 경우에는 섭취에 주의\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n\n(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,"〔녹차추출물〕항산화·체지방 감소·혈중 콜레스테롤 개선에 도움을 줄 수 있음\n〔판토텐산〕지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n〔비타민C〕결합조직 형성과 기능유지에 필요, 철의 흡수에 필요,\n 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n〔셀레늄〕유해산소로부터 세포를 보호하는데 필요\n〔비타민E〕항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n〔나이아신〕정상적인 엽산 대사에 필요\n\n〔비타민A〕어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 \n 필요, 상피세포의 성장과 발달에 필요\n〔비타민D〕칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요,\n 골다공증발생 위험 감소에 도움을 줌\n〔비타민B1〕탄수화물과 에너지 대사에 필요\n〔비타민B2〕체내 에너지 생성에 필요\n〔비타민B6〕단백질 및 아미노산 이용에 필요, 혈액의 호모식스테인 수준을 정상으로 유지하는데 \n 필요\n〔비타민B12〕정상적인 엽산 대사에 필요","(가) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인 후 섭취하십시오.\n(나) 용기 안의 실리카겔(방습제)은 절대 드시지 마십시오.\n(다) 유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다.\n(라) 임산부, 수유부, 어린이 및 수술전후 환자는 섭취를 피하시기 바랍니다.\n(마) 카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있으니 주의하시기 바랍니다.\n(바) 특히 6세 이하는 과량섭취하지 않도록 주의하시기 바랍니다.\n(사) 간질환이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다.\n(아) 식사 후 섭취하시기 바랍니다.\n(자) 카페인을 함유한 식품의 섭취에 주의하시기 바랍니다.\n(차) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다.\n(카) 특히 6세 이하는 과량섭취하지 않도록 주의 고칼슘혈증이 있거나 의약품 복용시 전문가와 상의하십시오."
6,①어두운 곳에서 시각 적응을 위해 필요②피부와 점막을 형성하고 기능을 유지하는데 필요③상피세포의 성장과 발달에 필요\n\n노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음,"〔루테인〕\n(가) 영‧유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n(나) 흡연자는 섭취 시 전문가와 상담할 것\n(다) 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
7,[프로폴리스추출물 제품]①항산화에 도움을 줄 수 있음 ②구강에서의 항균작용에 도움을 줄 수 있음\r\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요,"특이체질이거나 알러지 체질의 경우, 개인에 따라 과민반응이 나타날 수 있으므로 원료명을 확인하시고 섭취하십시오.\r\n- 유통기한을 확인하시기 바라며, 임산부, 수유 여성, 어린이는 섭취에 주의하십시오.\r\n- 〔프로폴리스추출물〕 : 프로폴리스에 알레르기를 나타내는 사람은 섭취에 주의"


In [271]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['〮'])[['product_function', 'product_caution']] # 제거

,product_function,product_caution
0,"[비타민C]\n〮 ①결합조직 형성과 기능유지에 필요②철의 흡수에 필요③항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요 \n[나이아신]\n· ①체내 에너지 생성에 필요\n[옥타코사놀]\n· ①지구력 증진에 도움을 줄 수 있음\n[비타민E]\n· 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[판토텐산]\n· ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[아연]\n· ①정상적인 면역기능에 필요②정상적인 세포분열에 필요\n[비타민 B6]\n· ①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민 B2]\n· ①체내 에너지 생성에 필요\n[비타민 B1]\n· ①탄수화물과 에너지 대사에 필요\n[엽산]\n· ①세포와 혈액생성에 필요②태아 신경관의 정상 발달에 필요③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민 B12]\n· ①정상적인 엽산 대사에 필요\n[N-아세틸글루코사민]\n· ①관절 및 연골 건강·피부보습에 도움을 줄 수 있음\n[비타민D]\n· ①칼슘과 인이 흡수되고 이용되는데 필요②뼈의 형성과 유지에 필요③골다공증발생 위험 감소에 도움을 줌\n[포스콜린]\n· 체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)","게 또는 새우에 알레르기가 있는 사람은 섭취에 주의(게 또는 새우를 원재료로 사용한 경우에 한함)\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n임산부, 수유여성, 어린이는 섭취에 주의"
1,①면역력 증진에 도움을 줄 수 있음\n②피로개선에 도움을 줄 수 있음\n③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n④기억력에 개선에 도움을 줄 수 있음\n⑤항산화에 도움을 줄 수 있음,〮 소비기한이 경과된 제품은 섭취하지 마십시오.\n〮 당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의하십시오.\n〮 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있으니 주의 하십시오.\n〮 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
2,(국문) ① 피부 보습에 도움을 줄 수 있음\n ② 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n ③관절 및 연골 건강에 도움을 줄 수 있음\n(영문) ① May help to maintain skin moisturizing\n ② May help to maintain skin health from skin damage by UV radiation\n ③ May help to maintain joint and cartilage health,"1) 영유〮아, 어린이, 임산부, 수유부는 섭취에 주의\n2) 특이체질, 알레르기 체질이신 분은 성분을 확인한 후 섭취\n3) 흡습이 잘 되는 원료가 포함되어 있어 병 내 손 삽입 금지\n4) 제품 내 습기제거제 섭취 금지\n5) 이상사례 발생 시, 섭취를 중단하고 전문가와 상담할 것"


In [272]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['ㅇ'])[['product_function', 'product_caution']] # 예외사항 처리 후 제거

product_function 컬럼값에 "ㅇ" 문자열이 없습니다.


,product_function,product_caution
0,"[비타민B1] \n①탄수화물과 에너지 대사에 필요\n[비타민B2]\n①체내 에너지 생성에 필요\n[비타민B6]\n①단백질 및 아미노산 이용에 필요 ②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민C]\n①결합조직 형성과 기능유지에 필요 ②철의 흡수에 필요 ③유해산소로부터 세포를 보호하는데 필요\n[나이아신]\n①체내 에너지 생성에 필요\n[판토텐산]\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[엽산]\n①세포와 혈액생성에 필요 ②태아 신경관의 정상 발달에 필요 ③혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴]\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민D]\n①칼슘과 인이 흡수되고 이용되는데 필요 ②뼈의 형성과 유지에 필요 ③골다공증발생 위험 감소에 도움을 줌\n[아연]\n①정상적인 면역기능에 필요 ②정상적인 세포분열에 필요\n[셀레늄(또는 셀렌)]\n①유해산소로부터 세포를 보호하는데 필요\n[망간]\n①뼈 형성에 필요 ②에너지 이용에 필요 ③유해산소로부터 세포를 보호하는데 필요\n[몰리브덴]\n①산화, 환원 효소의 활성에 필요","1) 섭취 전 제품에 이상이 있을 경우 섭취를 금하십시오\n2) 섭취량 및 섭취방법을 준수하시고 기타사항은 고객상담실로 문의하십시오\n3) 특이 체질 및 알레르기 체질이신 분은 성분ㅇ르 확인 후 섭취하시기 바랍니다.\n4) 영·유아, 임신·수유부나 질환보유자 또는 약물 복용중이신 분은 섭취 전에 의사, 약사 등 전문가와 상의 후 섭취하십시오\n5)식품의 품질관리를 위해 별도 포장하여 넣은 습기방지제(방습제)는 먹어서는 안되므로 주의하시기 바랍니다"
1,근력 개선에 도움을 줄 수 있음,"ㅇ영・유아, 어린이, 임산부 및 수유부는 섭취에 주의 \nㅇ수술 전・후 섭취에 주의\nㅇ식사조절, 운동을 병행하는 것이 바람직함"


In [273]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['ㅍ'])[['product_function', 'product_caution']] # '\n'으로 대체

product_caution 컬럼값에 "ㅍ" 문자열이 없습니다.


,product_function,product_caution
0,[홍삼제품]ㅍ①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선⑤항산화에 도움을 줄 수 있음\r\n[옥타코사놀함유유지 제품]①지구력 증진,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n알레르기 등 특이체질의 경우에는 성분을 확인하신 후 섭취하십시오.\r\n제품의 개봉 또는 섭취 시 포장재에 의해 상처를 입을 수 있으니 주의하시기 바랍니다."


In [274]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['ㅡ'])[['product_function', 'product_caution']] # 제거

product_function 컬럼값에 "ㅡ" 문자열이 없습니다.


,product_function,product_caution
0,[영지버섯자실체추출물 제품]①혈행개선 혈행개선혈행개선,ㅡ 알레르기체질 등 특이체질의 경우에는 성분을 확인 하신 후 섭취 하십시오. \r\nㅡ 내용물의 손상이나 변형 또는 팽창 시에는 섭취하지 마십시오.\r\nㅡ 캅셀이 목에 걸리지 않도록 충분한 양의 물과 함께 섭취 하십시오.\r\nㅡ 제품 개봉 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의 바랍니다.
1,[홍삼제품]①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름④기억력 개선⑤항산화에 도움을 줄 수 있음,ㅡ 알레르기체질 등 특이체질의 경우에는 성분을 확인 하신 후 섭취 하십시오. \r\n ㅡ 내용물의 손상이나 변형 또는 팽창 시에는 섭취하지 마십시오.\r\n ㅡ 캅셀이 목에 걸리지 않도록 충분한 양의 물과 함께 섭취 하십시오.\r\n ㅡ 제품 개봉 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의 바랍니다.
2,[홍삼제품]①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름④기억력 개선⑤항산화에 도움을 줄 수 있음,"의약품(당뇨치료제, 혈액항응고제)복용시 섭취에 주의 ㅡ 알레르기체질 등 특이체질의 경우에는 성분을 확인 하신 후 섭취 하십시오. \r\n ㅡ 내용물의 손상이나 변형 또는 팽창 시에는 섭취하지 마십시오.\r\n ㅡ 캅셀이 목에 걸리지 않도록 충분한 양의 물과 함께 섭취 하십시오.\r\n ㅡ 제품 개봉 또는 섭취시 포장재에 의해 상처를 입을 수 있으니 주의 바랍니다."
3,"①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④청년기 이전에 적절한 운동과 건강한 식습관을 유지하면서 충분한 칼슘을 섭취하면 향후 골다공증발생의 위험을 감소시킬 수 있음\r\n①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④청년기 이전에 적절한 운동과 건강한 식습관을 유지하면서 충분한 칼슘을 섭취하면 향후 골다공증발생의 위험을 감소시킬 수 있음\r\n①에너지 이용에 필요②신경과 근육 기능 유지에 필요\r\n①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\r\n①정상적인 면역기능에 필요② 정상적인 세포분열에 필요",ㅡ알레르기 체질이신 경우 성분을 확인한 후 섭취하시기 바랍니다.\nㅡ제품을 개봉할 때 혹 포장재에 의해 상처를 입을 수 있으니 주의 하십시오.


In [275]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['ㆍ'])[['product_function', 'product_caution']] # 'ㆍ' -> '⋅'로 대체 후 일괄처리

,product_function,product_caution
0,* 차전자피식이섬유\n① 혈중 콜레스테롤 개선ㆍ배변활동 원활에 도움을 줄 수 있음,* 차전자피식이섬유\n① 반드시 충분한 물과 함께 섭취할 것 (액상제외)
1,"감마리놀렌산 함유 유지 : 혈중 콜레스테롤 개선ㆍ혈행개선에 도움을 줄 수 있음, 월경전 변화에 의한 불편한 상태 개선에 도움을 줄 수 있음, 면역 과민반응에 의한 피부상태 개선에 도움을 줄 수 있음\n- 회화나무열매추출물 : 갱년기 여성의 건강에 도움을 줄 수 있음\n- 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌","1) 특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n2) 영ㆍ유아ㆍ어린이는 섭취 전 전문가와 상담한 후 섭취할 것\n(면역과민반응에 의한 피부상태 개선에 도움을 줄 수 있다는 기능성을 표시ㆍ판매하는 경우에 한함)\n3) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n4) 어린이, 임산부, 수유부는 섭취에 주의\n5) 에스트로겐 호르몬에 민감한 사람은 섭취에 주의 \n6) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것"
2,면역력 증진ㆍ피로회복ㆍ혈소판 응집 억제를 통한 혈액흐름ㆍ기억력 개선ㆍ항산화에 도움을 줄 수 있음,"① 가루상태로 직접 섭취시 기도 장애가 발생 할 수 있으므로 물에 타서 섭취한다.\r\n② 위장장애가 없는 경우 공복에 섭취함이 좋음\r\n③ 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의"
3,면역력 증진ㆍ피로회복ㆍ혈소판 응집 억제를 통한 혈액흐름ㆍ기억력 개선ㆍ항산화에 도움을 줄 수 있음,"① 섭취시 목에 걸리는 불편이 생길 수 있으므로 물과 같이 섭취한다.\r\n② 위장장애가 없는 경우 공복에 홍삼차와 같이 섭취하면 좋음\r\n③ 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의"
4,[홍삼제품]면역력 증진ㆍ피로회복ㆍ혈소판 응집 억제를 통한 혈액흐름ㆍ기억력 개선ㆍ항산화에 도움을 줄 수 있음,"씹어서 섭취하면 안되며 따뜻한 물과 함께 섭취한다.\r\n의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의"
...,...,...
179,"①근육, 결합조직 등 신체조직의 구성성분②효소, 호르몬, 항체의 구성에 필요③체내 필수 영양성분이나 활성물질의 운반과 저장에 필요④체액, 산-염기의 균형 유지에 필요⑤에너지, 포도당, 지질의 합성에 필요","특정 단백질에 알레르기를 나타내는 경우에는 섭취에 주의하십시오.\n영ㆍ유아, 임신ㆍ수유부나 질환보유자 또는 약물 복용중이신 분은 섭취전 의사, 약사등 전문가와\n상의하십시오.섭취량 및 섭취방법을 준수하시고 기타 사항은 고객상담실로 문의하십시오."
180,"[단백질]①근육, 결합조직 등 신체조직의 구성성분②효소, 호르몬, 항체의 구성에 필요③체내 필수 영양성분이나 활성물질의 운반과 저장에 필요④체액, 산-염기의 균형 유지에 필요⑤에너지, 포도당, 지질의 합성에 필요","영ㆍ유아, 임신ㆍ수유부나 질환보유자 또는 약물 복용중이신 분은 섭취전 의사, 약사등 전문가와 상의하십시오.\r\n섭취량 및 섭취방법을 준수하시고 기타 사항은 고객상담실로 문의하십시오."
181,"[단백질]①근육, 결합조직 등 신체조직의 구성성분②효소, 호르몬, 항체의 구성에 필요③체내 필수 영양성분이나 활성물질의 운반과 저장에 필요④체액, 산-염기의 균형 유지에 필요⑤에너지, 포도당, 지질의 합성에 필요","[단백질]특정 단백질에 알레르기를 나타내는 경우에는 섭취 주의\r\n영ㆍ유아, 임신ㆍ수유부나 질환보유자 또는 약물 복용중이신 분은 섭취전 의사, 약사등 전문가와 상의하십시오.\r\n섭취량 및 섭취방법을 준수하시고 기타 사항은 고객상담실로 문의하십시오."
182,①탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줌,"① 섭취량 및 섭취방법을 준수하십시오.\n② 어린이, 임산부 및 수유부는 섭취를 피하시기 바랍니다.\n③ 간ㆍ신장ㆍ심장질환, 알레르기 및 천식이 있거나 의약품 복용 시 전문가와 상담하십시오.\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."


In [276]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['㈜'])[['product_function', 'product_caution']] # 제외

,product_function,product_caution
0,"[은행잎 추출물]\n기억력 개선·혈행 개선에 도움을 줄 수 있음\n\n[나토균배양분말(지에프퍼멘텍㈜, 제2013-6호)]\n혈행 개선에 도움을 줄 수 있음","[은행잎 추출물]\n임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의\n의약품(항응고제) 복용 시 섭취에 주의\n\n[나토균배양분말(지에프퍼멘텍㈜, 제2013-6호)]\n-대두에 알레르기를 나타내는 사람은 섭취에 주의\n-임산부․수유부 섭취에 주의\n-의약품(혈액항응고제) 복용 시 섭취에 주의\n-수술 전․후 섭취에 주의"
1,"[레몬 밤 추출물 혼합분말(㈜안지오랩, 제2010-24호)] 체지방 감소에 도움을 줄 수 있음 (기타기능II)",알레르기 반응이 나타날 경우에는 섭취를 중단하시기 바랍니다\n-섭취 후 진정작용이 나타날 수 있습니다\n-어린이 및 임산부와 수유부 여성은 섭취에 주의하시기 바랍니다
2,"[스페인감초추출물(개별인정형 ㈜뉴트리, 제2019-7호)]\n(국문) 위 점막 내 헬리코박터균(Helicobacter pylori) 증식을 억제하고 위 점막을 보호하여 위 건강에 도움을 줄 수 있음\n(영문) May help to stomach health by protecting gastric mucosa and inhibiting proliferation of Helicobacter pylori in gastric mucosa.","[스페인감초추출물(개별인정형 ㈜뉴트리, 제2019-7호)]\n① 영 · 유아, 어린이, 임산부 및 수유부는 섭취에 주의\n② 특정 질환(알레르기 체질 등)이 있는 분은 섭취에 주의\n③ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
3,[루테인지아잔틴] 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n[비타민C] ① 결합조직 형성과 기능유지에 필요 ② 철의 흡수에 필요 ③ 유해산소로부터 세포를 보호하는데 필요\n[아연] ① 정상적인 면역기능에 필요 ② 정상적인 세포분열에 필요\n[비타민E] ① 유해산소로부터 세포를 보호하는데 필요\n[비타민B12] ① 정상적인 엽산 대사에 필요\n[비타민A] ① 어두운 곳에서 시각 적응을 위해 필요 ② 피부와 점막을 형성하고 기능을 유지하는데 필요 ③ 상피세포의 성장과 발달에 필요\n[비타민B6] ① 단백질 및 아미노산 이용에 필요 ② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B2] ① 체내 에너지 생성에 필요,"[루테인지아잔틴복합추출물(대한켐텍㈜, 제2018-4호)]\n- 영·유아, 어린이, 임산부 및 수유부는 섭취를 피하십시오.\n- 과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n- 흡연자는 섭취 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"


In [277]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['\ufeff'])[['product_function', 'product_caution']] # pass

,product_function,product_caution
0,﻿[핑거루트추출분말(판두라틴)(기능성원료 인정 제2013-5호)]\n① 자외선에 의한 피부손상으로부터 피부 건강을 유지하는데 도움을 줄 수 있음\n② 체지방감소에 도움을 줄 수 있음\n③ 피부보습에 도움을 줄 수 있음,NaN
1,﻿[철]\n① 체내 산소운반과 혈액생성에 필요\n② 에너지 생성에 필요,﻿[철]\n① 특히 6세 이하는 과량섭취하지 않도록 주의하십시오.
2,﻿[피크노제놀-프랑스해안송껍질추출물(제2006-1호)]\n① 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n② 갱년기 여성의 건강에 도움을 줄 수 있음\n[테아닌]\n스트레스로 인한 긴장완화에 도움을 줄 수 있음,"﻿[피크노제놀-프랑스해안송껍질추출물(제2006-1호)]\n① 임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것\n② 수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n③ 식사 후 또는 식사와 함께 할 것\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것 \n[테아닌]\n① 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하십시오.\n② 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오."
3,"[은행잎 추출물]\n① 기억력 개선･혈행 개선에 도움을 줄 수 있음\n[HK나토배양물]\n① 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하여, 혈액 흐름을 원활히 하는데 도움을 줄 수 있음\n[아연]\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n[비타민 D]\n① 칼슘과 인이 흡수되고 이용되는데 필요\n② 뼈의 형성과 유지에 필요\n③ 골다공증발생 위험 감소에 도움을 줌\n[철]\n① 체내 산소운반과 혈액생성에 필요\n② 에너지 생성에 필요","﻿[은행잎 추출물]\n① 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하십시오.\n② 의약품(항응고제) 복용 시 섭취에 주의하십시오.\n[HK나토배양물]\n① 알레르기 체질이신 분은 원재료를 확인하신 후 섭취하십시오.\n② 임산부와 수유부 여성은 섭취를 피하는 것이 좋습니다.\n③ 과다 섭취 시 혈액응고가 저해되어 출혈 위험이 증가될 수 있습니다.\n④ 항혈전 관련 약품(항응고제, 항혈소판제제)을 섭취하는 사람은 의사와 상담 후 섭취하여야 합니다.\n⑤ 수술 전과 후에는 섭취를 피해야 합니다.\n[비타민 D]\n① 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n② 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[철]\n① 특히 6세 이하는 과량섭취하지 않도록 주의하십시오."
4,"[코엔자임Q10]\n① 항산화•높은 혈압 감소에 도움을 줄 수 있음\n[비타민C]\n① 결합조직 형성과 기능유지에 필요 \n② 철의 흡수에 필요\n③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[나이아신]\n① 체내 에너지 생성에 필요\n[아연]\n① 정상적인 면역기능에 필요 ② 정상적인 세포분열에 필요\n[판토텐산]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B1]\n① 탄수화물과 에너지 대사에 필요\n[비타민B2]\n① 체내 에너지 생성에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[엽산]\n① 세포와 혈액생성에 필요\n② 태아 신경관의 정상 발달에 필요\n③ 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","﻿[비타민 B1, 비타민 B2, 판토텐산, 비오틴]\n이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,[홍삼]\n① 면역력 증진･피로개선･혈소판 응집억제를 통한 혈액흐름･기억력 개선･항산화에 도움을 줄 수 있음\n[아연]\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n[비타민B2]\n① 체내 에너지 생성에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[나이아신]\n① 체내 에너지 생성에 필요,﻿[홍삼]\n① 당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의 하십시오. \n② 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. \n③ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n[비타민B2]\n① 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것


In [278]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['，'])[['product_function', 'product_caution']] # ','으로 대체

product_function 컬럼값에 "，" 문자열이 없습니다.


,product_function,product_caution
0,[마그네슘]①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며，섭취량 및 섭취방법을 준수하시기 바랍니다．"
1,"[키토산]체지방 감소에 도움을 줄 수 있음 (생리활성기능 2등급)\r\n[판토텐산]①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","[키토산]게 또는 새우에 알레르기가 있는 사람은 섭취에 주의(게 또는 새우를 원재료로 사용한 경우에 한함)\r\n① 알레르기 반응이 나타나는 경우에는 섭취 중단하십시오.\r\n② 유통기한을 확인하시기 바라며, 섭취량 및 섭취방법을 준수하시기 바랍니다. \r\n③ 약물 복용 중이신 분은 섭취 전에 의사，약사 등 전문가와 상의하십시오."
2,[칼슘]①뼈와 치아 형성에 필요②신경과 근육 기능 유지에 필요③정상적인 혈액응고에 필요④골다공증발생 위험 감소에 도움을 줌,① 특정식품에 알레르기 증상이 계신 분은 성분을 확인 후 섭취\r\n② 약물 복용중이신 분은 섭취전에 의사，약사 등 전문가와 상의하십시오
3,[셀레늄(또는 셀렌)]①유해산소로부터 세포를 보호하는데 필요,① 특정식품에 알레르기 증상이 계신 분은 성분을 확인 후 섭취하십시오.\r\n② 약물 복용 중이신 분은 섭취 전에 의사，약사 등 전문가와 상의하십시오.
4,[곤약감자추출물] 피부 보습에 도움을 줄 수 있음\n[비타민B2] 체내 에너지 생성에 필요\n[비타민C] 결합조직 형성과 기능유지에 필요 / 철의 흡수에 필요 / 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요 ／ 정상적인 세포분열에 필요,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며，섭취량 및 섭취방법을 준수하시기 바랍니다．\n③ 임산부와 수유기 여성은 섭취를 피하십시오.\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
5,유산균 증식 및 유해균 억제·배변활동 원활·장건강에 도움을 줄 수 있음,• 질환이 있거나 의약품을 섭취하시는 분은 섭취 전 전문가와 상담 후 섭취하세요. \n• 특이체질，알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 전문가와 상담 후 섭취여부를 결정하세요. \n• 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다.\n• 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도하시기 바랍니다. \n• 뚜껑 내 동봉된 방습제는 드시지 마십시오.
6,[프로바이오틱스 제품]①유산균 증식 및 유해균 억제에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 수 있음 \r\n[아연]①정상적인 면역기능에 필요②정상적인 세포분열에 필요,① 특정식품에 알레르기 증상이 계신 분은 성분을 확인 후 섭취하십시오.\r\n② 약물 복용 중이신 분은 섭취 전에 의사，약사 등 전문가와 상의하십시오.


In [279]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['．'])[['product_function', 'product_caution']] # 숫자 다음에 나오는 경우 '.'으로 대체 이외에는 구분자로 사용 '\n'으로 대체

,product_function,product_caution
0,"1. 미강주정추출물(기능성원료인정제2018-3호) : 수면에 도움을 줄 수 있음\n2．비타민 B2 : 체내 에너지 생성에 필요\n3．비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n4. 나이아신 : 체내 에너지 생성에 필요\n5. 셀렌 : 유해산소로부터 세포를 보호하는데 필요","특이체질이나 알레르기 체질인 경우 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n- 의약품(항히스타민제 또는 중추신경계억제제)과 병용섭취 시 주의\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
1,"1. 비타민 B1 : 탄수화물과 에너지 대사에 필요\n2．비타민 B2 : 체내 에너지 생성에 필요\n3．나이아신 : 체내 에너지 생성에 필요\n4. 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n5．비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6. 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n7．비타민B12 : 정상적인 엽산 대사에 필요\n8. 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 유통기한이 경과된 제품은 섭취하지 마십시오.\n- 용기 안의 방습제는 섭취하지 마십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
2,"1. 비타민 B1 : 탄수화물과 에너지 대사에 필요\n2．비타민 B2 : 체내 에너지 생성에 필요\n3．나이아신 : 체내 에너지 생성에 필요\n4. 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n５．비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6. 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n７．비타민B12 : 정상적인 엽산 대사에 필요\n8. 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 소비기한이 경과된 제품은 섭취하지 마십시오.\n- 용기 안의 방습제는 섭취하지 마십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
3,"1. 비타민A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n2. 비타민 B1 : 탄수화물과 에너지 대사에 필요\n3．비타민 B2 : 체내 에너지 생성에 필요\n4. 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n５．비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6. 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n7. 비타민C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n8. 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요\n9. 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n10. 비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n11. 칼슘 : 뼈와 치아 형성에 필요, 신경과 근육 기능 유지에 필요, 정상적인 혈액응고에 필요, 골다공증발생 위험 \n감소에 도움을 줌",특이체질이나 알레르기 체질인 경우 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
4,은행잎 추출물\n①기억력 개선 ．혈행 개선에 도움을 줄 수 있음\n\n마리골드꽃추출물\n①노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음,"은행잎 추출물\n①임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의\n②의약품(항응고제) 복용 시 섭취에 주의\n\n마리골드꽃추출물\n①영．유아, 어린이, 임산부 및 수유부는 섭취를 피할 것\n②흡연자는 섭취 시 전문가와 상담할 것\n③과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음\n④이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,[마그네슘]①에너지 이용에 필요②신경과 근육 기능 유지에 필요\n\n[비타민B6]①단백질 및 아미노산 이용에 필요②혈액의 호모시스테인 수준을 정상으로 유지하는데 필요,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며，섭취량 및 섭취방법을 준수하시기 바랍니다．"
6,[곤약감자추출물] 피부 보습에 도움을 줄 수 있음\n[비타민B2] 체내 에너지 생성에 필요\n[비타민C] 결합조직 형성과 기능유지에 필요 / 철의 흡수에 필요 / 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요 ／ 정상적인 세포분열에 필요,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며，섭취량 및 섭취방법을 준수하시기 바랍니다．\n③ 임산부와 수유기 여성은 섭취를 피하십시오.\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."


In [280]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['／'])[['product_function', 'product_caution']] # 구분자로 사용 '\n'으로 대체

product_caution 컬럼값에 "／" 문자열이 없습니다.


,product_function,product_caution
0,[곤약감자추출물] 피부 보습에 도움을 줄 수 있음\n[비타민B2] 체내 에너지 생성에 필요\n[비타민C] 결합조직 형성과 기능유지에 필요 / 철의 흡수에 필요 / 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[아연] 정상적인 면역기능에 필요 ／ 정상적인 세포분열에 필요,"① 특이체질, 알레르기 체질의 경우 성분을 확인하시고 섭취하여 주시기 바랍니다.\n② 유통기한을 확인하시기 바라며，섭취량 및 섭취방법을 준수하시기 바랍니다．\n③ 임산부와 수유기 여성은 섭취를 피하십시오.\n④ 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."


In [281]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['７'])[['product_function', 'product_caution']] # 특수문자 숫자 -> 숫자로 대체

product_caution 컬럼값에 "７" 문자열이 없습니다.


,product_function,product_caution
0,"1. 비타민 B1 : 탄수화물과 에너지 대사에 필요\n2．비타민 B2 : 체내 에너지 생성에 필요\n3．나이아신 : 체내 에너지 생성에 필요\n4. 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n５．비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6. 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n７．비타민B12 : 정상적인 엽산 대사에 필요\n8. 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요","특이체질, 알레르기 체질의 경우에는 간혹 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오.\n- 소비기한이 경과된 제품은 섭취하지 마십시오.\n- 용기 안의 방습제는 섭취하지 마십시오.\n- 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."


In [282]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['：'])[['product_function', 'product_caution']] # ':'으로 대체 후 일괄 처리

product_caution 컬럼값에 "：" 문자열이 없습니다.


,product_function,product_caution
0,"감마리놀렌산함유유지 : 혈중 콜레스테롤 개선 · 혈행개선 · 월경전 변화에 의한 불편한 상태 개선 · 면역과민반응에 의한 피부상태 개선에 도움을 줄 수 있음\n알로에겔 : 피부건강 · 장 건강 · 면역력 증진에 도움을 줄 수 있음.\n아연：정상적인 면역기능에 필요, 정상적인 세포분열에 필요.",영·유아·어린이는 섭취 전 의사와 상담한 후 섭취할 것
1,아연 : (1) 정상적인 면역기능에 필요 (2) 정상적인 세포분열에 필요\n비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n나이아신 : 체내 에너지 생성에 필요\n비타민D：(1) 칼슘과 인이 흡수되고 이용되는데 필요 (2) 뼈의 형성과 유지에 필요 (3) 골다공증발생 위험 감소에 도움을 줌\n비타민B6 : (1) 단백질 및 아미노산 이용에 필요 (2) 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n비타민B1 : 탄수화물과 에너지 대사에 필요,(가) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
2,"1) 비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요. \n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌.\n3) 비타민E：항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n4) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요.","섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취 하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있고, 개봉 시 내용물이 흘러나올 수 있으므로취급 시 주의하십시오\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
3,"1) 비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요. \n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌.\n3) 비타민E：항산화작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n4) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요.","섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취 하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있고, 개봉 시 내용물이 흘러나올 수 있으므로취급 시 주의하십시오\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
4,1) 홍삼 : 면역력 증진 · 피로개선 · 혈소판 응집억제를 통한 혈액흐름 · 기억력개선 · 항산화에 도움을 줄 수 있음. \n2) 비타민E：항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.,"섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오. \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취 하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있고, 개봉 시 내용물이 흘러나올 수 있으므로취급 시 주의하십시오.\n당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의하십시오.\n알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."


In [283]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['𐩐'])[['product_function', 'product_caution']] # ':'으로 대체 후 일괄 처리

product_function 컬럼값에 "𐩐" 문자열이 없습니다.


,product_function,product_caution
0,(국문) 체지방 감소에 도움을 줄 수 있음\n(영문) May help to reduce body fat,"1. 영𐩐유아, 어린이, 임산부 및 수유부는 섭취에 주의하시기 바랍니다.\n2. 카페인이 함유되어 있어 초조감, 불면 등을 나타낼 수 있습니다.\n3. 섭취량 및 섭취방법을 준수하시고 기타 사항은 고객상담실로 문의주시기 바랍니다.\n4. 특이체질 또는 알레르기 체질인 경우에는 개인에 따라 과민반응을 나타낼 수 있으므로 원재료를 확인한 후 섭취하시기 바랍니다.\n5. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다."


In [284]:
find_chars_df(p_df, ['product_function', 'product_caution'], ['<'])[['product_function', 'product_caution']] # ':'으로 대체 후 일괄 처리

,product_function,product_caution
0,"1) 피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL> : 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음, 갱년기 여성의 건강에 도움을 줄 수 있음.\n2) 비타민C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n임산부 및 수유부와 12세 이하의 어린이는 섭취를 피하십시오.\n수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n식사 후 또는 식사와 함께 섭취하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오."
1,"1) 피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL> : 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음, 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하는데 도움을 줄 수 있음, 갱년기 여성의 건강에 도움을 줄 수 있음.\n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌.","섭취 시 목에 걸릴 수 있으므로 반드시 물과 함께 섭취하십시오. \n섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오.\n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.\n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오.\n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취하십시오.\n임산부 및 수유부와 12세 이하의 어린이는 섭취를 피하십시오.\n수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.\n식사 후 또는 식사와 함께 섭취하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오."
2,<프로바이오틱스> 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음\n\n<테아닌> 스트레스로 인한 긴장완화에 도움을 줄 수 있음,"1) 질환이 있거나 의약품 복용 시 의사, 약사 등 전문가와 상담 후 섭취하시기 바랍니다.\n2) 특정성분에 알레르기가 있는 분은 과민반응이 나타날 수 있으니 섭취 전에 반드시 원료(성분)를 확인하시기 바랍니다.\n3) 어린이가 함부로 섭취하지 않도록 일일섭취량을 확인하시기 바랍니다.\n4) 이상사례 발생 시 섭취를 중단하고 의사, 약사 등 전문가와 상담하시기 바랍니다.\n5) 임산부, 수유부, 어린이 및 수술전후 환자는 섭취에 주의하시기 바랍니다.\n6) 카페인 함유음료(커피, 홍차, 녹차 등)와의 병용 섭취에 주의하시기 바랍니다."
3,<마카젤라틴화분말>\n①정자 운동성 개선에 도움을 줄 수 있음\n<아연>\n①정상적인 면역기능에 필요 ②정상적인 세포분열에 필요\n<옥타코사놀>\n①지구력 증진에 도움을 줄 수 있음,"특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따라 과민반응이 나타날 수 있으므로 원료를 확인하시고, 섭취전에 전문가와 상담하시기 \n바랍니다."
4,[피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호)]\n① 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n② 혈액의 흐름을 방해할 수 있는 혈소판응집을 억제하는데 도움을 줄 수 있음\n[망간]\n① 뼈 형성에 필요 \n② 에너지 이용에 필요 \n③ 유해산소로부터 세포를 보호하는데 필요\n[셀레늄(또는 셀렌)]\n유해산소로부터 세포를 보호하는데 필요\n[비타민B12]\n정상적인 엽산 대사에 필요\n[비타민B6]\n① 단백질 및 아미노산 이용에 필요 \n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B2]\n체내 에너지 생성에 필요,"① 알레르기 체질이신 경우 성분을 확인한 후 섭취여부를 결정하여야 함\n[피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호)]\n② 임산부 및 수유부와 12세 이하의 어린이는 섭취를 피할 것 \n③ 수술 전후, 질환이 있거나 의약품 복용 시 전문가와 상담할 것 \n④ 식사 후 또는 식사와 함께 섭취할 것 \n⑤ 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것"
5,"<정제>\n1) 비타민 A : 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요\n2) 비타민 E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n3) 비타민 K : 정상적인 혈액응고에 필요, 뼈의 구성에 필요\n4) 판토텐산 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n5) 엽산 : 세포와 혈액생성에 필요, 태아 신경관의 정상 발달에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n6) 비타민B12 : 정상적인 엽산 대사에 필요\n7) 비오틴 : 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n8) 비타민 C : 결합조직 형성과 기능유지에 필요, 철의 흡수에 필요, 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n9) 셀렌 : 유해산소로부터 세포를 보호하는데 필요\n10) 망간 : 뼈 형성에 필요, 에너지 이용에 필요, 유해산소로부터 세포를 보호하는데 필요 \n11) 몰리브덴 : 산화·환원 효소의 활성에 필요\n12) 크롬 : 체내 탄수화물, 지방, 단백질 대사에 관여\n\n<액상>\n1) 비타민 D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌\n2) 비타민 B1 : 탄수화물과 에너지 대사에 필요\n3) 비타민 B2 : 체내 에너지 생성에 필요...","1) 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n2) 항응고제 등 복용 시 전문가와 상담하십시오.\n3) 당뇨병이 있는 경우 섭취 전 전문가와 상담하십시오.\n4) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n5) 알레르기 특이체질이신 경우 성분을 확인하신 후 섭취하시기 바랍니다.\n6) 제품의 개봉 또는 섭취 시, 포장재에 의해 상처를 입을 수 있으니 주의하십시오."
6,<아연>\n①정상적인 면역기능에 필요\n②정상적인 세포분열에 필요\n\n<MSM>\n① 관절 및 연골건강에 도움을 줄 수 있음,(가) 신장질환이 있는 사람은 섭취 전 전문가와 상담할 것\n(나) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
7,"<정제1>\n[비타민A] ① 어두운 곳에서 시각 적응을 위해 필요 ② 피부와 점막을 형성하고 기능을 유지하는데 필요 ③ 상피세포의 성장과 발달에 필요\n[비타민D] ① 칼슘과 인이 흡수되고 이용되는데 필요 ② 뼈의 형성과 유지에 필요 ③ 골다공증발생 위험 감소에 도움을 줌\n[비타민E] ① 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[비타민K] ① 정상적인 혈액응고에 필요 ② 뼈의 구성에 필요\n[비타민B1] ① 탄수화물과 에너지 대사에 필요\n[비타민B2] ① 체내 에너지 생성에 필요\n[나이아신] ① 체내 에너지 생성에 필요\n[판토텐산] ① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비오틴] ① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민B6] ① 단백질 및 아미노산 이용에 필요 ② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[엽산] ① 세포와 혈액생성에 필요 ② 태아 신경관의 정상 발달에 필요 ③ 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비타민B12] ① 정상적인 엽산 대사에 필요\n[비타민C] ① 결합조직 형성과 기능유지에 필요 ② 철의 흡수에 필요 ③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요\n[철] ① 체내 산소운반과 혈액생성에 필요 ② 에너지 생성에 필요\n[구리] ① 철의 운반과 이용에 필요 ② 유해산소로부터 세포를 보호하는...","정

## 건강기능, 주의사항 컬럼에 대해 텍스트 전처리

In [285]:
from data_processing import cleaned_product_df_data

p_df, cleaned_function_data = cleaned_product_df_data(p_df, 'product_function', 'product_function_content')
p_df, cleaned_caution_data = cleaned_product_df_data(p_df, 'product_caution', 'product_caution_content')

In [286]:
p_df[['product_function', 'product_function_content']]

,product_function,product_function_content
0,혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.,혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.
2,①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선⑤항산화에 도움을 줄 수 있음,면역력 증진\n피로개선\n혈소판 응집 억제를 통한 혈액흐름에 도움\n기억력 개선\n항산화에 도움을 줄 수 있음
3,⑴ 면역력 증진에 도움을 줄 수 있음\n ⑵ 피로개선에 도움을 줄 수 있음\n ⑶ 혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n ⑷ 기억력 개선에 도움을 줄 수 있음\n ⑸ 항산화에 도움을 줄 수 있음,면역력 증진에 도움을 줄 수 있음\n피로개선에 도움을 줄 수 있음\n혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n기억력 개선에 도움을 줄 수 있음\n항산화에 도움을 줄 수 있음
8,[EPA 및 DHA 함유유지] 혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음.\n[비타민E] 유해산소로부터 세포를 보호하는데 필요,EPA 및 DHA 함유유지\n혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음.\n비타민E\n유해산소로부터 세포를 보호하는데 필요
9,[비타민D] 칼슘과 인이 흡수되고 이용되는데 필요 / 뼈의 형성과 유지에 필요 / 골다공증발생 위험 감소에 도움을 줌,비타민D\n칼슘과 인이 흡수되고 이용되는데 필요\n뼈의 형성과 유지에 필요\n골다공증발생 위험 감소에 도움을 줌
...,...,...
36786,탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음,탄수화물이 지방으로 합성되는 것을 억제하여 체지방 감소에 도움을 줄 수 있음
36787,"[비타민 B1]\n① 탄수화물과 에너지 대사에 필요\n[비타민 B2]\n① 체내 에너지 생성에 필요\n[나이아신]\n① 체내 에너지 생성에 필요\n[판토텐산]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민 B6]\n① 단백질 및 아미노산 이용에 필요\n② 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[엽산]\n① 세포와 혈액생성에 필요\n② 태아 신경관의 정상 발달에 필요\n③ 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n[비오틴]\n① 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n[비타민 B12]\n① 정상적인 엽산 대사에 필요\n[아연]\n① 정상적인 면역기능에 필요\n② 정상적인 세포분열에 필요\n[크롬]\n① 체내 탄수화물, 지방, 단백질 대사에 관여\n[비타민 C]\n① 결합조직 형성과 기능유지에 필요\n② 철의 흡수에 필요\n③ 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요","비타민 B1\n탄수화물과 에너지 대사에 필요\n비타민 B2\n체내 에너지 생성에 필요\n나이아신\n체내 에너지 생성에 필요\n판토텐산\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n비타민 B6\n단백질 및 아미노산 이용에 필요\n혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n엽산\n세포와 혈액생성에 필요\n태아 신경관의 정상 발달에 필요\n혈액의 호모시스테인 수준을 정상으로 유지하는데 필요\n비오틴\n지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n비타민 B12\n정상적인 엽산 대사에 필요\n아연\n정상적인 면역기능에 필요\n정상적인 세포분열에 필요\n크롬\n체내 탄수화물, 지방, 단백질 대사에 관여\n비타민 C\n결합조직 형성과 기능유지에 필요\n철의 흡수에 필요\n항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요"
36788,﻿[철]\n① 체내 산소운반과 혈액생성에 필요\n② 에너지 생성에 필요,철\n체내 산소운반과 혈액생성에 필요\n에너지 생성에 필요
36789,﻿[피크노제놀-프랑스해안송껍질추출물(제2006-1호)]\n① 인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n② 갱년기 여성의 건강에 도움을 줄 수 있음\n[테아닌]\n스트레스로 인한 긴장완화에 도움을 줄 수 있음,피크노제놀-프랑스해안송껍질추출물(제2006-1호)\n인체에 유해한 활성산소를 제거하는데 도움을 줄 수 있음\n갱년기 여성의 건강에 도움을 줄 수 있음\n테아닌\n스트레스로 인한 긴장완화에 도움을 줄 수 있음


## 건강기능 코드성 컬럼 생성

In [287]:
from stopwords import stopwords
from updated_nouns_words import updated_nouns_words
from konlpy.tag import Hannanum

from data_processing import created_text_to_nonus, process_with_hannanum_for_stopwords, process_text_for_stopwords

### 불용어 체크

In [288]:
# # 불용어 체크용 함수적용
# function_text_df = process_text_for_stopwords(cleaned_function_data, stopwords, updated_nouns_words)
# caution_text_df = process_text_for_stopwords(cleaned_caution_data, stopwords, updated_nouns_words)

In [289]:
# # 확인
# function_text_df[['origin_text', 'hannanum']]

### 텍스트를 명사화하여 코드성 컬럼으로 분류

In [290]:
# 코드성 컬럼을 생성하기 위해 명사화 함수적용
p_df = created_text_to_nonus(p_df, 'product_function_content', 'function_nonus')
p_df = created_text_to_nonus(p_df, 'product_caution_content', 'caution_nonus')

In [ ]:
# 확인
p_df[['product_function_content', 'function_nonus', 'product_caution_content', 'caution_nonus']].head()

,product_function_content,function_nonus,product_caution_content,caution_nonus
0,혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.,혈중 콜레스테롤 수치 개선,"임산부, 수유부, 청소년 및 어린이는 섭취를 피하십시오.\n심혈관약 등 복용, 혈액 응고장애가 있거나 수술예정인 사람은 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n본 제품은 섭취자의 신체상태에 따라 반응에 차이가 있을 수 있습니다.\n임산부, 수유부, 질병으로 인해 치료중인 분은 섭취 전 고객센터나 구입처로 문의하십시오.\n특정 성분에 알레르기 체질이신 분은 섭취 전에 반드시 원료(성분)를 확인하신 후 섭취하시기 바랍니다.\n섭취량 및 섭취방법을 확인 한 후 섭취하시고, 기타 제품관련사항은 고객센터로 문의해 주십시오.",임산부 수유부 청소년 어린이 섭취 피하 심혈관약 복용 혈액 응고장애 수술예정인 사람은 전문가 상담 이상사례 발생 섭취 중단 전문가 상담 제품 섭취자의 신체상태 반응 차이 임산부 수유부 질병 치료중인 분 섭취 전 고객센터 구입처 문의 특정 성분 알레르기 체질이 분 섭취 전 원료 성분 확인 후 섭취하시기 섭취량 섭취방법 확인 후 섭취하시 기타 제품관련사항 고객센터 문의해
2,면역력 증진\n피로개선\n혈소판 응집 억제를 통한 혈액흐름에 도움\n기억력 개선\n항산화에 도움을 줄 수 있음,면역력 증진 피로개선 혈소판 응집 억제 혈액흐름 기억력 개선 항산화,"홍삼제품\n의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.\n15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.\n제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의 하십시오.",홍삼제품 의약품 당뇨치료제 혈액항응고제 복용 섭취 주의 특이체질등 알레르기 체질 경우 제품성분 확인 후 섭취하시기 15세 이하 어린이 상기 섭취량 절반 정도 섭취하시요 제품 개봉 섭취시 포장재 상처 있으니주 하십시오
3,면역력 증진에 도움을 줄 수 있음\n피로개선에 도움을 줄 수 있음\n혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n기억력 개선에 도움을 줄 수 있음\n항산화에 도움을 줄 수 있음,면역력 증진 피로개선 혈소판 응집 억제 혈액흐름 기억력 개선 항산화,"특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n제품 개봉 또는 섭취 시 포장재에 의한 상처에 주의하시기 바랍니다.\n의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의 하십시오.\n유통기한이 경과된 제품은 섭취하지 마십시오.",특수체질 알레르기 체질인 경우 섭취 전 원료명 함량 확인하시기 제품 개봉 섭취 포장재 상처 주의하시기 의약품 당뇨치료제 혈액항응고제 복용 섭취 주의 하십시오 유통기한 경과 제품 섭취
8,EPA 및 DHA 함유유지\n혈중 중성지질 개선·혈행 개선에 도움을 줄 수 있음.\n비타민E\n유해산소로부터 세포를 보호하는데 필요,EPA DHA 함유유지 혈중 중성지질 개선 혈행 개선 비타민E 유해산소 세포 보호,알레르기 체질이신 경우 성분을 확인 한 후 섭취하시기 바랍니다.,알레르기 체질이 경우 성분 확인 후 섭취하시기
9,비타민D\n칼슘과 인이 흡수되고 이용되는데 필요\n뼈의 형성과 유지에 필요\n골다공증발생 위험 감소에 도움을 줌,비타민D 칼슘 인 흡수 이용 뼈 형성 골다공증발생 위험 감소,고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.,고칼슘혈증 의약품 복용 전문가 상담 이상사례 발생 섭취 중단 전문가 상담


### 건강기능 코드성 컬럼 생성

In [ ]:
from function_dict import function_dict
print(f'function_dict : {function_dict}')

from data_processing import create_dict_col, check_zero_value, create_json_code_col

function_dict : {'HF00': ['아무것도 선택하지 않음'], 'HF01': ['간 건강', '간건강', '간 보호', '간손상', '간 손상', '알콜'], 'HF02': ['피로개선', '피로 개선', '에너지 이용', '에너지 생성', '탄수화물 에너지 대사', '지방 탄수화물 단백질 대사 에너지 생성'], 'HF03': ['뼈', '골다공증', '관절', '연골', '칼슘'], 'HF04': ['충치', '치아건강', '구강', '자일리톨'], 'HF05': ['면역기능', '면역 기능', '면역력', '면역조절', '면역 과민반응 개선', '세포분열', '코 상태', '코상태', '코막힘'], 'HF06': ['유해산소', '항산화', '산화', '유해산소', '유해 산소', '활성산소', '활성 산소'], 'HF07': ['피부', '상피세포', '모발상태 윤기 탄력 개선'], 'HF08': ['눈 건강', '눈건강', '건조한 눈 개선', '눈 피로', '시각'], 'HF09': ['위 점막', '위 건강', '위건강', '지방 소화', '위 불편감 개선', '소화 기능', '소화기능'], 'HF10': ['배변활동', '배변 활동', '식이섬유', '장 건강', '장내', '락추로스'], 'HF11': ['방광', '배뇨'], 'HF12': ['요로'], 'HF13': ['전립선'], 'HF14': ['갱년기 남성', '남성 갱년기', '정자 운동성'], 'HF15': ['갱년기 여성'], 'HF16': ['혈액 생성', '질 건강', '엽산', '철', '질 유익균 증식 유해균 억제', '월경'], 'HF17': ['결합조직', '근력', '근육', '운동수행', '운동능력', '운동수행능력', '지구성 운동', '지구력', '단백질 아미노산 이용', '에너지 포도당 지질 합성', '근육 결합조직 신체조직', '과도한 운동', '신체저항능력'], 'HF18': ['체지방', '체내 탄수화물 지방 단백질 대사 관여'], 'HF19'

In [ ]:
# 건강기능 코드성 컬럼 생성
p_df = create_dict_col(p_df, 'function_nonus', function_dict)
p_df[function_dict.keys()]

function_dict : {'HF00': ['아무것도 선택하지 않음'], 'HF01': ['간 건강', '간건강', '간 보호', '간손상', '간 손상', '알콜'], 'HF02': ['피로개선', '피로 개선', '에너지 이용', '에너지 생성', '탄수화물 에너지 대사', '지방 탄수화물 단백질 대사 에너지 생성'], 'HF03': ['뼈', '골다공증', '관절', '연골', '칼슘'], 'HF04': ['충치', '치아건강', '구강', '자일리톨'], 'HF05': ['면역기능', '면역 기능', '면역력', '면역조절', '면역 과민반응 개선', '세포분열', '코 상태', '코상태', '코막힘'], 'HF06': ['유해산소', '항산화', '산화', '유해산소', '유해 산소', '활성산소', '활성 산소'], 'HF07': ['피부', '상피세포', '모발상태 윤기 탄력 개선'], 'HF08': ['눈 건강', '눈건강', '건조한 눈 개선', '눈 피로', '시각'], 'HF09': ['위 점막', '위 건강', '위건강', '지방 소화', '위 불편감 개선', '소화 기능', '소화기능'], 'HF10': ['배변활동', '배변 활동', '식이섬유', '장 건강', '장내', '락추로스'], 'HF11': ['방광', '배뇨'], 'HF12': ['요로'], 'HF13': ['전립선'], 'HF14': ['갱년기 남성', '남성 갱년기', '정자 운동성'], 'HF15': ['갱년기 여성'], 'HF16': ['혈액 생성', '질 건강', '엽산', '철', '질 유익균 증식 유해균 억제', '월경'], 'HF17': ['결합조직', '근력', '근육', '운동수행', '운동능력', '운동수행능력', '지구성 운동', '지구력', '단백질 아미노산 이용', '에너지 포도당 지질 합성', '근육 결합조직 신체조직', '과도한 운동', '신체저항능력'], 'HF18': ['체지방', '체내 탄수화물 지방 단백질 대사 관여'], 'HF19'

,HF00,HF01,HF02,HF03,HF04,HF05,HF06,HF07,HF08,HF09,...,HF17,HF18,HF19,HF20,HF21,HF22,HF23,HF24,HF25,HF26
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,0,0,1,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36786,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
36787,0,0,1,0,0,1,1,0,0,0,...,1,1,0,0,0,1,0,1,1,0
36788,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36789,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
# 건강기능 컬럼 예외사항 - 하드코딩
p_df.loc[p_df['product_id'] == 201000190041272, 'HF01'] = 1

p_df.loc[p_df['product_id'] == 20040020006500, 'HF08'] = 1

p_df.loc[p_df['product_id'] == 20040015107616, 'HF18'] = 1
p_df.loc[p_df['product_id'] == 20040015107616, 'HF22'] = 1
p_df.loc[p_df['product_id'] == 20040015107616, 'HF24'] = 1

p_df.loc[p_df['product_id'] == 20130020024148, 'HF10'] = 1

In [ ]:
# 건강기능 코드 컬럼 모든 값이 0인 행 조회
check_zero_value(p_df, function_dict)[['product_id', 'product_function', 'function_nonus']] # 모두 건강기능코드 존재 확인

,product_id,product_function,function_nonus


In [ ]:
# 중간 백업 - 피클 파일 저장
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_processing.pkl', 'wb') as f:
    pickle.dump(p_df, f)   

In [ ]:
# 백업파일 열기
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_processing.pkl', 'rb') as f:
    p_df = pickle.load(f)

## 건강기능 JSON 컬럼 생성

In [ ]:
# 건강기능 코드 JSON 컬럼 생성
p_df = create_json_code_col(p_df, list(function_dict.keys()), 'product_function_code')

# 확인
p_df['product_function_code']

0        {'HF00': 0, 'HF01': 0, 'HF02': 0, 'HF03': 0, 'HF04': 0, 'HF05': 0, 'HF06': 0, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 0, 'HF18': 0, 'HF19': 0, 'HF20': 0, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 1, 'HF25': 0, 'HF26': 0}
2        {'HF00': 0, 'HF01': 0, 'HF02': 1, 'HF03': 0, 'HF04': 0, 'HF05': 1, 'HF06': 1, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 0, 'HF18': 0, 'HF19': 0, 'HF20': 1, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 1, 'HF25': 0, 'HF26': 0}
3        {'HF00': 0, 'HF01': 0, 'HF02': 1, 'HF03': 0, 'HF04': 0, 'HF05': 1, 'HF06': 1, 'HF07': 0, 'HF08': 0, 'HF09': 0, 'HF10': 0, 'HF11': 0, 'HF12': 0, 'HF13': 0, 'HF14': 0, 'HF15': 0, 'HF16': 0, 'HF17': 0, 'HF18': 0, 'HF19': 0, 'HF20': 1, 'HF21': 0, 'HF22': 0, 'HF23': 0, 'HF24': 1, 'HF25': 0, 'HF26': 0}
8        {'HF00': 0, 'HF01': 0, 'HF02': 0, 'HF03': 0, 'HF04': 0, 'HF05': 0, 'HF

## 주의사항 코드성 컬럼 생성

In [ ]:
# 주의사항 코드성 컬럼 생성
from caution_dict import caution_dict
print(f'caution_dict : {caution_dict}')

p_df = create_dict_col(p_df, 'caution_nonus', caution_dict)

caution_dict : {'DI01': ['위 간 신장 질환', '위 신장 간질환', '위장관 질환', '위장관질환', '위장관 장애오메프라졸', '위산분비 억제제', '위산분비억제제'], 'DI02': ['간 질환', '간질환', '간 심장질환', '간 신장 질환', '간 신장 심장질환', '간 신장 기능 이상자', '간기능 저하', '위 간 신장 질환', '위 신장 간질환', '담관'], 'DI03': ['심장', '심혈관계 질환', '심혈관약 복용', '부정맥치료제', '부정맥 치료제'], 'DI04': ['신장', '신부전'], 'DI05': ['당뇨치료제', '당뇨 천식', '당뇨병 치료 받으시는 분', '당뇨환자', '당뇨 환자', '당뇨병 있는 경우', '당뇨 혈압 질환', '항당뇨제', '혈당강하제', '당뇨 질환', '당뇨질환', '높은 혈당'], 'DI06': ['고혈압', '혈압강하제', '혈압약', '혈압 질환', '혈압질환'], 'DI07': ['고지혈증'], 'DI08': ['항응고제', '항혈소판제', '비스테로이드계 항염증약', '혈압조절제', '혈액응고', '항혈전', '혈전용해제', '출혈'], 'DI09': ['갑상선'], 'DI10': ['천식', '기관지', '해소', '기도 심한 염증'], 'DI11': ['담낭'], 'DI12': ['면역억제제', '자가면역'], 'DI13': ['수면', '신경', '항히스타민제', '항우울제', '항경련제', '항뇌전증제'], 'DI14': ['호르몬제', '에스트로겐', '대두'], 'DI15': ['단장증후군', '유당불내증'], 'DI16': ['광과민성'], 'DI17': ['호르몬제를 복용'], 'AL01': ['게', '새우'], 'AL02': ['옻'], 'AL03': ['땅콩'], 'AL04': ['프로폴리스'], 'AL05': ['대두알레르기', '대두 알레르기', '대두 난황 알레르기', '대두단백'], 'AL06': ['대두 이소플라본', '에스트로겐'], '

## 주의사항 JSON 컬럼 생성

In [ ]:
# 주의사항 코드 JSON 컬럼 생성
p_df = create_json_code_col(p_df, caution_dict, 'product_caution_code')

# 확인
p_df['product_caution_code']

0        {'DI01': 0, 'DI02': 0, 'DI03': 1, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 1, 'P2': 1, 'P1': 0, 'INFANT_TODDLER': 1, 'KIDS': 1, 'TEENAGER': 1, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 1, 'O2': 1, 'O1': 1, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}
2        {'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 1, 'DI06': 0, 'DI07': 0, 'DI08': 1, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 1, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0,

In [ ]:
# 중간 백업 - 피클 파일 저장
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_processing.pkl', 'wb') as f:
    pickle.dump(p_df, f)   

In [ ]:
# 백업파일 열기
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_processing.pkl', 'rb') as f:
    p_df = pickle.load(f)

## `product_ingredient_id` 컬럼 생성 

- '품목유형(기능지표성분)' : `ingredient_type1`
- '기능성 원재료' : `ingredeint_type2`
- 품목유형을 바탕으로 `product_ingredient_id`를 생성하고, 결측치가 존재할 경우 기능성 원재료 컬럼을 참고한다.

In [ ]:
# 전처리 완료된 영양성분 데이터셋 가져오기
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\ingredient_processing.pkl', 'rb') as f:
    i_df = pickle.load(f)

In [ ]:
i_df['ingredient_id'] = i_df['ingredient_id'].astype(int)

In [ ]:
i_df[['ingredient_id', 'ingredient_raw_name', 'ingredient_name', 'ingredient_auth_num', 'ingredient_grp_name', 'ingredient_recom_name']]

,ingredient_id,ingredient_raw_name,ingredient_name,ingredient_auth_num,ingredient_grp_name,ingredient_recom_name
0,428,아연,아연,NaN,아연,아연
1,426,철,철,NaN,철,철
2,424,비타민 B6,비타민 B6,NaN,비타민 B6,비타민B6
3,425,마그네슘,마그네슘,NaN,마그네슘,마그네슘
4,429,요오드,요오드,NaN,요오드,요오드
...,...,...,...,...,...,...
431,434,백편두추출분말(NOVAponinⓇ)(제2024-6호),백편두추출분말(NOVAponinⓇ),제2024-6호,백편두추출분말,백편두추출분말
432,435,발효굴추출물(제2023-41호),발효굴추출물,제2023-41호,발효굴추출물,발효굴추출물
433,436,풋사과 농축분말(제2024-4호),풋사과 농축분말,제2024-4호,풋사과 농축분말,풋사과농축분말
434,437,기장밀추출복합물(KeranatTM)(제2024-16호),기장밀추출복합물(KeranatTM),제2024-16호,기장밀추출복합물,기장밀추출복합물


### 품목유형이 결측치일 경우 기능성 원재료 컬럼 값으로 대체

In [ ]:
# 품목유형이 결측치인 값 확인
p_df[p_df['ingredient_type1'].isnull()][['product_id', 'ingredient_type1', 'ingredient_type2']]

,product_id,ingredient_type1,ingredient_type2
1137,201600100021,NaN,BT-11원지추출분말
35162,2006001800222,NaN,홍삼(고시형)
35583,202300010424,NaN,"홍삼농축액(고시형), 벌꿀, L-아르지닌, 정제수"
36076,2007001901812,NaN,홍삼농축액(기능성원료 인정 제2012-8호)


In [ ]:
# 기능성 원재료 컬럼에서 '(고시형)' 문자열 제거 후 품목유형 컬럼값의 결측치를 대체한다.
p_df['ingredient_raw_name'] = p_df['ingredient_type1']
p_df.loc[p_df['ingredient_type1'].isnull(), 'ingredient_raw_name'] = p_df.loc[p_df['ingredient_type1'].isnull(), 'ingredient_type2'].apply(lambda x: re.sub(r'\(고시형\)', '', x))

# 확인
p_df[p_df['ingredient_type1'].isnull()][['ingredient_type1', 'ingredient_type2', 'ingredient_raw_name']]

,ingredient_type1,ingredient_type2,ingredient_raw_name
1137,NaN,BT-11원지추출분말,BT-11원지추출분말
35162,NaN,홍삼(고시형),홍삼
35583,NaN,"홍삼농축액(고시형), 벌꿀, L-아르지닌, 정제수","홍삼농축액, 벌꿀, L-아르지닌, 정제수"
36076,NaN,홍삼농축액(기능성원료 인정 제2012-8호),홍삼농축액(기능성원료 인정 제2012-8호)


### ', '를 기준으로 `ingredient_raw_name` 컬럼 행을 나누기

In [ ]:
from data_processing import normalize_table

# ','를 기준으로 ingredient_raw_name 컬럼 행을 나누는 함수 적용
p_i_df = normalize_table(p_df[['product_id', 'ingredient_raw_name']], 'ingredient_raw_name')
p_i_df

,product_id,ingredient_raw_name
0,200400200021640,폴리코사놀-사탕수수 왁스알코올(2006-4)
1,201100200015,홍삼제품
2,20040015100230,홍삼
3,200400150831149,비타민E
4,200400150831149,EPA 및 DHA 함유 유지
...,...,...
115548,2017286000733,비타민 C
115549,2017286000729,철
115550,2017286000728,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호)
115551,2017286000728,테아닌


In [ ]:
from data_processing import extract_auth_num

# 영양성분명과 기능성원료 인정번호 분리하는 함수 적용
p_i_df = extract_auth_num(p_i_df, 'ingredient_raw_name', 'ingredient_name', 'ingredient_auth_num')
p_i_df

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name
0,200400200021640,폴리코사놀-사탕수수 왁스알코올(2006-4),NaN,폴리코사놀-사탕수수 왁스알코올(2006-4)
1,201100200015,홍삼제품,NaN,홍삼제품
2,20040015100230,홍삼,NaN,홍삼
3,200400150831149,비타민E,NaN,비타민E
4,200400150831149,EPA 및 DHA 함유 유지,NaN,EPA 및 DHA 함유 유지
...,...,...,...,...
115548,2017286000733,비타민 C,NaN,비타민 C
115549,2017286000729,철,NaN,철
115550,2017286000728,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호),제2006-1호,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>
115551,2017286000728,테아닌,NaN,테아닌


## `ingredient_name` 전처리

### '제품' 문자열 제거

In [ ]:
# '제품' 문자열 확인
p_i_df.loc[p_i_df['ingredient_name'].str.contains('제품'), 'ingredient_name'].value_counts()

홍삼제품                   1001
프로바이오틱스 제품              346
인삼제품                    165
프로폴리스추출물 제품             122
오메가-3지방산함유유지 제품         112
                       ... 
 EPA 및 DHA 함유 유지 제품       1
구아바잎 추출물 제품               1
EPA 및 DHA 함유 유지 제품        1
식물스테롤에스테르 제품              1
 홍국 제품                    1
Name: ingredient_name, Length: 84, dtype: int64

In [ ]:
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'제품', '', x))

### 특수문자 예외사항 처리

In [ ]:
# '®'  => 'Ⓡ'로 대체
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'®', 'Ⓡ', x))

# 꺽쇠기호 => 소괄호로 대체
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'\<', '(', x))
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'\>', ')', x))

# '․'  => '·'로 대체
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'\․', '\·', x))

# '뮤코다당.단백' => '뮤코다당·단백'
p_i_df.loc[p_i_df['ingredient_name'].str.contains('뮤코다당'), 'ingredient_name'] = '뮤코다당·단백'

# '.' => '.'로 대체
p_i_df['ingredient_name'] = p_i_df['ingredient_name'].apply(lambda x: re.sub(r'\.', '.', x))

### 소괄호 및 안의 내용 분리

In [ ]:
from data_processing import create_ingredient_grp_name

# 소괄호 분리하는 함수 적용
# 함수 내에서 update name dict의  value의 키워드 포함시 key값으로 값 대체됨
update_ingredient_grp_name_dict = {'자일로올리고당' : ['자일로올리고당'],
                                #    '프로바이오틱스' : ['프로바이오틱스'],
                                #    '루테인/지아잔틴' : ['루테인'],
                                   '오메가3' : ['오메가-3', 'EPA'],
                                   '밀크씨슬' : ['밀크씨슬'],
                                   '셀레늄/셀렌' : ['셀레늄'],
                                   '그린마테 추출물' : ['Green Mate Extract', '그린마떼'],
                                   '홍삼' : ['홍삼농축액']
                                   }

p_i_df = create_ingredient_grp_name(p_i_df, 
                                    'ingredient_name', 
                                    'ingredient_grp_name', 
                                    update_ingredient_grp_name_dict,
                                    'ingredient_bracket_str'
                                    )

# 확인
p_i_df

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str
0,200400200021640,폴리코사놀-사탕수수 왁스알코올(2006-4),NaN,폴리코사놀-사탕수수 왁스알코올(2006-4),폴리코사놀-사탕수수 왁스알코올,(2006-4)
1,201100200015,홍삼제품,NaN,홍삼,홍삼,NaN
2,20040015100230,홍삼,NaN,홍삼,홍삼,NaN
3,200400150831149,비타민E,NaN,비타민E,비타민E,NaN
4,200400150831149,EPA 및 DHA 함유 유지,NaN,EPA 및 DHA 함유 유지,오메가3,NaN
...,...,...,...,...,...,...
115548,2017286000733,비타민 C,NaN,비타민 C,비타민 C,NaN
115549,2017286000729,철,NaN,철,철,NaN
115550,2017286000728,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호),제2006-1호,피크노제놀-프랑스해안송껍질추출물(PYCNOGENOL),피크노제놀-프랑스해안송껍질추출물,(PYCNOGENOL)
115551,2017286000728,테아닌,NaN,테아닌,테아닌,NaN


### 특수문자 및 공백문자 제거

In [ ]:
# 특수문자 및 공백문자 제거
p_i_df['ingredient_recom_name'] = p_i_df['ingredient_grp_name'].apply(lambda x: re.sub(r'[^가-힣A-Za-z0-9]', '', x))

# 확인
p_i_df

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name
0,200400200021640,폴리코사놀-사탕수수 왁스알코올(2006-4),NaN,폴리코사놀-사탕수수 왁스알코올(2006-4),폴리코사놀-사탕수수 왁스알코올,(2006-4),폴리코사놀사탕수수왁스알코올
1,201100200015,홍삼제품,NaN,홍삼,홍삼,NaN,홍삼
2,20040015100230,홍삼,NaN,홍삼,홍삼,NaN,홍삼
3,200400150831149,비타민E,NaN,비타민E,비타민E,NaN,비타민E
4,200400150831149,EPA 및 DHA 함유 유지,NaN,EPA 및 DHA 함유 유지,오메가3,NaN,오메가3
...,...,...,...,...,...,...,...
115548,2017286000733,비타민 C,NaN,비타민 C,비타민 C,NaN,비타민C
115549,2017286000729,철,NaN,철,철,NaN,철
115550,2017286000728,피크노제놀-프랑스해안송껍질추출물<PYCNOGENOL>(제2006-1호),제2006-1호,피크노제놀-프랑스해안송껍질추출물(PYCNOGENOL),피크노제놀-프랑스해안송껍질추출물,(PYCNOGENOL),피크노제놀프랑스해안송껍질추출물
115551,2017286000728,테아닌,NaN,테아닌,테아닌,NaN,테아닌


## `ingredient_auth_num` 존재할 경우 `ingredient_id` 부여

In [ ]:
# ingredeint_auth_num은 존재하는 경우
auth_num_mask =  (p_i_df['ingredient_auth_num'].notnull())
auth_num_df = p_i_df[auth_num_mask] # 2956 행

# # ingredeint_auth_num이 결측치 인 경우
non_auth_num_mask =  (p_i_df['ingredient_auth_num'].isnull())
non_auth_num_df = p_i_df[non_auth_num_mask] # 112597 행

In [ ]:
# ingredient_auth_num을 기준으로 영양성분 데이터셋과 merge
merge_auth_num_df = pd.merge(auth_num_df, i_df[['ingredient_id', 'ingredient_auth_num']], how='left', on='ingredient_auth_num')

# ingredient_id 존재하는 행 확인
merge_auth_num_df_1 = merge_auth_num_df[merge_auth_num_df['ingredient_id'].notnull()] # 2866 행

# ingredient_id 결측치인 행 확인
merge_auth_num_df_2 = merge_auth_num_df[merge_auth_num_df['ingredient_id'].isnull()] # 90 행

# merge_auth_num_df_2 와 non_auth_num_df 를 concat 후 ingredient_id 부여 예정
check_id_df = pd.concat([non_auth_num_df, merge_auth_num_df_2], ignore_index=True)
check_id_df = check_id_df[['product_id', 'ingredient_raw_name', 'ingredient_auth_num', 'ingredient_name', 
                           'ingredient_grp_name', 'ingredient_bracket_str', 'ingredient_recom_name']]
check_id_df      

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name
0,200400200021640,폴리코사놀-사탕수수 왁스알코올(2006-4),NaN,폴리코사놀-사탕수수 왁스알코올(2006-4),폴리코사놀-사탕수수 왁스알코올,(2006-4),폴리코사놀사탕수수왁스알코올
1,201100200015,홍삼제품,NaN,홍삼,홍삼,NaN,홍삼
2,20040015100230,홍삼,NaN,홍삼,홍삼,NaN,홍삼
3,200400150831149,비타민E,NaN,비타민E,비타민E,NaN,비타민E
4,200400150831149,EPA 및 DHA 함유 유지,NaN,EPA 및 DHA 함유 유지,오메가3,NaN,오메가3
...,...,...,...,...,...,...,...
112682,2007001901812,홍삼농축액(기능성원료 인정 제2012-8호),제2012-8호,홍삼농축액,홍삼,NaN,홍삼
112683,20040020001163,히알루론산(제2008-37호),제2008-37호,히알루론산,히알루론산,NaN,히알루론산
112684,200400200021928,감마리놀렌산함유유지(제2015-5호),제2015-5호,감마리놀렌산함유유지,감마리놀렌산함유유지,NaN,감마리놀렌산함유유지
112685,200400200021928,감마리놀렌산함유유지(제2015-4호),제2015-4호,감마리놀렌산함유유지,감마리놀렌산함유유지,NaN,감마리놀렌산함유유지


## `ingredient_recom_name`을 기준으로 `ingredient_id` 부여

In [ ]:
# ingredient_recom_name을 기준으로 영양성분 데이터셋과 merge
merge_recom_name_df = pd.merge(check_id_df , i_df[['ingredient_id', 'ingredient_recom_name']], how='left', on='ingredient_recom_name')

# ingredient_id 존재하는 행 확인
merge_recom_name_df_1 = merge_recom_name_df[merge_recom_name_df['ingredient_id'].notnull()] # 151594 행

# ingredient_id 결측치인 행 확인
merge_recom_name_df_2 = merge_recom_name_df[merge_recom_name_df['ingredient_id'].isnull()] # 4447 행

### 하드코딩 필요한 내용 확인

In [ ]:
merge_recom_name_df_2['ingredient_recom_name'].value_counts()

엠에스엠MSM                                   942
Methylsulfonylmethane                     942
디메틸설폰                                     942
N아세틸글루코사민                                 392
NAG엔에이지                                   331
NAcetylglucosamine                        331
루테인                                       142
차전자피                                      105
MSM                                        84
디메틸설폰Methylsulfonylmethane                 84
키토산                                        36
이눌린                                        20
오비엑스                                       16
대웅코엔자임Q10200719                             5
200836                                      5
곤약감자추출물분말                                   5
PME88메론추출물                                  5
치커리추출물                                      5
Dimethylsulfone                             4
리포젠공액리놀렌산                                   2
로즈힙                                         2
히알우론산HALFP                        

In [ ]:
merge_recom_name_df_2[merge_recom_name_df_2['ingredient_recom_name'].str.contains('표고')]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name
149065,20120019007158,표고버섯균사체추출물분말,NaN,표고버섯균사체추출물분말,표고버섯균사체추출물분말,NaN,표고버섯균사체추출물분말


In [ ]:
i_df[i_df['ingredient_raw_name'].str.contains('루테인')]

,ingredient_raw_name,ingredient_function,ingredient_caution,ingredient_limit_high,ingredient_limit_low,ingredient_unit,ingredient_etc,ingredient_auth_num,ingredient_name,ingredient_grp_name,...,S1,O3,O2,O1,단백질,비타민 A,베타카로틴,요오드,ingredient_caution_code,ingredient_id
96,루테인/지아잔틴 복합추출물(제2008-66호),(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging,-임산부는 섭취 전 의사와 상담이 필요합니다\n-과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있습니다,20,10,mg,NaN,제2008-66호,루테인/지아잔틴 복합추출물,루테인/지아자틴,...,0,0,0,0,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 1, 'P2': 0, 'P1': 0, 'INFANT_TODDLER': 0, 'KIDS': 0, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",289
234,루테인지아잔틴복합추출물20%(제2013-23호),(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging,-임산부는 섭취 전 의사와 상담이 필요합니다.\n-과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있습니다.,20,10,mg/일,NaN,제2013-23호,루테인지아잔틴복합추출물20%,루테인/지아자틴,...,0,0,0,0,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 1, 'P2': 0, 'P1': 0, 'INFANT_TODDLER': 0, 'KIDS': 0, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",195
248,마리골드 추출물(루테인 에스테르)(제2012-22호),(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging,-과다 섭취시 일시적으로 피부가 황색으로 변할 수 있음,20,18.5,mg/일,NaN,제2012-22호,마리골드 추출물(루테인 에스테르),마리골드 추출물,...,0,0,0,0,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 0, 'P2': 0, 'P1': 0, 'INFANT_TODDLER': 0, 'KIDS': 0, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",180
339,루테인지아잔틴복합추출물(제2018-4호),(국문) 노화로 인해 감소될 수 있는 황반색소밀도를 유지하여 눈 건강에 도움을 줄 수 있음\n(영문) May help the eye health by maintaining the density of macular pigments which can be decreased by aging,"-영·유아, 어린이, 임산부 및 수유부는 섭취에 주의\n-과다 섭취 시 일시적으로 피부가 황색으로 변할 수 있음",20,10,mg/일,NaN,제2018-4호,루테인지아잔틴복합추출물,루테인/지아자틴,...,0,0,0,0,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 1, 'P2': 1, 'P1': 0, 'INFANT_TODDLER': 1, 'KIDS': 1, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",90
349,루테인지아잔틴추출복합물(제2022-29호

In [ ]:
# 모두 안지오랩 회사 제품 -> 개별인정원료 '레몬 밤 추출물 혼합분말'
check_p_ids = merge_recom_name_df_2[merge_recom_name_df_2['ingredient_recom_name'].str.contains('오비엑스')]['product_id'].tolist()
cleaning_p_df[cleaning_p_df['product_id'].isin(check_p_ids)]

# 로즈힙 분말 관련 개별인정원료 모두 포함
cleaning_p_df[(cleaning_p_df['product_id'] == 20040015107596) | (cleaning_p_df['product_id'] == 20040015110535)]

# 표고버섯균사체추출물(제2009-3호)에 해당
cleaning_p_df[(cleaning_p_df['product_id'] == 20120019007158)]

# 발효 울금 -> 인정 취소 
cleaning_p_df[(cleaning_p_df['product_id'] == 20040017025341)]

# 식물스테롤에스테르 -> 식물스타놀 에스테르(제2014-32호), 식물스타놀에스테르(제2006-11호)
cleaning_p_df[(cleaning_p_df['product_id'] == 20040020006881)]

# 감초추출물 -> 스페인감초추출물(제2019-7호), 스페인감초추출물(제2014-4호)
cleaning_p_df[(cleaning_p_df['product_id'] == 20040020013279)]

,product_id,product_name,product_company,product_instruction,product_serving,product_dispos,product_storage_method,product_function_content,product_caution,product_image,...,MALE,단백질,비타민 A,베타카로틴,요오드,function_code,product_caution_code,function_code_str,product_caution_code_str,ingredient_id
21566,20040020013279,위내력골드,(주)한국인삼공사,"1일 2회, 1회 1정을 물과 함께 섭취",제조일로부터 2년,[정],,"[위 점막 내 헬리코박터균(Helicobacter pylori) 증식을 억제하고 위 점막을 보호하여 위 건강에 도움을 줄 수 있음, 어두운 곳에서 시각 적응을 위해 필요, 피부와 점막을 형성하고 기능을 유지하는데 필요, 상피세포의 성장과 발달에 필요, 지방, 탄수화물, 단백질 대사와 에너지 생성에 필요]","[임산부, 수유부 섭취에 주의, 간장, 신장, 심장 등의 기능 이상자는 섭취에 주의]",0,...,0,0,0,0,0,"[HF02, HF07, HF08, HF09, HF26]","[DI03, DI04, P3, P2]","HF02, HF07, HF08, HF09, HF26","DI03, DI04, P3, P2","[18, 142, 158]"


- update name dict 수정
    - 곤약감자추출물분말 or 제2008-35호 -> 곤약감자추출물
    - 빌베리주정추출물 or 제2008-22호 -> 기능성원료 빌베리 추출물
    - 동부테아닌 or 제2008-70호 -> 기능성원료 테아닌
    - 가르시니아캄보지아 or 제2009-42호 -> 가르시니아캄보지아추출물
    - 일신바나바주정추출분말 or 제2007-19호 -> 기능성원료 바나바잎 추출물
    - 디메틸설폰 or 엠에스엠 or Methylsulfonylmethane or MSM or Dimethylsulfone -> 엠에스엠
    - 아세틸글루코사민 or NAG or 엔에이지 or Acetylglucosamine -> NAG
    - 차전자피 -> 차전자피식이섬유
    - 키토산 -> 키토산/키토올리고당
    - 이눌린 or 치커리 -> 이눌린/치커리추출물
    - 오비엑스 -> 레몬 밤 추출물 혼합분말
    - 히알우론산 or 히알루론산 나트륨 -> 히알루론산
    - 비즈왁스알코올 or BeesWaxAlcohol or BWA -> 비즈왁스알코올
    - palmitoyl or PLAG or 피엘에이지 -> 피엘에이지
    - 정어리펩타이드 -> 정어리펩타이드 SP100N
    - 와일드망고 -> 아프리카망고 종자추출물
    - 리포젠 or 공액리놀레산 or Tonalin or CLA FFA -> 공액리놀레산
    - 이소플라본 -> 대두이소플라본
    - 아르지닌 -> L-아르기닌
    - 표고버섯균사체추출물
    - 대두배아 -> APIC 대두배아열수추출물 등 복합물
    - 풋사과 -> 풋사과추출물 애플페논
    - 표고버섯균사체추출물 -> 표고버섯균사체추출물
    
- 인정 폐지 혹은 취소(결측치로 남겨두고 삭제 예정)
    - L글루타민 -> 인정 폐지
    - 피니톨 분말 -> 인정 취소
    - 발효울금 -> 인정 취소
    
- API에 존재 X
    - PME-88 메론추출물
    - 정제수
    - 벌꿀
    - 루테인복합물VBAF(제2009-51호)
    
- 개별 하드코딩 필요
    - 루테인, 루테인지아잔틴복합추출물20 
    - 로즈힙 분말
    - 식물스테롤에스테르
    - 감초추출물

### 확인된 내용 하드코딩 후 다시 `ingredient_id` 부여

In [ ]:
# name 관련 dict 업데이트
update_ingredient_grp_name_dict = {'자일로올리고당' : ['자일로올리고당'],
                                   '오메가3' : ['오메가-3', 'EPA'],
                                   '밀크씨슬' : ['밀크씨슬'],
                                   '셀레늄/셀렌' : ['셀레늄'],
                                   '그린마테 추출물' : ['Green Mate Extract', '그린마떼'],
                                   '홍삼' : ['홍삼농축액'],
                                   '코엔자임Q10' : ['코엔자임', '2008-36'],
                                   '곤약감자추출물' : ['곤약감자'],
                                   '빌베리 추출물' : ['빌베리주정추출물'],
                                   '테아닌' : ['동부테아닌'],
                                   '가르시니아캄보지아 추출물' : ['가르시니아'],
                                   '바나바잎 추출물' : ['바나바'],
                                   '엠에스엠' : ['디메틸설폰', '엠에스엠', 'Methyl', 'sulfonylmethane', 'MSM', 'Dimethylsulfone'],
                                   'NAG' : ['아세틸글루코사민', 'NAG', '엔에이지', 'Acetylglucosamine'],
                                   '차전자피식이섬유' : ['차전자피'],
                                   '키토산/키토올리고당' : ['키토산'],
                                   '이눌린/치커리추출물' : ['이눌린', '치커리'],
                                   '레몬 밤 추출물 혼합분말' : ['오비엑스'],
                                   '히알루론산' : ['히알우론산', '히알루론산'],
                                   '비즈왁스알코올' : ['비즈왁스알코올', 'BeesWaxAlcohol', 'BWA'],
                                   '피엘에이지' : ['palmitoyl', 'PLAG', '피엘에이지'],
                                   '정어리펩타이드 SP100N' : ['정어리펩타이드'],
                                   '아프리카망고 종자추출물' : ['망고'],
                                   '공액리놀레산' : ['리포젠', '공액리놀레산', 'Tonalin', 'CLA FFA'],
                                   '대두이소플라본' : ['이소플라본'],
                                   'L-아르기닌' : ['아르지닌'],
                                   'APIC 대두배아열수추출물 등 복합물' : ['대두배아'],
                                   '풋사과추출물 애플페논' : ['풋사과'],
                                   '표고버섯균사체추출물' : ['표고버섯균사체추출물']
                                   }

In [ ]:
updated_recom_name_df = create_ingredient_grp_name(merge_recom_name_df_2, 
                                    'ingredient_name', 
                                    'ingredient_grp_name', 
                                    update_ingredient_grp_name_dict,
                                    'ingredient_bracket_str'
                                    )

# 특수문자 및 공백문자 제거
updated_recom_name_df['ingredient_recom_name'] = updated_recom_name_df['ingredient_grp_name'].apply(lambda x: re.sub(r'[^가-힣A-Za-z0-9]', '', x))

# 확인
updated_recom_name_df

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name
111,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인
112,20040020006948,디메틸설폰(Methyl sulfonylmethane,NaN,디메틸설폰(Methyl sulfonylmethane,엠에스엠,NaN,엠에스엠
113,20040020006948,MSM) 제품,NaN,MSM),엠에스엠,NaN,엠에스엠
142,200400200071853,루테인,NaN,루테인,루테인,NaN,루테인
150,200400200071272,N-아세틸글루코사민 제품,NaN,N-아세틸글루코사민,NAG,NaN,NAG
...,...,...,...,...,...,...,...
156029,200400200071784,피니톨 분말(기능성원료인정제2014-65호),제2014-65호,피니톨 분말,피니톨 분말,NaN,피니톨분말
156030,2004001710665,동우코엔자임Q10(제2007-14호. 제2008-45호),제2007-14호제2008-45호,동우코엔자임Q10,코엔자임Q10,NaN,코엔자임Q10
156031,20040020006514,코엔자임큐텐(Co Q10)(제2008-63호),제2008-63호,코엔자임큐텐(Co Q10),코엔자임Q10,(Co Q10),코엔자임Q10
156034,201100200994,동부테아닌(제2008-70호),제2008-70호,동부테아닌,테아닌,NaN,테아닌


In [ ]:
# ingredient id 컬럼 제거
updated_recom_name_df.drop(columns='ingredient_id', inplace=True)

KeyError: "['ingredient_id'] not found in axis"

In [ ]:
# 다시 ingredient_recom_name 컬럼으로 merge
merge_recom_name_df = pd.merge(updated_recom_name_df , i_df[['ingredient_id', 'ingredient_recom_name']], how='left', on='ingredient_recom_name')

# ingredient_id 존재하는 행 확인
merge_recom_name_df_3 = merge_recom_name_df[merge_recom_name_df['ingredient_id'].notnull()] # 4295 행
merge_recom_name_df_3

# ingredient_id 결측치인 행 확인
merge_recom_name_df_4 = merge_recom_name_df[merge_recom_name_df['ingredient_id'].isnull()] # 159 행
merge_recom_name_df_4

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
0,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,NaN
3,200400200071853,루테인,NaN,루테인,루테인,NaN,루테인,NaN
45,2004001606132,루테인 제품,NaN,루테인,루테인,NaN,루테인,NaN
272,20040015083386,PME 88 메론 추출물,NaN,PME 88 메론 추출물,PME 88 메론 추출물,NaN,PME88메론추출물,NaN
273,2004001536219,PME 88 메론 추출물,NaN,PME 88 메론 추출물,PME 88 메론 추출물,NaN,PME88메론추출물,NaN
...,...,...,...,...,...,...,...,...
4371,202300010424,정제수,NaN,정제수,정제수,NaN,정제수,NaN
4434,2004001900928,L-글루타민(제2008-7호),제2008-7호,L-글루타민,L-글루타민,NaN,L글루타민,NaN
4435,20040020006615,루테인복합물VBAF(제2009-51호),제2009-51호,루테인복합물VBAF,루테인복합물VBAF,NaN,루테인복합물VBAF,NaN
4448,2019000948341,피니톨분말(제2014-65호),제2014-65호,피니톨분말,피니톨분말,NaN,피니톨분말,NaN


In [ ]:
# 결측값을 가질 행 데이터 제거
nan_check_list = ['PME88메론추출물', '정제수', '벌꿀', '루테인복합물VBAF', 'L글루타민', '피니톨분말', '발효울금']
drop_nan_idxs = merge_recom_name_df_4[merge_recom_name_df_4['ingredient_recom_name'].isin(nan_check_list)].index.tolist()
print(len(drop_nan_idxs))

# 해당 행 제거
merge_recom_name_df_4.drop(index=drop_nan_idxs, inplace=True)

### 개별 하드코딩 진행

In [ ]:
# 1. 로즈힙 -> 로즈힙 분말
i_ids_1 = re.sub('[\[\]]+', '', str(i_df[i_df['ingredient_recom_name']=='로즈힙분말']['ingredient_id'].to_list())) # '299, 238, 250, 263'

merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('로즈힙'), 'ingredient_recom_name'] = '로즈힙분말'
merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('로즈힙'), 'ingredient_id'] = str(i_ids_1)
merge_recom_name_df_4[merge_recom_name_df_4['ingredient_recom_name'].str.contains('로즈힙')]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
1817,20040015107596,로즈힙(Rosa Canina),NaN,로즈힙(Rosa Canina),로즈힙,(Rosa Canina),로즈힙분말,"299, 238, 250, 263"
1844,20040015110535,로즈힙(Rosa Canina),NaN,로즈힙(Rosa Canina),로즈힙,(Rosa Canina),로즈힙분말,"299, 238, 250, 263"


In [ ]:
# 2. 식물스테롤에스테르 -> '식물스타놀에스테르'
i_ids_2 = re.sub('[\[\]]+', '', str(i_df[i_df['ingredient_recom_name']=='식물스타놀에스테르']['ingredient_id'].to_list())) # '402, 311'

merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('식물'), 'ingredient_recom_name'] = '식물스테롤에스테르'
merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('식물'), 'ingredient_id'] = str(i_ids_2)
merge_recom_name_df_4[merge_recom_name_df_4['ingredient_recom_name'].str.contains('식물')]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
1786,20040020006881,식물스테롤에스테르 제품,NaN,식물스테롤에스테르,식물스테롤에스테르,NaN,식물스테롤에스테르,"402, 311"


In [ ]:
# 3. 감초추출물 -> '스페인감초추출물'
i_ids_3 = re.sub('[\[\]]+', '', str(i_df[i_df['ingredient_recom_name']=='스페인감초추출물']['ingredient_id'].to_list()))
merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('감초'), 'ingredient_recom_name'] = '스페인감초추출물'
merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('감초'), 'ingredient_id'] = str(i_ids_3)
merge_recom_name_df_4[merge_recom_name_df_4['ingredient_recom_name'].str.contains('감초')]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
3255,20040020013279,감초추출물,NaN,감초추출물,감초추출물,NaN,스페인감초추출물,"328, 158"


In [ ]:
# 4. 루테인지아잔틴복합추출물20
i_ids_4 = re.sub('[\[\]]+', '', str(i_df[i_df['ingredient_name'].str.contains('루테인지아잔틴복합추출물20')]['ingredient_id'].to_list()))
merge_recom_name_df_4.loc[merge_recom_name_df_4['ingredient_recom_name'].str.contains('루테인지아잔틴복합추출물20'), 'ingredient_id'] = i_ids_4
merge_recom_name_df_4[merge_recom_name_df_4['ingredient_recom_name'].str.contains('루테인지아잔틴복합추출물20')]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
1668,20060020008210,루테인지아잔틴복합추출물20%,NaN,루테인지아잔틴복합추출물20%,루테인지아잔틴복합추출물20%,NaN,루테인지아잔틴복합추출물20,195


In [ ]:
# 5. 루테인
i_ids_5 = re.sub('[\[\]]+', '', str(i_df[i_df['ingredient_name'].str.contains('루테인')]['ingredient_id'].to_list()))
check_mask = (merge_recom_name_df_4['ingredient_id'].isnull()) & (merge_recom_name_df_4['ingredient_recom_name'].str.contains('루테인'))
merge_recom_name_df_4.loc[check_mask, 'ingredient_id'] = i_ids_5
merge_recom_name_df_4[check_mask]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
0,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
3,200400200071853,루테인,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
45,2004001606132,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
339,20040015083817,루테인,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
340,2013002000883,루테인,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
...,...,...,...,...,...,...,...,...
4207,20040015083541,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
4208,20040015083401,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
4211,20040015083398,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"
4254,20040020006466,루테인 제품,NaN,루테인,루테인,NaN,루테인,"289, 195, 180, 90, 79, 74, 66, 13"


In [ ]:
# ','를 기준으로 행 나누기
merge_recom_name_df_5 = normalize_table(merge_recom_name_df_4, 'ingredient_id')

# ingredient id str -> int 타입 변경
merge_recom_name_df_5['ingredient_id'] = merge_recom_name_df_5['ingredient_id'].astype(int)

merge_recom_name_df_5

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id
0,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,289
1,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,195
2,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,180
3,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,90
4,20040020006811,루테인 제품,NaN,루테인,루테인,NaN,루테인,79
...,...,...,...,...,...,...,...,...
1144,20040015162111,루테인,NaN,루테인,루테인,NaN,루테인,90
1145,20040015162111,루테인,NaN,루테인,루테인,NaN,루테인,79
1146,20040015162111,루테인,NaN,루테인,루테인,NaN,루테인,74
1147,20040015162111,루테인,NaN,루테인,루테인,NaN,루테인,66


## `product_ingredient_table 생성`

### `ingredient_id` 값 부여한 데이터프레임 모두 합치기

In [ ]:
product_ingredient_table = pd.concat([merge_auth_num_df_1, merge_recom_name_df_1, merge_recom_name_df_3,merge_recom_name_df_5], axis=0, ignore_index=True)

# 결측치 없는 것을 확인
product_ingredient_table[product_ingredient_table['ingredient_id'].isnull()]

,product_id,ingredient_raw_name,ingredient_auth_num,ingredient_name,ingredient_grp_name,ingredient_bracket_str,ingredient_recom_name,ingredient_id


In [ ]:
# float -> int 타입변경
product_ingredient_table['ingredient_id'] = product_ingredient_table['ingredient_id'].astype(int)

In [ ]:
# 필요한 컬럼만 남기기
product_ingredient_table = product_ingredient_table[['product_id', 'ingredient_id']]

product_ingredient_table

,product_id,ingredient_id
0,200400200061442,342
1,20040017059172,176
2,20040017106325,276
3,20040016020168,276
4,200400200081854,158
...,...,...
159899,20040015162111,90
159900,20040015162111,79
159901,20040015162111,74
159902,20040015162111,66


In [ ]:
len(product_ingredient_table)

159904

range(1, 159904)

### `product_ingredient_id` 생성

In [ ]:
product_ingredient_table['product_ingredient_id'] = range(1, len(product_ingredient_table) + 1)

product_ingredient_table = product_ingredient_table[['product_ingredient_id', 'product_id', 'ingredient_id']]
product_ingredient_table

,product_ingredient_id,product_id,ingredient_id
0,1,200400200061442,342
1,2,20040017059172,176
2,3,20040017106325,276
3,4,20040016020168,276
4,5,200400200081854,158
...,...,...,...
159899,159900,20040015162111,90
159900,159901,20040015162111,79
159901,159902,20040015162111,74
159902,159903,20040015162111,66


## product 테이블의 `product_ingredient_id` 컬럼 생성

In [ ]:
# product_id로 그룹화하여 각 그룹의 ingredient_id 값을 리스트로 변환
product_ingredient_dict = product_ingredient_table.groupby('product_id')['ingredient_id'].apply(list).to_dict()
product_ingredient_dict

{200400150395: [355, 423, 140, 152, 142],
 200400150396: [355, 140, 423, 152, 142],
 200400150703: [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6],
 200400150704: [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6],
 200400150708: [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6],
 200400150709: [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6],
 200400151042: [369],
 200400151048: [423],
 200400151072: [181],
 200400151074: [423],
 200400151627: [17],
 200400151628: [181, 343],
 200400151897: [419],
 200400152012: [20],
 200400152925: [340, 153, 102],
 200400152926: [340, 153, 102],
 200400153622: [369],
 200400153626: [16],
 200400153629: [423],
 200400160314: [418],
 200400160315: [418],
 200400160316: [418],
 200400160323: [368],
 200400170117: [357],
 200400170132: [362, 423],
 200400170297: [369],
 200400170303: [419],
 200400170307: [419],
 200400170385: [419],
 200400170541: [183, 18, 289, 195, 180, 90, 79, 74, 66, 13],
 200400170542: [389, 362],
 200400170543: [136, 150, 2

In [ ]:
# product_ingredient_dict의 key, value를 이용해 temp 데이터프레임 생성
temp_p_df = pd.DataFrame({'product_id' : list(product_ingredient_dict.keys()),
                          'product_ingredient_id' : list(product_ingredient_dict.values())
                          })

temp_p_df

,product_id,product_ingredient_id
0,200400150395,"[355, 423, 140, 152, 142]"
1,200400150396,"[355, 140, 423, 152, 142]"
2,200400150703,"[277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]"
3,200400150704,"[277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]"
4,200400150708,"[277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]"
...,...,...
32932,201000190041382,"[419, 152, 140, 355, 424, 141, 142, 21, 18, 362, 428, 392]"
32933,201000190041383,"[392, 428]"
32934,201000190041384,"[118, 14]"
32935,201000190041385,[428]


In [ ]:
# 'product_ingredient_id' 컬럼을 JSON 컬럼으로 만들기 위해 dict화
temp_p_df['product_ingredient_id'] = temp_p_df['product_ingredient_id'].apply(lambda x : {'ingredient_id' : x})

temp_p_df

,product_id,product_ingredient_id
0,200400150395,"{'ingredient_id': [355, 423, 140, 152, 142]}"
1,200400150396,"{'ingredient_id': [355, 140, 423, 152, 142]}"
2,200400150703,"{'ingredient_id': [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]}"
3,200400150704,"{'ingredient_id': [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]}"
4,200400150708,"{'ingredient_id': [277, 151, 107, 68, 56, 41, 40, 38, 33, 29, 23, 6]}"
...,...,...
32932,201000190041382,"{'ingredient_id': [419, 152, 140, 355, 424, 141, 142, 21, 18, 362, 428, 392]}"
32933,201000190041383,"{'ingredient_id': [392, 428]}"
32934,201000190041384,"{'ingredient_id': [118, 14]}"
32935,201000190041385,{'ingredient_id': [428]}


In [ ]:
# 기존 product 데이터프레임과 merge
fin_p_df = pd.merge(p_df, temp_p_df, how='left', on='product_id')
fin_p_df.head()

,product_id,product_function,product_caution,ingredient_type1,ingredient_type2,product_function_content,product_caution_content,function_nonus,caution_nonus,HF00,...,O3,O2,O1,단백질,비타민 A,베타카로틴,요오드,product_caution_code,ingredient_raw_name,product_ingredient_id
0,200400200021640,혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.,"임산부, 수유부, 청소년 및 어린이는 섭취를 피하십시오.\n심혈관약 등 복용, 혈액 응고장애가 있거나 수술예정인 사람은 전문가와 상담하십시오. \n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n본 제품은 섭취자의 신체상태에 따라 반응에 차이가 있을 수 있습니다.\n임산부, 수유부, 질병으로 인해 치료중인 분은 섭취 전 고객센터나 구입처로 문의하십시오.\n특정 성분에 알레르기 체질이신 분은 섭취 전에 반드시 원료(성분)를 확인하신 후 섭취하시기 바랍니다.\n섭취량 및 섭취방법을 확인 한 후 섭취하시고, 기타 제품관련사항은 고객센터로 문의해 주십시오.",폴리코사놀-사탕수수 왁스알코올(2006-4),폴리코사놀-사탕수수왁스알코올(기능성원료인정제2006-4호),혈중 콜레스테롤 수치 개선에 도움을 줄 수 있음.,"임산부, 수유부, 청소년 및 어린이는 섭취를 피하십시오.\n심혈관약 등 복용, 혈액 응고장애가 있거나 수술예정인 사람은 전문가와 상담하십시오.\n이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n본 제품은 섭취자의 신체상태에 따라 반응에 차이가 있을 수 있습니다.\n임산부, 수유부, 질병으로 인해 치료중인 분은 섭취 전 고객센터나 구입처로 문의하십시오.\n특정 성분에 알레르기 체질이신 분은 섭취 전에 반드시 원료(성분)를 확인하신 후 섭취하시기 바랍니다.\n섭취량 및 섭취방법을 확인 한 후 섭취하시고, 기타 제품관련사항은 고객센터로 문의해 주십시오.",혈중 콜레스테롤 수치 개선,임산부 수유부 청소년 어린이 섭취 피하 심혈관약 복용 혈액 응고장애 수술예정인 사람은 전문가 상담 이상사례 발생 섭취 중단 전문가 상담 제품 섭취자의 신체상태 반응 차이 임산부 수유부 질병 치료중인 분 섭취 전 고객센터 구입처 문의 특정 성분 알레르기 체질이 분 섭취 전 원료 성분 확인 후 섭취하시기 섭취량 섭취방법 확인 후 섭취하시 기타 제품관련사항 고객센터 문의해,0,...,1,1,1,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 1, 'DI04': 0, 'DI05': 0, 'DI06': 0, 'DI07': 0, 'DI08': 0, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 0, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 1, 'P2': 1, 'P1': 0, 'INFANT_TODDLER': 1, 'KIDS': 1, 'TEENAGER': 1, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 1, 'O2': 1, 'O1': 1, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",폴리코사놀-사탕수수 왁스알코올(2006-4),{'ingredient_id': [110]}
1,201100200015,①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선⑤항산화에 도움을 줄 수 있음,"[홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\r\n2) 특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.\r\n 3) 15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.\r\n 4) 제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의 하십시오.",홍삼제품,홍삼추출 농축액(농축추출물),면역력 증진\n피로개선\n혈소판 응집 억제를 통한 혈액흐름에 도움\n기억력 개선\n항산화에 도움을 줄 수 있음,"홍삼제품\n의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의\n특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.\n15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.\n제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의 하십시오.",면역력 증진 피로개선 혈소판 응집 억제 혈액흐름 기억력 개선 항산화,홍삼제품 의약품 당뇨치료제 혈액항응고제 복용 섭취 주의 특이체질등 알레르기 체질 경우 제품성분 확인 후 섭취하시기 15세 이하 어린이 상기 섭취량 절반 정도 섭취하시요 제품 개봉 섭취시 포장재 상처 있으니주 하십시오,0,...,0,0,0,0,0,0,0,"{'DI01': 0, 'DI02': 0, 'DI03': 0, 'DI04': 0, 'DI05': 1, 'DI06': 0, 'DI07': 0, 'DI08': 1, 'DI09': 0, 'DI10': 0, 'DI11': 0, 'DI12': 0, 'DI13': 0, 'DI14': 0, 'DI15': 0, 'DI16': 0, 'DI17': 0, 'AL01': 0, 'AL02': 0, 'AL03': 0, 'AL04': 0, 'AL05': 0, 'AL06': 0, 'AL07': 0, 'AL08': 0, 'AL09': 0, 'AL10': 0, 'AL11': 1, 'AL12': 0, 'AL13': 0, 'AL14': 0, 'AL15': 0, 'AL16': 0, 'AL17': 0, 'AL18': 0, 'AL19': 0, 'AL20': 0, 'P3': 0, 'P2': 0, 'P1': 0, 'INFANT_TODDLER': 1, 'KIDS': 1, 'TEENAGER': 0, 'ADULT': 0, 'OLD': 0, 'FEMALE': 0, 'MALE': 0, 'S1': 0, 'O3': 0, 'O2': 0, 'O1': 0, '단백질': 0, '비타민 A': 0, '베타카로틴': 0, '요오드': 0}",홍삼제품,{'ingredient_id': [419]}
2,20040015100230,⑴ 면역력 증진에 도움을 줄 수 있음\n ⑵ 피로개선에 도움을 줄 수 있음\n ⑶ 혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n ⑷ 기억력 개선에 도움을 줄 수 있음\n ⑸ 항산화에 도움을 줄 수 있음,"⑴ 특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n⑵ 제품 개봉 또는 섭취 시 포장재에 의한 상처에 주의하시기 바랍니다.\n⑶ 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의 하십시오.\n⑷ 유통기한이 경과된 제품은 섭취하지 마십시오.",홍삼,"홍삼, 6년근(고시형)",면역력 증진에 도움을 줄 수 있음\n피로개선에 도움을 줄 수 있음\n혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n기억력 개선에 도움을 줄 수 있음\n항산화에 도움을 줄 수 있음,"특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n제품 개봉 또는 섭취 시 포장재에 의한 상처에 주의하시기 바랍니다.\n의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의 하십시오.\n유통기한이 경과된 제품은 섭취하지 마십시오.",면역력 증진 피로개선 혈소판 응집 억제 혈액흐름 기억력 개선 항산화,특수체질 알레르기 체질인 경우 섭취 전 원료명 함량 확인하시기 제품 개봉 섭취 포장재 상처 주의하시기 의약품 당뇨치료제 혈액항응고제 복용 섭취 주의 하십시오 유통기한 경과 제품 섭취,0,...,0,0,0,0,0,0,0,"{'DI01': 0

## product 테이블 생성

In [ ]:
# 기존 작업된 product 테이블과 추가 전처리한 컬럼 합치기
product_table = pd.merge(cleaning_p_df[['product_id', 'product_name', 'product_company', 'product_instruction', 'product_image',
                                        'product_storage_method', 'product_dispos', 'product_serving', 'product_rating_avg', 'product_rating_cnt']], 
                         fin_p_df[['product_id', 'product_function_content', 'product_caution_content',
                                   'product_function_code', 'product_caution_code', 'product_ingredient_id']], 
                         how='left',
                         on='product_id')

In [ ]:
# JSON 컬럼 변환
for code_col in ['product_function_code', 'product_caution_code', 'product_ingredient_id']:
    product_table[code_col] = product_table[code_col].astype(str)
    product_table[code_col] = product_table[code_col].apply(lambda x: json.dumps(x, ensure_ascii=False))
    
# 변환 함수 정의
def convert_quotes(s):
    # 내부의 작은따옴표를 큰따옴표로 변환
    s = s.replace("'", '"')
    # 앞뒤의 큰따옴표를 작은따옴표로 변환 및 앞뒤 작은 따옴표 제거
    if s.startswith('"') and s.endswith('"'):
        s = s[1:-1]
    return s

# 함수 적용
for code_col in ['product_function_code', 'product_caution_code', 'product_ingredient_id']:
    product_table[code_col] = product_table[code_col].apply(convert_quotes)

In [ ]:
product_table[['product_function_code', 'product_caution_code', 'product_ingredient_id']].head()

,product_function_code,product_caution_code,product_ingredient_id
0,"{""HF00"": 0, ""HF01"": 0, ""HF02"": 0, ""HF03"": 0, ""HF04"": 0, ""HF05"": 0, ""HF06"": 0, ""HF07"": 0, ""HF08"": 0, ""HF09"": 0, ""HF10"": 0, ""HF11"": 0, ""HF12"": 0, ""HF13"": 0, ""HF14"": 0, ""HF15"": 0, ""HF16"": 0, ""HF17"": 0, ""HF18"": 0, ""HF19"": 0, ""HF20"": 0, ""HF21"": 0, ""HF22"": 0, ""HF23"": 0, ""HF24"": 1, ""HF25"": 0, ""HF26"": 0}","{""DI01"": 0, ""DI02"": 0, ""DI03"": 1, ""DI04"": 0, ""DI05"": 0, ""DI06"": 0, ""DI07"": 0, ""DI08"": 0, ""DI09"": 0, ""DI10"": 0, ""DI11"": 0, ""DI12"": 0, ""DI13"": 0, ""DI14"": 0, ""DI15"": 0, ""DI16"": 0, ""DI17"": 0, ""AL01"": 0, ""AL02"": 0, ""AL03"": 0, ""AL04"": 0, ""AL05"": 0, ""AL06"": 0, ""AL07"": 0, ""AL08"": 0, ""AL09"": 0, ""AL10"": 0, ""AL11"": 0, ""AL12"": 0, ""AL13"": 0, ""AL14"": 0, ""AL15"": 0, ""AL16"": 0, ""AL17"": 0, ""AL18"": 0, ""AL19"": 0, ""AL20"": 0, ""P3"": 1, ""P2"": 1, ""P1"": 0, ""INFANT_TODDLER"": 1, ""KIDS"": 1, ""TEENAGER"": 1, ""ADULT"": 0, ""OLD"": 0, ""FEMALE"": 0, ""MALE"": 0, ""S1"": 0, ""O3"": 1, ""O2"": 1, ""O1"": 1, ""단백질"": 0, ""비타민 A"": 0, ""베타카로틴"": 0, ""요오드"": 0}","{""ingredient_id"": [110]}"
1,"{""HF00"": 0, ""HF01"": 0, ""HF02"": 1, ""HF03"": 0, ""HF04"": 0, ""HF05"": 1, ""HF06"": 1, ""HF07"": 0, ""HF08"": 0, ""HF09"": 0, ""HF10"": 0, ""HF11"": 0, ""HF12"": 0, ""HF13"": 0, ""HF14"": 0, ""HF15"": 0, ""HF16"": 0, ""HF17"": 0, ""HF18"": 0, ""HF19"": 0, ""HF20"": 1, ""HF21"": 0, ""HF22"": 0, ""HF23"": 0, ""HF24"": 1, ""HF25"": 0, ""HF26"": 0}","{""DI01"": 0, ""DI02"": 0, ""DI03"": 0, ""DI04"": 0, ""DI05"": 1, ""DI06"": 0, ""DI07"": 0, ""DI08"": 1, ""DI09"": 0, ""DI10"": 0, ""DI11"": 0, ""DI12"": 0, ""DI13"": 0, ""DI14"": 0, ""DI15"": 0, ""DI16"": 0, ""DI17"": 0, ""AL01"": 0, ""AL02"": 0, ""AL03"": 0, ""AL04"": 0, ""AL05"": 0, ""AL06"": 0, ""AL07"": 0, ""AL08"": 0, ""AL09"": 0, ""AL10"": 0, ""AL11"": 1, ""AL12"": 0, ""AL13"": 0, ""AL14"": 0, ""AL15"": 0, ""AL16"": 0, ""AL17"": 0, ""AL18"": 0, ""AL19"": 0, ""AL20"": 0, ""P3"": 0, ""P2"": 0, ""P1"": 0, ""INFANT_TODDLER"": 1, ""KIDS"": 1, ""TEENAGER"": 0, ""ADULT"": 0, ""OLD"": 0, ""FEMALE"": 0, ""MALE"": 0, ""S1"": 0, ""O3"": 0, ""O2"": 0, ""O1"": 0, ""단백질"": 0, ""비타민 A"": 0, ""베타카로틴"": 0, ""요오드"": 0}","{""ingredient_id"": [419]}"
2,"{""HF00"": 0, ""HF01"": 0, ""HF02"": 1, ""HF03"": 0, ""HF04"": 0, ""HF05"": 1, ""HF06"": 1, ""HF07"": 0, ""HF08"": 0, ""HF09"": 0, ""HF10"": 0, ""HF11"": 0, ""HF12"": 0, ""HF13"": 0, ""HF14"": 0, ""HF15"": 0, ""HF16"": 0, ""HF17"": 0, ""HF18"": 0, ""HF19"": 0, ""HF20"": 1, ""HF21"": 0, ""HF22"": 0, ""HF23"": 0, ""HF24"": 1, ""HF25"": 0, ""HF26"": 0}","{""DI01"": 0, ""DI02"": 0, ""DI03"": 0, ""DI04"": 0, ""DI05"": 1, ""DI06"": 0, ""DI07"": 0, ""DI08"": 1, ""DI09"": 0, ""DI10"": 0, ""DI11"": 0, ""DI12"": 0, ""DI13"": 0, ""DI14"": 0, ""DI15"": 0, ""DI16"": 0, ""DI17"": 0, ""AL01"": 0, ""AL02"": 0, ""AL03"": 0, ""AL04"": 0, ""AL05"": 0, ""AL06"": 0, ""AL07"": 0, ""AL08"": 0, ""AL09"": 0, ""AL10"": 0, ""AL11"": 0, ""AL12"": 0, ""AL13"": 0, ""AL14"": 0, ""AL15"": 0, ""AL16"": 0, ""AL17"": 0, ""AL18"": 0, ""AL19"": 0, ""AL20"": 0, ""P3"": 0, ""P2"": 0, ""P1"": 0, ""INFANT_TODDLER"": 0, ""KIDS"": 0, ""TEENAGER"": 0, ""ADULT"": 0, ""OLD"": 0, ""FEMALE"": 0, ""MALE"": 0, ""S1"": 0, ""O3"": 0, ""O2"": 0, ""O1"": 0, ""단백질"": 0, ""비타민 A"": 0, ""베타카로틴"": 0, ""요오드"": 0}","{""ingredient_id"": [419]}"
3,"{""HF00"": 0, ""HF01"": 0, ""HF02"": 0, ""HF03"": 0, ""HF04"": 0, ""HF05"": 0, ""HF06"": 1, ""HF07"": 0, ""HF08"": 0, ""HF09"": 0, ""HF10"": 0, ""HF11"": 0, ""HF12"": 0, ""HF13"": 0, ""HF14"": 0, ""HF15"": 0, ""HF16"": 0, ""HF17"": 0, ""HF18"": 0, ""HF19"": 0, ""HF20"": 0, ""HF21"": 0, ""HF22"": 0, ""HF23"": 0, ""HF24"": 1, ""HF25"": 0, ""HF26"": 0}","{""DI01"": 0, ""DI02"": 0, ""DI03"": 0, ""DI04"": 0, ""DI05"": 0, ""DI06"": 0, ""DI07"": 0, ""DI08"": 0, ""DI09"": 0, ""DI10"": 0, ""DI11"": 0, ""DI12"": 0, ""DI13"": 0, ""DI14"": 0, ""DI15"": 0, ""DI16"": 0, ""DI17"": 0, ""AL01"": 0, ""AL02"": 0, ""AL03"": 0, "

## 피클 파일로 저장

### product_ingredient 테이블

In [ ]:
product_ingredient_table.columns

Index(['product_ingredient_id', 'product_id', 'ingredient_id'], dtype='object')

### product 테이블

In [ ]:
product_table.columns

Index(['product_id', 'product_name', 'product_company', 'product_instruction',
       'product_image', 'product_storage_method', 'product_dispos',
       'product_serving', 'product_rating_avg', 'product_rating_cnt',
       'product_function_content', 'product_caution_content',
       'product_function_code', 'product_caution_code',
       'product_ingredient_id'],
      dtype='object')

In [ ]:
# 피클 파일 및 csv 파일로 저장

# 피클 파일 저장
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_ingredient_table.pkl', 'wb') as f:
    pickle.dump(product_ingredient_table, f)    
    
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_table.pkl', 'wb') as f:
    pickle.dump(product_table, f)    
    
with open('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_processing.pkl', 'wb') as f:
    pickle.dump(fin_p_df, f)
    
# csv 파일 저장
product_ingredient_table.to_csv('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_ingredient_table.csv', index=False, encoding='utf-8-sig')

product_table.to_csv('C:\\Users\\user\\working\\Our-family-pharmacist\\data\\product_table.csv', index=False, encoding='utf-8-sig')